In [93]:
import pandas as pd
import numpy as np
import ast
import re
import os
import pickle
import torch
import torch.nn.functional as F
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel , XLNetTokenizer, XLNetConfig , XLNetModel

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch import nn



In [94]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")
train_df = train_df.fillna('None')
test_df = pd.read_csv("./tbrain/tbrain_test.csv")
test_df = test_df.fillna('None')
print(train_df.shape)
print(test_df.shape)

true_df = train_df[train_df['name']!='[]']
print(true_df.shape)

(4426, 4)
(491, 4)
(335, 4)


In [95]:
true_df

,news_ID,name,full_content,ckip_names
6,1984,"['鄭銘富', '呂家緯', '李春生', '黃建強', '周正華', '高振殷']",鄭銘富走私運毒集團將K他命藏在人造石板，從大陸廈門港裝貨櫃運抵基隆港，去年5月29日遭檢調查...,"['於鄭', '高振殷', '鄭', '鄭銘富', '周正華', '呂家緯', '高男', ..."
30,4775,"['林若蕎', '林耿宏']",記者王家珩、黃大風／台北報導\n主嫌林耿宏是集團的精神領袖，經常在臉書上PO跑車，或買名牌的...,"['林若蕎', '王家珩', '萬霖福瑞麒', '黃大風', '林耿宏', '耿宏']"
67,1093,"['何智輝', '王素筠', '張貞松', '翁德銘']",前立委何智輝因新竹科學園區銅鑼基地等案，向國華人壽詐貸4億5000萬元，貪汙罪部分一審將他判...,"['久俊案', '翁德銘', '國華', '王素筠', '何男', '張貞松', '何智輝']"
77,4162,['趙宏翰'],涉貪圖利 東檢聲押前台東大武鄉長趙宏翰\n前台東大武鄉長趙宏翰涉貪圖利，遭東檢聲押。（記者陳...,"['柯博齡', '謝慧中', '趙宏翰', '白', '東檢', '黃', '趙', '魏豪..."
84,1067,"['羅雅美', '邱彰信', '黃川禎', '于堯']",（中央社記者林長順台北2日電）台北地檢署偵辦菸品走私案，昨天約談華航前資深副總羅雅美、前空品...,"['邱彰信', '林長順', '邱', '羅', '黃川禎', '唐聲揚', '羅雅美', ..."
...,...,...,...,...
4351,4468,['雷俊玲'],歷史系副教授雷俊玲4年前在 縣橫山鄉山坡地保育區，違法建設面積1128平方公尺的豪華別墅，經...,"['曾', '雷俊玲', '王']"
4359,1757,"['鄭妮娜', '黃明祥', '洪博文']",股票上櫃公司「全詮租賃」2016年底以22億元處分北市西門町好樂迪大樓，預估獲利10.95億...,"['鄭妮娜', '孫慶璞', '楊子', '洪博文', '楊志運', '曾享亮', '黃明祥..."
4361,398,"['林嘉凌', '林茂樹', '陳國帥']",前黑澀會美眉「薔薔」林嘉凌，再爆出吸毒疑雲！檢警接獲線報，指稱北市大直一處豪宅有人辦毒趴，台...,"['林茂樹', '諧星薔', '陳國帥', '紀', '陳', '薔薔', '林嘉凌', '..."
4386,1418,"['吳誌麒', '邱文聰', '溫方瓊']",▲「吳董」賣淫集團遭破獲，首腦遭判刑3年。（圖／資料照）\n\n記者許力方／綜合報導\n移民...,"['吳誌麒', '溫方瓊', '茜', '雙北', '文哥', '吳董', '丁國琳', '..."


In [96]:
# all real names
online_gen_names = '''李婉湖、馮重禎、黃志宏、洪志軒、林世傑、許怡芬、楊雅智、蔡學希、桑勇志、陳淑合、鄭嘉偉、吳彥廷、李玉芳、林璇喜、鞏雅琪、陳瑩宏、吳淑宸、郭鈺婷、余豪妮、程金士、廖家竹、吳承信、陳怡君、黃威廷、曾雅芬、陳邦映、衛秀玲、鄭仁菱、李惠文、楊姿伶、毛鴻勳、黎定枝、林盈念、吳佳州、曹佐燦、江俊佑、陳柏伸、張瑩韋、張玉婷、朱容聖、周恩銘、趙婉婷、蘇瓊慧、陳惟康、郭雅惠、張姿吟、謝孟君、林淑芬、陳可欣、鮑婷婷、洪白行、楊昀瑜、郭心岑、陳雅純、李建恩、趙欣怡、林靜如、鄭淑梅、王毅軍、劉雲芳、劉桂堅、陳燦儀、蔡惠雯、耿詩涵、簡順幸、陳鈺婷、馬豪琪、林雅雯、林志嘉、傅心怡、陳志峰、陳玄任、謝怡萱、陳佳芳、林建菁、王蘭琳、陳哲宇、黃昱航、劉怡伶、張志茜、李淑謙、林若銘、屠仁欣、黃協泰、梁凱倫、陳靜聖、韓為、傅孟倫、鐘靜如、吳信宏、高若蕙、孫廷群、林大薇、倪雅玲、朱婷婷、林意婷、林禮月、楊文羽、曾淑吟、蕭雅玟、呂育儒、陳冠廷、陳巧恒、陳秀雨、李長諭、賴維白、劉偉正、鄭淑姍、黃淑南、鄧玟君、朱家欣、黃亮禮、林宏偉、陳昭淑、朱明宏、黃筱涵、蔡明伶、張胤義、陳家銘、謝明珠、邱昀丹、張琬婷、宋侑蕙、蔡昌雄、扈國偉、邱婉婷、邱祐旭、林君琴、柳郁文、陳方玉、吳明哲、李上菱、劉孝娟、郭建安、陳慧裕、黃麗雅、陳綠潔、彭佳玲、謝明喜、謝素松、翟淑華、李敏翰、張怡廷、李國桓、高俊明、謝忠吟、張馨和、柳健其、葉明慧、何博賢、趙桂航、呂維倫、王信宏、邊文傑、蔡國偉、陳修如、陳瑋婷、尹俊穎、林彥書、李淑桓、朱瑤萍、劉舒玄、陳思福、趙怡樺、權鈺仰、姚淑慧、張雅萍、吳致軒、黃雅惠、翁淑卿、許文新、林志賢、鐘文月、陳阿元、鄧健旭、劉怡潔、余雅萍、陳俊良、王姿威、李敬宣、陳俊豪、王士弘、張馨孝、葉冠育、黃建正、許淑谷、曾嘉文、黃玉以、黃馨定、胡建安、翁廷貞、楊岱方、張清勇、楊水智、白慧君、李翊以、張佩穎、陳鈺行、歐怡婷、張凱男、阮炳嬌、沈宗儒、楊淑洋、黃金儒、劉慧娟、梁雪萱、林志祥、黃柏宇、蔡家豪、林慧旺、鈕俊穎、曹予玲、陳健忠、盧冠勳、蔡佳慧、詹潔甫、蘇怡萱、陳宇舜、王振豪、張怡安、羅鈞雨、陳雅雯、王長音、鐘瓊峰、趙美玲、劉采堯、蔡宗吉、趙寧菱、陳玉順、黃雅玲、夏心怡、林嘉念、李合財、顧石任、林麗孜、林志群、詹右妍、吳珮傑、劉淑慧、黃玉珮、陳茂偉、樊念昆、張哲豪、黃舜映、黃子竹、程君豪、康樂瑤、高于庭、錢容倩、游秀娟、王守麟、賴建廷、王韻婷、黃秉綸、王俊宏、吳嘉辰、李皇芝、胡恩枝、杜俊毅、紀恆劭、劉馨儀、王志萱、王姿英、謝夢方、黃文珠、賴欣怡、王佳靜、劉容琴、陳雅慧、王怡臻、張慧娟、吳秋華、林姿吟、林依彬、葉百旭、賴緯琬、謝文能、陳家方、陳宗翰、陳怡然、許志忠、陳康貞、葉映筠、胡欣怡、謝建緯、楊家奇、羅祐淳、李政依、張志和、廖彥宇、陳美光、江智鈞、陳威易、周淑慧、蔡惠婷、張書亦、吳忠翰、鄭毓火、鐘奇毓、蔡興胤、李思穎、盧新禎、黃靜宜、賴旭君、王寶昇、林宛臻、劉淑芳、儲欣弘、盧志傑、楊堅智、陳勇志、楊志啟、王郁涵、張聖美、林雅婷、陳建航、彭真堅、黃威宇、錢薇城、王宛君、林子芸、彭曉薇、宋春仲、陳瑞卿、韓均怡、陳建德、楊安琪、方育維、蘇育廷、陳燦治、黃士凱、潘小坤、陳雅雯、郭可齊、黃瓊妏、侯亭瑄、杜峻幸、周珮如、黃柏士、李柏怡、謝家豪、李佳芷、張冰士、郭冠宇、楊尹財、李淑芬、金敬紫、何文豪、陳佩希、陳玉霖、朱貞雲、李與明、林仲惟、王先賢、盧昕燦、袁意緯、吳山裕、姚珈玲、余怡君、李方正、劉淑娟、陳志峰、黃俊傑、黃廷延、張淑琪、韓美君、閔紀屏、王巧禎、林倫意、王依宸、趙怡婷、張俊緯、蔡書岳、袁琪達、邱姿瑩、張俊明、林羿筠、劉良君、沈雅雯、錢淑媛、施世昌、趙淳蓉、蔡惠裕、李孟鑫、楊家安、廖婉然、李淑玲、林彥志、林志原、謝宜潔、李浩瑋、陳志齊、王玟嘉、胡世吉、楊少蕙、蔡肇傑、杜秀玲、蔡嘉彬、蘇涵恭、林思瑜、張政廷、陳俊宏、趙凱琳、呂書瑋、李信宏、陳佩珊、梁慧君、周靜宜、魏家豪、魏志偉、吳家慧、劉家宣、陳為州、林麗美、黃中容、王韻柏、蘇至伶、高建文、袁雅萍、陳重豐、葉智貴、楊書豪、翁文凱、翁欣嬌、洪淑真、賴懿清、陳智岑、林明辛、趙盛霖、宋虹喜、陳恭友、呂玉婷、周美慧、張惠雯、陳詩辰、李姿威、柳念歡、黃有男、潘旭豪、許劭侑、林純紹、楊佳琪、章宜珊、李柏成、陳姿一、蔡萬陽、羊鈞群、童吟海、王劭霞、童宜啟、陳政勳、孫思婷、陳柔蕙、林威諭、許宏誠、楊郁文、孫木清、高紋齊、吳怡如、許建成、陳佳財、邰易俊、郁嘉慧、陳怡淳、林逸正、劉家仁、黃韋伶、侯志偉、吳恭心、陳冠舜、蔡佳蓉、謝怡君、林雅元、鄭俊逸、丁心怡、陳麗娟、陳茂為、趙志明、廖信宏、陳韻星、林雅梅、童怡芬、黃慧君、陳雅婷、鄭莉夢、郁欣樺、周豪佐、鐘維倫、方蘭啟、林雅玲、王秉勳、張淑惠、鄭旻帆、林慧青、單子玄、施佳宏、梁盛樂、吳美名、夏夢迪、張淑妹、洪俊憲、李國謙、許文雨、惠木盈、蔡佩玲、陳韻萍、周淑玲、林宛臻、林瑋婷、江育哲、陳筠韋、藍俊銘、黃科謙、陳明珠、黃雅云、黃麗光、劉家新、鄭玫娥、費秀美、郭妍沛、陳上菱、楊惠如、林宜念、林佩儒、陳馨儀、許哲容、劉政彬、吳信宏、張哲銘、林敬任、黃瓊信、張靜如、呂庭伶、王京卉、林偉哲、陳孟涵、陳玉玲、李采麟、陳惠敏、李紀以、郭志遠、全怡婷、蔡正陽、張俊銘、陳俊達、鄭致定、黃麗伸、吳介辛、王雅強、陳尚喬、何維倫、金雅漢、冉志杰、李婷婷、曹淑芬、宋美慧、邱士銘、蔣青斌、邱瑜勳、陳瑋佩、和山谷、林雅惠、羅南華、王文昀、黃怡宏、陳詩吉、葉書瑋、劉家峰、陳志翔、蔡鈞妃、張佳琪、郭欣怡、姜山郁、劉柏豪、林大玉、陳世聖、王惠年、農湘坤、陳君財、方靜雯、簡明輝、沈建宇、明夢男、林哲榮、葉雅玲、許文傑、張皇憲、林鈺舜、陳蓉易、劉虹城、陳冠勳、毛彥璋、黃雅君、杜智傑、郭彥君、林智仁、安淑玲、阮明儒、黃鈺伸、謝依恭、林惟宸、張雅希、李崇和、毛聖傑、馬儀堅、翁立吉、李家瑋、張美甫、宋禎啟、沈姿穎、王山亮、楊昆霖、李木念、駱志豪、古剛綸、蔣心怡、蔡湖為、張哲海、林欣潔、沈怡伶、黃雅合、陳惠蘋、潘宇翔、游俊諺、傅珊穎、李文真、張琦昇、吳建正、曾菁善、白素美、王茹賢、吳純花、詹如珮、王依萍、陳睿昕、王婉婷、黃美華、蔡佳群、李政皓、林志萱、蔡政菁、吳民松、許慧玲、吳俊豪、王珮甄、劉志成、白延喜、許奕白、李子翔、傅曉妹、阮禎全、陳婉婷、黃惟賢、張祖友、陳建宇、涂詩涵、曹孟涵、林山霖、蘇建霖、黃依婷、劉凱倫、李皓柔、侯萱恆、溫政宏、楊登山、顧雅文、林筠憲、林鈺智、林文彥、張乃音、鄭任毓、顏家豪、張武佳、沈明軒、方家宏、郭雅婷、陳欣峰、曾冠志、陳怡彬、吳力治、王茹茜、彭舜昕、邱卉威、徐柏廷、劉奕湖、羅新羽、陳學辛、許牧謙、吳宜皓、謝俊宏、鄭明慧、李馥鳳、沈宥威、鄭豪惠、姚婷來、戴光惠、郭聿雲、黃妙苓、謝祐柏、張建宏、鄧尹玫、李欣宜、王怡君、吳寶盛、蘇志強、童芸陽、羅則憲、趙協禎、方承翰、吳芷珮、劉伯琴、劉育彬、梁法聿、羅宣邦、徐晴陽、劉睿花、楊美珍、吳火富、林鈺婷、談乃迪、楊智翔、蔡明琴、林家合、周佳儒、王郁帆、劉孟儒、陳寶玄、楊宗軒、詹偉誠、詹雅琴、陳惠萍、陸倍琪、林明儒、張尹宇、張嘉豪、陳慧君、陳孟穎、張書筠、蔡駿青、謝依菁、李豪昇、王雅雯、彭素欣、吳佳宜、吳淑禾、賴玫鴻、陳其惟、劉茹岳、沈文婷、宋與名、夏竹季、劉盈君、謝芝平、黃國傑、張璇孝、李佩盛、李育菁、李心怡、陳哲銘、金國靖、黃淑宣、吳淑慧、黃禎木、邱美娟、周逸群、許苑剛、林佑霖、李佳穎、烏惠雯、張蘭珊、蔡麗玲、謝宗盛、陳馨云、周芳其、許琬婷、陳冠廷、何亦芃、溫志偉、郭淑芳、張琳倫、徐惠君、陳詩涵、邱夙綠、鍾如順、黃姿蓉、黃水群、陳政侑、邱宛真、傅盈君、李方迪、周淑美、張玉玲、張毓行、楊美惠、吳銘香、溫紋淑、鄭孟君、曾凱婷、陳瑋妤、江家祥、韓佳慧、許怡安、金樂南、應士昇、徐志偉、林佳純、陳君達、陳欣昀、林豪柔、闞哲榮、藍安慈、楊肇貞、吳成賢、許惠君、曾淑惠、劉佳安、陳麗玲、林威宇、張家榮、林郁真、鄭雅婷、吳仰綸、藍介芬、陳侑山、林筱婷、李宜君、姜欣怡、楊怡安、趙哲榮、陳雅玟、李宗菱、陳子翔、傅淑惠、王世昌、陳希樺、戴建輝、蔡孟臻、林淑真、張怡潔、李俊達、張家瑋、張羿臻、吳佳春、葉哲瑋、許淑雄、蔡松辰、張馨杰、齊維婷、劉祐誠、陳俐恭、陳俊民、楊俊傑、林昆帆、邱安雪、錢淑華、許梅為、溫怡伶、陳緯然、林敬安、詹先維、黃守娥、張慧君、王筠順、呂婷婷、蔡淑一、陳瑤弘、陳雅文、王郁君、林靖雯、陳虹珠、賴勳玄、袁奕然、王麗龍、張智偉、和雅君、李恩芷、劉志宏、黃紋偉、王惠玲、韓雅雯、陳淑琴、林俊彥、林立盛、陳麗梅、王涵謙、張平喬、鄭昭胤、陳崇寧、陳文婷、蕭俊毅、吳姿蓉、黃敬博、李雅發、陳信能、吳欣潔、徐菁忠、張紹夢、趙燕東、劉育如、李承珮、陳肇孝、曾欣蘋、榮天男、陳姿秋、林敬佩、陳麗梅、賴佳儒、潘琳生、楊世寶、劉孟璇、吳佳冰、張淑惠、陳喜貞、溫裕仁、楊思潔、陳中芸、林海中、陳慈均、楊欣潔、楊維倫、袁志偉、陳欣純、駱于婷、趙孟軒、陳筱宜、劉佳靜、洪志明、鄭麗君、楊冠良、張家瑜、周恭聿、蔡惠文、李美惠、張皓瑜、劉正偉、王劭源、黃初惟、方欣宜、林慧吟、侯國瑋、方柔元、王以欣、陳彥宇、李瓊以、陳平倫、邱柏毅、王佳琪、家敏夢、謝坤愛、楊凱亨、邱怡君、謝宛谷、沈則忠、李馨儀、吳宜靜、謝珮財、趙曉貞、陳佩璇、謝珮瑜、梁雅娟、陳建輝、葉志銘、林彥娥、陳浩惟、王明亦、張駿堯、白冠良、林季萍、汪蓉聿、倪琬婷、陳俐花、王清麟、戴志翔、曾萱光、張人豪、連宜婷、鄭雅雯、陳麗萍、陳士賢、李秀芬、李恆木、孫雅惠、張怡珊、詹呈哲、張維定、黃麗映、曹雅文、駱麗霞、郭怡靜、廖政宏、蔡欣東、劉振宇、游俊廷、葉宗翰、劉惠美、趙育瑋、李嘉玲、劉欣儀、阮書映、馬孟璇、童清軍、趙水來、吳郁文、林左儒、蕭合銘、隆欣怡、林子汝、朱婷宏、湯怡婷、陳宜蓁、楊致遠、鄭雅旺、王左孜、周柏毅、曾琬禾、陳哲辰、林善宸、黃柏恭、傅耀亮、洪翔軍、孫力清、陳揚賢、藍少意、林士順、王靜怡、胡廷城、袁雅亮、黃語劭、張法雪、黃怡祥、周延志、杜哲宇、林志明、陳方辛、楊友蘭、王佩怡、李進諭、金韋志、張惠文、劉志銘、鄧威宇、吳惠安、陳卓瑜、陳人豪、黃育儒、徐隆洋、高大鈺、楊勳政、羅元法、吳旭音、柯慧君、顏玟怡、王明愛、傅秀玉、何淑娟、馬佩玲、林佳妤、洪佩琪、李明宏、陳美佩、顏偉銘、王宗翰、陳家慧、暴宗豪、黃明玉、陳淑君、高修憲、陳綺彥、羅任雅、王可娟、郭柏廷、洪勇火、林怡發、王剛良、陳映人、黃莉孝、李俊宏、賴建智、許吉萱、陳美光、曾詩旺、蔡松發、廖珮正、盧淑芬、黃筠菱、陳惠雯、丁家偉、賴萬輝、吳家苓、黃榮易、王國華、車維倫、張定妏、林康淳、方璇月、詹家佑、張馨儀、林彥儒、李文彬、蔡承月、陳致勇、劉冠雯、林有紫、陳冠希、楊冠宇、吳怡伶、陳怡雯、姚家佑、陳倫群、李旭州、林邦紹、蔣鎮宇、賴志銘、盧俊明、周毅仰、沈宜潔、陳睿仁、林淑茹、黃初喬、李淑希、陳智彥、楊雅雯、劉敬舜、鄭庭智、楊欣宜、王昱昆、蔡柏翔、黃明杰、潘小仰、蔡瓊慧、蔡惠雯、張瑞士、袁金成、周綠燦、楊瓊慧、楊淑慧、劉朝喜、余佳怡、洪紫睿、陳冠芸、黃天春、許志軒、林筱春、賴杰任、蔡禹宣、林佩羽、倪恩柏、宋喬函、蔡孟念、陳建文、賴雯珮、蔡坤禾、沈孝貴、蔡欣州、周聿鈺、陳怡維、劉淑明、許建智、林淑岑、許小德、崔明倫、陳容韋、吳美軍、王函芳、邱育德、余亞蓁、許人豪、王奕君、張台潔、李育以、李岳哲、劉育容、李冠宏、方兆聿、劉冠伯、吳武福、徐于倫、陳淑謙、黎陽玉、林鈺剛、黃依婷、陳佳慧、李依孜、楊美君、黃舒涵、宋佩珊、黃瑩心、楊哲喬、鄭凱秋、陳嘉豪、吳佳樺、蔡登忠、邱雅婷、蔡博佳、林法秋、陳俊吟、王維倫、傅曼木、陳治芸、馮志豐、陳彥帆、安姵明、葉宏儒、趙美娟、謝妤辰、謝胤臻、黃惠珮、陳佩祥、李涵玫、林靜怡、鄧正達、張淑娟、郭佩珊、顏淑芳、王峻琪、林倍宸、徐勳啟、郭怡君、郭坤岑、林蘭琪、曲雅萍、鄭伊萱、李婷婷、張清禾、張嘉希、李政麟、陳丹佳、李思昆、郭永妍、余家豪、柯舒杰、黃嘉玲、何立偉、黃雅慧、蔡翰佐、黃淑芳、鄭玉鳳、謝幸桓、吳和沛、杜怡欣、陳依一、劉淑娟、洪佳儀、張哲娟、黃靖恬、王美全、翁品雪、賴旻亮、陳信宏、陳行火、林于芷、童靖廷、黃倫虹、陳雯佳、張祖妃、林嘉偉、施群姍、陳裕均、陳振蘋、黃佳然、陳婷婷、張家瑋、吳政宏、王沛漢、阮語喬、虞賢君、詹盛民、黃昱順、王祐麟、余怡君、鄭仰和、楊芷泰、謝山尹、林文瑄、白博芝、陳卉儀、黃義夢、韓苑育、吳佳伶、劉筱芸、王建菱、徐松明、陳冠廷、陳佳穎、童禎紫、盧宜君、蔡初龍、張俊維、翁家豪、葉思涵、李玉芳、許舒心、羅陽儒、楊松木、張竣傑、楊敬東、黃靜如、楊育誠、孫茜峰、王怡如、李怡樺、涂有翰、吳宗霖、崔剛琦、詹舒治、張世雪、王婉婷、陳彥宏、李希劭、劉彥華、王志杰、李大鈞、葉炳娟、寇法仲、陳逸芳、潘玉宏、張建霖、吳法嬌、吳宏杰、丁嘉蕙、廉彥廷、侯茜霞、陳佳穎、杜德邦、林耀德、林君雄、李羽杰、施仰蓉、吳紀智、蔡佳財、李怡萱、韓慧玲、藍國瑋、沈堯菱、郭淳、陳燕虹、吳俊志、家芳芝、金人郁、郭枝柏、劉建緯、溫心怡、周貴吟、洪卉志、陳怡江、蕭剛秀、陳得昀、吳宗凌、陳雅文、劉志成、馮雅雯、李順琴、顏雲軒、林信豪、林郁菱、陳文吟、陳姿紹、游琬菱、胡紫堅、王宏偉、王筱然、崔台諭、滑燦易、馮恩政、李吉明、李惠君、錢芃依、鄧志翔、王姵君、黃冠霖、陳俊賢、呂涵任、黃啟如、崔雅君、黃啟舜、陳淑華、司雅惠、黃珮如、楊珈明、郭名燕、劉修雨、霍林伶、李智天、楊婉昌、張昕翰、金綠宣、林怡沛、陳清甫、馮麗君、魏建輝、江怡潔、陸介俐、林姿蓉、葉嘉玲、游歡青、李玉娟、張俞儒、王豐芸、黃佳佳、黃筱婷、許雅茹、陳昭妤、盧怡樺、林信宏、蕭雅鑫、李秀琳、黃松春、蔡明紹、黃隆育、林肇伯、吳文宏、葛智堯、黎于真、陳威瑜、謝其蘭、陳雯智、王智迪、黃聖卿、黃家維、林慧岳、林依婷、蕭怡君、張方傑、林義琬、林伯雅、蔡瑞軍、陳宛良、黃可丞、羅俊嘉、周怡行、許家舜、賴惠雯、吳珮諭、葉俊鑫、楊樂勳、李佳仁、陳韻婷、黃國姍、簡文馨、楊佳燕、黃淑娟、黃文忠、陳建文、王雅婷、李宜琪、呂丞郁、丁玟蘋、王俊菁、藍淳火、陳凡燦、戚皓來、陳乃育、張可季、李文蓮、陳雅雯、陳夢凌、李素貞、王佳玲、洪治銘、羅華康、涂堅娥、蔣姿穎、李雅輝、林俊陽、賴友偉、簡佩穎、蔡佳穎、房健銘、游懿亮、張建銘、鄭定強、吳金亞、潘淑芳、陳志芬、張凱玫、陳俊恩、蘇天堯、吳博欣、李姿仁、房瑜香、黃佩芳、郭聿祥、郭淳娟、錢英傑、方惟幸、陳先燦、蔡瑞康、張揚秀、陳珮君、藍志遠、謝仁璇、鄭雅伯、林宗毅、桂雅琳、蔡明真、余大鈞、朱昌紹、崔承穎、成玉燕、貝志明、彭弘孝、鄭青聖、張登竹、吳百成、許鈞瑤、陳憶青、楊向秀、潘至燕、李仰斌、趙婉婷、陳淑華、隆承卉、陳嘉智、陳盈山、周穎然、簡伯和、林舒涵、林祐儀、賴季娟、陳智恬、黃世昌、唐欣怡、柳毅禎、徐淳合、楊珮青、鄭小夫、彭浩伶、黃美心、林威江、林孟倫、張志臻、李孟智、李依紫、黃俊賢、胡雅茹、曾若緯、劉怡慧、李香鴻、葉苑松、吳智傑、周于偉、王立偉、程可欣、陳玉軒、黃靖羽、張怡伶、劉育菁、林世星、柳芳儀、柳添香、張乃真、彭慧娟、洪雅雯、羅政霖、趙宏達、蘇隆枝、黃政旭、林怡平、陳冠傑、鄭偉哲、陳恒亦、李珮雪、鄧豐祥、吳婉玲、張惠妮、黃彥良、陳秀君、羅美惠、吳皓妍、王憶樺、張文喜、郭奕翔、林宗喜、許清慧、鄭科萱、吳佐迪、王豐均、陳維哲、邰惠玲、陳世恩、王涵凌、陳慧貞、楊涵和、胡俐昆、李慧娟、張碧伯、陸啟元、宋瑩禎、涂君鈺、陳孟欣、李怡蓁、林妍蓉、鄭伊昆、陳敏樂、韓姿吟、仇婷憲、蔡俊男、費淑真、吳品美、連玫航、劉婉瑜、金嘉豪、屠雅君、奚馨禾、張均、葉愛雨、賴哲嘉、陳維人、劉玉容、黃智英、楊姵水、姜愛聖、張東琇、陳雨芷、楊宗甫、張亮瑞、林友江、陳俊佑、黃淑君、林淑萍、陳啟音、姚歡行、李惠玲、黃秀芬、蔡威依、邱淑傑、張綠伶、顏曼聖、牛伶倫、蔡秋伯、孫俊傑、許禎琴、徐琪蓮、廖維偉、陳石全、李亞禎、馮軒萱、吳佩雄、陳秀英、鞏惠玲、潘郁宸、夏堅謙、唐鳳盛、翁真仲、林振宇、馮婉蓉、林哲琴、林敬意、張志明、謝君姍、陳彥歡、黃冠傑、黃廷嘉、張哲善、鄧佑霖、黃忠美、陳逸綸、張怡靜、林沛綺、侯政廷、賴佳宏、顏雅馨、杜明倫、黃健銘、陳碧旺、張月芳、張光鑫、錢合純、黃香枝、楊竣傑、陳建瑋、蔡信宏、陳佩珊、林玉娥、姚冰山、謝千薇、游浩惠、黃啟平、林安紋、韓京泰、陳薇恆、陳玟杰、李依茹、翁冠宇、賴俊宇、陳怡均、林聿名、周月映、陳姿伶、鄭右妍、汪國偉、李欣樺、吳樂娥、楊雅萱、王智文、陳若馨、陳秀中、曾宗憲、許郁翔、鄭宗霖、林宛君、韓淑婷、葉峻豪、馬彥博、簡延恬、張盈甄、蔡竹芷、蘇原漢、謝筱雯、陳佳蓉、王秀霖、夏函歡、張佳慧、郭怡婷、林柏雪、林秀友、王韻婷、張志豪、李季穎、單蘭坤、劉政峰、蔡堯正、林書妏、李夙禾、蔡佳和、吳仁杰、李淑君、張阿哲、蔡明瑋、羅儀平、陳威新、林鈺琬、黃世容、張美玲、林晏康、詹宗安、陳明宏、李思坤、嚴婉婷、鄧建志、吳昕男、劉邦帆、宋岳璇、王姿穎、張婉君、施佳儒、馮宗翰、高維哲、陳佩芸、趙友裕、林韻如、林芳如、張思韋、吳雅岑、宋冠良、高珍意、盧柔瑄、鍾明雄、晏子豪、黃博文、駱仁添、黃新帆、錢晉昕、汪薇銘、許穎寧、朱宛儒、黎建輝、張維玉、李俊諭、劉孝劭、陳國雅、游進舜、劉孟嘉、姜欣怡、陳恆強、劉千江、鄭意婷、王鈞如、林雅玲、溫家弘、楊燦瑩、林淑信、鄭維孝、蔡文啟、游家欣、李佳虹、吳家佑、陳佩娟、劉詩涵、鄭介中、陳牧剛、周凡佐、黃淑惠、陳成奇、黃沛林、王彥廷、吳柏汝、劉睿全、羅雅雯、楊珮珊、陳俊銘、賴盛元、朱青昆、郭潔韻、謝雅芳、賴志偉、林佳蓉、陳勇志、謝佩誠、王珊康、鄭博枝、李羿孜、郭芸蘋、謝子菁、邱瓊真、王宜君、黃彥瑞、李宛娥、張玉萍、林泰雄、鄭文凱、陳彥廷、張家宏、林喜淑、許惠昆、毛峻翰、李雅筑、林家亦、黃信宏、王華恒、楊怡君、陳淑映、王俊賢、張虹菱、吳欣樺、劉佩玲、黃行元、吳家孝、劉怡蓁、周敬琪、劉馥韋、湯佳琪、裴一泉、洪建良、唐怡如、林書茹、杜冠良、張鈞哲、林美雲、涂韋志、張玉季、鄭景鴻、陳劭娟、黃雅惠、蔡馨文、林智仁、唐碧妤、簡晴其、鄭枝利、曾佩如、吳莉婷、李文駿、陳靜如、蔡宜亦、桂蕙銘、陳佳柏、張宗達、杜泰喜、葉珈駿、丁靜霖、蔡卓昕、符仁奇、許宗憲、王琬君、陳萱妍、黃建彰、陳美珠、劉京妹、劉志鴻、陳怡婷、張芸政、趙枝哲、江偉翔、賴聖凌、吳宜瑜、黃姿源、林國音、王木法、陳伯洋、陳家旺、鄭家瑩、林育維、張碧學、韓瑤元、李靜妤、李美松、王哲榮、蔣中信、許珮駿、吳宏達、宋少胤、平家瑜、陳琬甫、蔡宏儒、李威昀、林美玲、周雅雯、王柏易、蔡倍天、李麗雯、李美苓、蔣育菁、江柏勳、張佳伯、李冠任、黃文芸、鄭淑惠、謝明琳、連傑然、鄭泰季、黃武旺、林宜威、陳盛英、柳宗翰、翁淑惠、曾玉華、陳茹士、廖夢劭、楊安中、吳人貞、杜聖銘、劉紋宣、陳金瑜、曾中念、諸嬌欣、陳得麟、羅至靖、謝歡文、尹玉梅、黃孟哲、袁定人、王金陽、吳翊輝、林育維、賴建岳、趙凱元、張涵伶、陳嘉韋、黃詩婷、吳欣舜、王兆韻、周淑君、蔣慧綠、于郁翔、馮佩蓉、王惠友、劉火萱、鍾紋平、黃靜雯、蔡初凱、陳倍冰、陳佳音、許姵君、賴合陽、陳彥均、賴芝弘、蔣合昌、林雅婷、賴欣怡、謝明倩、傅哲臻、黃怡雯、黃雅玲、黃初齊、簡明軒、黃燦雅、李建銘、林君輝、林國依、林力娥、王淑中、林緯茵、陳芝玫、林彥心、許中航、吳孟玫、陳鈺雯、蔡靜怡、謝淑美、許培水、周潔偉、陳瑞蘋、黃明珠、柳雅文、彭樂火、陳俊洋、洪欣怡、崔敏盛、李嘉惠、潘典光、張景原、杜家瑋、徐怡珊、葉曉良、馮誠紫、黃正甫、林柏秀、呂貴誠、黃展雄、張凱仲、宗士弘、鄭雅慧、戎彥志、黃欣穎、陳友儒、趙進元、李雅雯、林欣穎、王允杰、羅佳穎、葉孟木、顏雅玲、林建貴、劉怡婷、陳明雲、夏介惟、王東如、何昇桓、朱文賢、黃盈君、陳宗泉、劉家音、林政行、湛炳婷、林秀玲、羅佳穎、劉枝霞、葉志偉、陳任蘋、黎希育、李威宇、許志琬、柳湖寧、陳芃宜、蔡嘉友、沈盈亨、阮儀瑜、李俊傑、陳友貴、徐佳靜、盧千慧、李佩玲、鄒家榮、曾慈龍、張季辰、林明輝、金智文、沈秋萍、昌青鴻、林家銘、詹怡靜、許建宏、翟美玉、黃雅晴、楊淑娟、林易修、高介龍、蘇怡君、楊雅惠、魏義屏、王淑琪、陳俞萍、羅靜怡、黃俊霞、何君豪、洪國瑄、黃岱妏、朱美祥、陳秀夫、周志嘉、廖嘉文、王凱翔、楊育霖、謝志祥、劉雅芬、劉睿秋、陳素凌、陳珮雯、張淑華、蔡明倫、李至哲、洪偉富、趙逸凡、吳秉勳、陳俊宏、黃冠中、林文賢、蕭佳慧、余靜雯、施凱元、郭威瑜、鄭玉鳳、酆仰均、林雅琪、蔡佩喬、黃家宣、陳景惠、林建仁、周琳鈺、蔡美珍、林文群、高維哲、何珮瑜、陳兆白、方俊傑、高詩婷、洪皇婷、蕭雅云、施泓汝、謝信宏、鄭雅隆、錢雅鈴、蘇立旺、蔡依綸、鄭至維、張東紋、王明輝、陳建海、唐舒雄、江佳光、林筠民、曹志成、耿家祥、楊志光、黃佩樺、童蕙財、林婉君、余于珊、陳建珠、吳凱倫、熊俊男、黃建欣、黃冠萍、金玫漢、杜平慈、耿雅萍、邱淑臻、陸康蓮、蘇志偉、吳宛新、劉家慧、蘇家豪、蔡采宣、譚思潔、王婷婷、彭中谷、曾武水、林孟明、蕭佑霖、王茂雨、劉大駿、王淑奇、袁丹然、鄭肇念、刁枝興、吳志宇、杜民芃、吳欣潔、陳彥屏、陳若綸、楊宇劭、張彥璋、劉建一、黃智苓、陳鎮輝、陳怡迪、陳雨君、葉曉薇、林依琇、齊怡文、杜俊緯、廖文芬、趙隆漢、萬思涵、劉小蘭、陳偉珠、王韻菱、陳雅君、徐信宏、趙岱瑜、陳雅雪、吳孟哲、張秀英、張詠群、廖家祥、柴慧茹、吳俊成、楊志花、張佳玲、陳政達、謝璇啟、胡志苓、陳佳蓉、黃詠全、李安柔、張可欣、張哲昆、施元娟、曾雅雯、王聖仁、傅湘綺、曾博仁、韓賢航、楊佩穎、王朝函、吳堅筠、陳宗憲、陳惠男、林育裕、陳晉彥、張明學、毛秀芬、榮淑芬、林淑貞、陳豪禾、郭致珍、黃佳霖、徐育誠、黃柏玫、林芳財、宋姿君、荀俊嘉、廖莉宏、駱肇凌、林婷婷、陳逸弘、李孝芷、林思賢、宮盈君、郭婉婷、吳愛函、陳建佑、林靜凌、黃子劭、王雅君、謝枝蘭、蔡政亞、王曼憲、林進樺、荀珮君、黃鈺喜、郭世昌、侯雅芳、姚思穎、李芸卉、汪秀娟、邱俊宏、古逸男、梁俞白、許幼亮、溫俊松、郭桂蕙、童姿瑩、林宜劭、盧曼吟、吳泰友、許怡君、林振豪、林淑芬、王蘭誠、王佩芬、謝婉玲、童婷清、許庭瑋、楊孟涵、吳夢筠、童智傑、林育雨、潘筱涵、鄭希映、陳堯寶、謝妙育、張亦芬、林品美、張科祥、王國榮、呂瓊慧、林承穎、張金伸、林婉婷、盧建銘、陳軍安、陳孟倫、吳家康、尤雅雯、蔡嘉峰、林冠霖、黃小瑋、楊彥志、林俊德、鄧家弘、蔡彥宇、何志銘、傅雨麟、黃純修、毛法英、陸與雅、林良來、張曉萍、楊惠宸、鄭慧娟、許子以、陳姵丹、李志偉、陳禹恭、崔鎮盛、王曉萍、鄧依婷、王美惠、張博真、楊麗娟、趙星仲、洪國昇、林宜雯、黃富雯、陳玫然、馮怡慧、魏愛貞、黃合龍、蕭雅萍、陳威富、楊長倩、李心怡、黃歡富、程心怡、宗家弘、吳智翔、駱康齊、劉上雄、施育興、陳昭強、陳淑雄、黃丹宇、賴俊士、劉佩樺、沈瓊慧、李志昆、鄭尚東、郭國偉、洪珮雯、王姵君、葉琬婷、周祥玉、王柏珊、陳揚辛、顏雅如、張威廷、陳逸梅、謝孟倫、王思舜、黃群雲、張慧娟、謝翠雲、鄭姿穎、王凡泉、王嘉天、羅樂芷、李俊安、蔡香珍、藍雅婷、李佳宏、童子豪、錢姿伶、葉宇嬌、連欣怡、張定添、趙慧瑞、吳瑤安、王麗雅、余鴻玄、侯偉智、吳軒銘、崔盈君、劉佩珊、洪玫正、陳逸伶、曹語妏、楊裕淑、衡婉紹、周怡安、吳柔婷、黃鈺香、張俊傑、張淑華、烏彥平、周明儒、陳淑萍、李宜鑫、林政龍、蔡亞娥、許丞蘭、鄭至群、潘美惠、王妤瑤、倪淑芳、潘建宇、黃嘉民、蔡宗春、林哲娥、姚淑真、陳宥珊、洪偉坤、王政峰、王江珍、張剛樂、賴嘉慧、張智偉、吳淑郁、鐘睿茜、翁雅惠、林家佐、林蓉麟、阮怡君、林彥儒、王靜怡、蔡佳伶、張建智、廣慧萍、吳昀苓、酈亮真、鄭義淳、王協平、程柔郁、王木秋、陳民辰、王聖全、陳佳玲、余偉倫、王昱宏、劉瑋堅、劉欣穎、王珈堯、陳文彥、許哲妤、林歡恬、陳牧芸、陳莉婷、張群行、陳裕仁、吳宗富、謝明柏、林俊傑、蔡雅惟、黃淑惠、洪姿蓉、黃舒彬、朱建蘭、趙明軒、謝嘉玲、黃冠萱、陳嬌為、劉竹坤、林珮妮、林鈺法、張涵杰、孫力信、陳怡茜、蔡法龍、賴怡宇、朱瑋婷、吳明玉、陳益水、吳月學、朱國榮、謝佩瑞、梁恩嘉、賴淑芬、張淑慧、蔡俊霖、吳宜潔、蔡玉梅、陳昭峰、李智偉、黃佩珊、馬善季、林士賢、黃姿伶、姚介娥、吳志偉、林雅琳、陳怡光、阮語苓、黃苡霖、許育誠、岑蓉恩、黃紋漢、張智蘋、李偉哲、許重汝、黃文宏、張于倫、林翔苓、藍裕伶、張瓊文、林佑霖、黃任弘、顏祖柔、倪毓玄、張淑貞、陳協尹、陳冠廷、陳立偉、林素恒、謝怡文、蔡良慈、符佩蓉、林冠婷、黃克鑫、鄭雅諭、汪思賢、王和修、蔡欣春、王志季、郭佩璇、吳雅婷、陳奕新、王立志、張瑞蓁、謝恩嘉、林嘉芷、陳美新、戴韻城、周華松、洪振豪、林雅娟、衛勳玉、蔡惠如、王峻綸、黃于亦、浦佩蓉、連秀娟、馮乃娥、陳怡婷、丁吟柏、吳毓映、趙子卉、黃其水、張孟君、陳德宇、張雅婷、邱水學、黃建銘、汪怡璇、林書豪、何欣怡、劉杰惠、王儀韻、陳姿富、林仁杰、陳惠聿、陳育維、陳玉行、吳鎮月、王文羽、劉士瑤、時雅慧、陳雅慧、吳宜強、林淑芬、夏恒青、張欣潔、王詩成、陳欣傑、陳朝友、陳筠妏、羅珮伶、林胤治、黃惠珍、王智蓁、顏郁婷、林嘉政、戴惠如、林逸明、林婉福、蔡志綸、黃右名、陳明儒、黃重富、沈珮凱、張忠翰、杜雅琪、李孟政、吳振安、黃竣傑、方佳燕、蔡佳妹、周苑緯、吳慧韋、黃思育、甘佳蓉、張佳欣、吳佳穎、謝佩琪、林欣怡、許原佳、趙志育、張雅云、鄧宛臻、蔡芸行、邱佳蓉、吳心怡、瞿竣傑、鄧嘉玲、徐虹來、林彥佳、劉欣怡、翁瑤凡、陳怡伶、黃鈺霞、陳又學、陳淑娟、許雅鈴、陳逸裕、楊佳伶、白世昌、黎翠惟、曹辰雯、王雅玲、李平燕、李定音、王儒廷、吳明婷、陳雅芳、包紹琬、陳柏航、陳冠廷、張姵琬、賈惠玲、郭怡如、白奎瑋、王彥哲、吳明娥、楊美惠、吳慧羽、韓家銘、黃俊賢、羅靜怡、林孟倫、梁炳亦、彭雅婷、金勇倩、鐘峻豪、陳美雲、張敏玫、陳盈希、曹又恒、林孝谷、許玄綠、白佳靜、郭建宇、張倫皓、孫馨儀、陳茹正、周政念、賴俊男、楊蕙雄、蔡沛妏、楊定燕、鄭靜怡、李翊幸、林信宏、呂明杰、劉左琪、黃美雲、楊郁水、顧剛合、蔡雅忠、楊勇志、黎東鳳、路盈秀、簡紹緯、張仁傑、林信琦、黃淑妏、黃文華、曹登茹、林欣蓮、曹士銘、許品禾、曲靜寧、劉玉財、徐苡文、陳國音、馮淑惠、葉士銘、周淑君、陳淑琴、黃筱寧、施偉智、陳世妏、蔡原琳、呂欣怡、林健光、吳佳儒、彭睿琦、陳家吉、吳淑惠、蘇立康、李綠慈、賀素香、楊昇貴、周千昕、張怡鑫、李秀娟、陳怡書、李麗雯、陳淑福、謝奎峰、陳志穎、郭豐利、蔡明瑋、陳淑萍、林欣芷、吳俊彬、李旺勳、蔡登娥、林育琦、楊偉倫、白意龍、馮涵虹、何佳法、吳欣元、黃均成、陳文湖、黃怡舜、林學文、馬哲瑋、鄭雨鴻、黃志偉、賴俊豪、簡杰孝、吳湖喜、涂志豪、謝智文、賴敬辰、方意侑、陳家銘、孫辛吉、洪桂岳、馮雅文、師雅筑、車淑娟、黃玉能、張毓名、林靖雯、黃育儒、宋偉翔、黃崇儀、錢行明、翁添政、姜琪禾、徐婷婷、鐘品南、馮玉婷、陳宗琴、崔麗友、余淑娟、周軍蓮、陳憲雅、蔡欣怡、游智翔、王禎柔、許坤劭、連宜婷、林怡均、張韻如、林少安、鄭威寶、鄭雅雯、儲韻如、楊軒馨、謝佳純、謝怡君、張隆陽、許家齊、張廖佩玲、莊惠瑞、黃名中、林家堅、金與娟、謝文輝、郭柏樂、黃瑋婷、宋文彬、楊俊賢、林冠茜、蔡信宏、趙建銘、李嘉玲、黃裕仁、姜翰發、郭淑慧、鄭昕光、陳美瑜、蔡佳富、陳宗翰、黃婉玲、陳志凌、鄭秀慧、倪佐軒、王麗華、陳竹季、林素彥、許怡萍、賴怡瑜、何偉銘、陳柏豪、白伯佐、傅佳純、周淑芬、徐盈甄、宮姿穎、謝文貴、曾仁杰、林玉梅、林冰禎、黃美惠、陳韋湖、袁怡潔、王昀茂、張政光、黃偉書、齊奇書、陳人雯、沈昱盈、林明元、黃鈺杰、陳俊穎、唐凱元、郭怡樺、張冠傑、陳裕仁、蔡青雲、蔡宜貴、林靜如、楊郁文、黃文文、王竹木、張嘉行、黃士順、方善岑、劉承恩、陳姿瑩、袁紫瑋、陳雅雯、劉琳茜、林宜紹、劉志瑋、曾慧娟、林彥博、黃彥儒、童法仰、陳彥翔、李俊茂、李彥菱、甘杰歡、孫瓊慧、許家毓、張育星、曾秀玲、劉聿延、曾振宇、施雪貴、謝泰綸、葉明杰、許典火、趙珊慈、劉軒佩、黃建宏、周怡花、李明慧、王冠凡、林慧弘、陳盛士、簡盈秀、李宜發、趙銘岳、劉俊銘、沈欣怡、郭辛紹、張宣紋、林韋光、吳雯妍、林秀慧、林家美、邱昱廷、黃俊如、蔡德薇、黃鈺玉、楊惟珍、吳倫夢、仇家良、洪淑旺、黃怡順、何淑敏、馮智文、王昱迪、吳靜宸、蔡奕花、張耀廷、李建中、張昇蓮、謝佳孜、徐幼茜、黃嘉佐、張家香、陳智仁、童文坤、蕭敏紋、暴昆憲、林宗青、沈潔民、趙歡恒、沈瑩善、鄭倩惟、陳雅齊、黃俊江、吳宗秀、楊揚靖、駱昱亮、盧佑霖、陳志宏、馬劭娥、楊智芸、阮依婷、蕭詩涵、張宗翰、吳卓汝、謝盈秀、蔡倩花、吳政強、彭淑惠、張敬剛、袁寧辛、劉秀美、林欣穎、丁培均、楊大恆、張婉瑜、陳琬婷、林力映、吳淑芬、賴玉州、黃家綺、林美惠、謝依政、郭容輝、王威廷、侯玉玲、謝俊霖、林舒婷、林嘉慧、汪茜達、高翔禮、楊志豪、楊玉婷、黃倩峰、黃美軍、林筱雯、陳彥純、翁晏容、黃奕月、陳姿行、袁姿伶、余淑芳、詹佳芳、王凱鈞、林志俊、楊木瑞、張韋成、黃泰香、陳曉亨、朱希蕙、黃雅凌、胡亞輝、陳亞真、張雅甫、袁馥筠、黃忠清、鄭純意、陳宏峰、李盛堯、高琳祥、張大鈞、唐珮珊、吳淑芳、吳明珠、邱珮君、林家月、王麗雯、王怡一、游馨星、曹立坤、陳俊彥、吳怡能、蕭欣怡、曾盈秀、馬俊安、黃星佐、阮佳欣、謝志漢、楊雅星、朱美娟、張博蓮、張昆吉、盧名南、陳麗雯、謝怡婷、柳承翰、張乃文、弓純音、楊雅慧、吳萱其、穆珮陽、龍昆甫、季俊賢、王建啟、林智文、陳耀德、張耿雄、鄭萬舜、涂介合、張萬孝、鄧文馨、廖志韻、盧可康、葉新鈺、洪建彰、張旺源、陳盈天、王千惠、璩語湖、劉與堯、李守勳、謝雅芳、羅坤琳、黃士群、涂佳玲、陳瓊方、劉歡娟、劉茹寧、諸函博、葉健銘、農維倫、蔡文政、游永娥、強宜芳、鄭文彬、黃珮盛、林莉婷、高心怡、王偉皓、王建緯、韓建智、翁紹剛、趙嘉琪、陳美玲、方淑敏、陳于庭、陳芸洋、沈雅芬、黃盈君、吳雅航、郭筱涵、吳威廷、林淑諭、李綺芳、張怡靜、洪嘉洋、林美泉、林先桓、藍佳純、顧怡菁、李家豪、楊柏豪、邱鈺婷、張柏毅、賴耿紫、林秀希、梁佳琪、賴軒能、鄭俞添、陳逸岑、蔡秀芬、柳宛蓉、柳法映、楊涵儀、吳思美、邱珮如、謝心怡、林吉珍、夏秀玲、吳宜帆、林姿霖、陳嘉慧、璩雅婷、陳立芷、陳慧玲、吳宗吉、危淑娟、蘇有玲、林思妤、吳京亞、蔡韻如、林信年、賴柏成、陳士燦、蔡宛禎、楊蕙軍、邱穎旭、錢佩岳、陳琦儀、高宗憲、張竣傑、林威宇、韓海淳、趙淑琴、鐘佐萍、黃薇伶、蔣璇季、鄭文揚、林建琪、唐馥芬、屈欣怡、連長年、林淑甫、涂俊傑、馬淑娟、詹珊仰、楊初泰、鄭俊傑、葉宜婷、趙依萍、江民志、鄭文云、呂木昆、張大旺、王其剛、楊士凱、楊宗蓁、錢哲榮、許惠桓、陳劭宏、陳嘉豪、葉重歡、唐秀芝、徐芝淳、宮怡君、彭傑沛、張儀淑、柳芳如、洪俊宏、楊家豪、王佩君、張博鑫、虞青明、王偉隆、宋宗翰、陳昱琦、謝欣堯、郭佩玲、張筱涵、鐘建勳、沈剛平、蔣翊一、林玉尹、周雨霖、林妤妃、左和茜、王姿紫、姜上雅、張伊妃、吳玉萍、趙沛佳、戴俊毅、蔡博仁、黃威廷、王舜郁、馮志峰、林佩芬、陳家欣、鄭碧安、陳以添、李亭松、陳珮倩、郭韋廷、胡雅玲、郭麗卿、王瑩娥、陳明夫、毛典燕、林永函、林孟軒、楊歡修、陳予軒、張得幸、倪家瑜、林恩達、姚玉婷、陳曉俊、黃子臻、許仁卉、莫凱鈞、毛怡雯、趙欣樺、陳淑學、陳雅玲、林雯珠、蔡孟儒、陳柏偉、林雅琪、錢聖緯、賴玉鑫、陳興亦、王有能、柳雅雯、廖至生、黃佳怡、林欣皓、陳俊郁、齊志偉、洪國州、陳彥君、楊竣傑、林睿桓、劉振瑋、吳維倫、王一瑤、郭姿伶、吳美堯、葉彥志、溫勝仲、郭宣勇、謝依華、蔡憲利、浦怡萍、汪清夢、許俊凱、曾佳怡、黃文春、溫佳蓉、劉怡伶、馮堅凌、鬱宜潔、王志維、鐘婉緯、陳淑娟、王偉臻、林俊彥、林瑋育、姜益紫、吳秉定、楊彥霖、王文琴、郭思涵、曹淑馨、馬琬娟、楊偉盈、黃子合、袁宗翰、齊淑惠、羅怡婷、連文華、陳家榮、李乃昀、游均州、陳建正、張雅法、吳怡昀、謝嘉琪、李恆喬、李盈君、夏俊明、陳廷裕、許宜靜、謝彥志、黃詩琪、黃家豪、陳志法、唐婉婷、方瑋玲、陳君欣、吳碧雅、陳奕全、袁詠珮、林郁雄、羅璇彬、張家岑、臧庭齊、陳惠哲、顏靖玟、鐘育菁、王明軒、吳佩怡、黃淑亮、賴士賢、胡育瑋、賴俊成、包昌萱、陳政云、吳必菁、陳雅芬、王雅男、趙松寧、柯浩怡、黃紹佳、林瑞香、黃建威、張明軒、陳彥儒、賴婷婷、陳智傑、謝揚維、謝雅芬、王淑娟、謝俊宏、王珈以、林慧君、程宛君、楊俐修、王淑倫、李慧玲、林玉華、宋振菁、林慧忠、胡秀劭、黃筱輝、林育如、黃佑霖、張瑞陽、賴佳霖、潘婉裕、陳翔婷、施依博、王偉智、田彥儒、程仰禎、高韋伶、李佩恩、曾亭慈、陳翊志、陳逸幸、楊怡芳、彭明儒、謝家榮、江涵元、曹宗翰、夏志祥、許育柏、陳信伸、黃俊博、林健諭、吳秋新、麻柏勳、劉懿光、陳倩廷、梁裕然、曾仕玄、吳俊貴、吳欣怡、游南娟、賴銘源、郭培萱、錢其旭、黃家誠、何佳其、黃嘉順、曹浩秋、丁穎娟、鄭伊婷、程君定、鄧玉云、吳林佩、蘇冠伶、胡淑玲、平佩儒、劉阿治、謝裕仁、劉家屏、陳芷忠、何美慧、吳凡琇、劉靜芳、邱怡修、賴慧龍、邰琬婷、傅芳如、劉美慧、廖雅玲、鄧維茜、陸雅文、邱涵士、李明行、傅宜靜、劉家合、賴美雲、郭家榮、盧佳霖、鄂柏勳、李建江、李芳彬、王玉娟、張耀貞、吳政奇、林宛白、曾淑婷、何祐誠、沈淑玲、張昱宏、王雅佳、鄭愛青、黃志強、劉子娟、蔡俞諭、陳佩珊、林怡婷、陳雅琪、黃靜怡、黃靜如、林淑坤、黃牧花、許文定、黃秀香、吳姿吟、林麗堅、鄭惠樺、麻琬行、陳祐昇、柳薇月、王家欣、蔡怡君、林聖士、李順志、邊希均、許雅禾、蔡怡靜、黃鈞佳、楊彥良、黃茹鴻、王依婷、林于婷、楊惠珍、方莉一、邱家齊、郭淑美、梁冠勳、詹彥宏、張偉宜、張淑洋、廖俊榮、陳家銘、張宏儒、黃慧天、高怡婷、賴欣怡、蔡翊毓、謝雅筑、蔡雅婷、趙吟忠、陳仁傑、張治伶、張紀桂、鍾倍琴、洪妍元、吳雅惠、林怡廷、許佩珊、林新軒、黃宏偉、楊怡婷、黃啟哲、許智淑、黃婉城、黃冠宏、黃雅如、張怡樺、彭鈺婷、謝南天、廖寧堯、張政宏、劉嘉偉、許燕嬌、王翰儒、吳佩如、阮朝軍、孫宗儒、金于聿、黃紫源、李乃淑、曹小泉、劉怡萱、于于真、沈靜怡、陳雅惠、林雅來、黃舜文、黃萬妹、羅梅夫、蔣奎慧、韓鎮元、範宜靜、陳怡瑋、林雅旭、張維琴、林華韋、榮宛臻、邱婉玲、劉雅聖、黃柏成、張雅雯、陳思穎、許俊毅、王立為、陳宜蘋、林美堯、王琦霞、王盈君、劉怡欣、黃建舜、張志嘉、李婉吟、陳俞毓、林庭昕、李姵聖、林孟涵、謝法禎、江茜原、吳佳幸、董嘉雪、陳佩璇、黃文新、王志松、林昇達、林盈盈、林彥霖、袁炳芬、王怡良、龍怡君、謝慧玲、胡明輝、黃于真、胡海喜、李欣苓、高喜陽、劉法珍、周智剛、王瑜萱、簡常方、許宗穎、張瑋玲、許銘利、謝綺亦、周景元、林智翔、茹憲羽、閔貴屏、林鈺紹、師怡雯、張瑞枝、葉雲東、許玄君、李瑋婷、鄭美惠、許玫年、陳芝亨、朱馨文、溫佳蓉、許博妃、楊淑芬、阮明輝、黃陽以、吳瑋谷、林則漢、溫妤邦、林昀佳、蔡揚珠、王建勇、鄭怡如、劉韋成、李家郁、劉建成、郭之倩、洪明雯、楊宥定、林秀貞、劉舒生、王雅彬、馮宛華、吳佳英、謝郁雯、吳淑娟、陳君惟、梁宏偉、黃麗東、鄧佩穎、吳淑真、鐘心怡、黃彥舜、錢昱榮、鄭淑淳、黃意婷、蔣丞以、黃宗達、陳行全、蔡雅希、詹真東、古冠宇、羅志亨、沈志成、翁俊男、丁淑娟、林亞善、汪明宏、劉家宏、何佳瑩、吳芳如、孫怡萍、倪維然、陳玉紫、陳世斌、蔡宜盛、謝鈺雯、劉彥良、林思天、李潔均、林夢清、葉良娟、童家瑩、楊紋妮、王儒源、黃冠俐、李恆琴、黃俊勇、楊育菁、王詩婷、唐泰珠、崔漢禎、鄧嘉琪、鄭東慧、陳千慧、經莉歡、林威廷、林旺白、黃淑元、吳于婷、游紫玄、陳明珍、黃正樺、駱文姍、林家仲、邱淑妤、陳金吟、季恭月、周燦哲、李虹明、甘建瑋、林苑堯、溫珊蓮、趙茂忠、林宛以、劉盛法、張平貴、洪冠宇、蔡佳航、林彥恭、張威琴、楊怡芬、林宗毅、陳曉雯、黃志明、徐揚純、陳珊意、林雅如、瞿宥明、謝慧義、林佑霖、王郁人、方智鈞、劉淑吉、張翠瑤、洪旺成、蔣耀文、金林任、楊佩珊、李宇汝、吳泓雨、蔡健銘、李枝菁、簡建宏、陳怡博、陳維倫、吳明凌、馮俊嘉、方詩德、顏瑋奇、黃美明、劉文屏、方翰榮、蕭軍妃、陳永娟、貝世豪、朱育萱、唐沛宣、陳俊諺、黃淑娟、楊佳慧、吳皓辛、吳柏光、王志吟、陳于庭、林淑惠、游南婷、陳貴弘、林智毓、郭恆舜、唐志翔、李千劭、陳思惟、陳婷泰、曾雲水、鄭文和、呂重蓉、張美珠、鄭少能、吳馨玲、顏茹瑞、吳姿珍、張偉音、陳萬發、劉政霖、陳惠淳、張佩穎、黃彥良、陳美惠、宋得亞、楊彥青、張凱琪、沈奕吉、蔡佩珮、吳宜廷、杜湖洋、張恒德、陳永苓、吳寧謙、楊季杰、駱枝亮、王裕峰、王肇雨、汪辛淑、魏秀慧、魏智文、姚思妤、吳姿淑、陳彥良、郭淑娟、李俊彥、蔡宜君、陳盈君、王建輝、黃筱婷、楊玉玲、陳家榮、李孟修、王佳玲、劉尹雄、邱芃育、張嘉豪、林郁松、劉至君、陳柏英、林怡君、陳尚海、林于政、陳麗霞、張怡靜、蔡政信、陳文紫、宋信茜、林聖傑、黎宇華、劉吉綺、黃逸凡、曾亞亨、張丞典、楊鎮怡、黃大鈞、黃淑惠、蔡堯映、鄔俊良、文初念、陳忠定、黃俊瑋、劉育達、柳淑娟、陳國琦、朱彥志、陳春亞、吳允文、陳柏然、劉幼原、鄭彥中、駱孟修、林朝麟、林威德、李宜茂、黃榮星、宋宛君、宋玉鳳、黃志平、林惠季、林淑玲、蔡凱倫、張曼中、鄭善盈、張孝亞、姚羽怡、張文修、謝家豪、吳育萱、夏怡均、巴興玉、陳建佑、吳秉勳、陳雅惠、王佳霖、李玉軒、黃怡丹、盧康政、黃琬淑、黃珮如、杜雅婷、陳盈君、宋嬌利、鄭寧彥、謝旭音、蔣新馨、陳信堯、吳志群、曾佩行、陳守天、連宏木、鐘雅文、袁曼瑩、林欣怡、吳紋胤、吳緯萱、陳偉倫、王美良、鄧淑如、楊雅雯、林韋東、姜佳欣、楊璇香、蔡嘉謙、蕭林原、程琪亮、林怡君、羅啟柔、吳語枝、李虹諭、李佳義、楊珮雯、周進鑫、王容柔、李怡法、林士琇、陳淑岳、陳佩蓁、張山寶、劉浩瑞、蕭俊德、童燦賢、葛宛蓉、吳美婷、涂璇伶、郭惟麟、陳順弘、林郁文、劉治凌、劉紀博、孫威廷、楊天惠、李文秋、陸淳帆、盧文宏、林思伸、尤聖冰、吳益樺、陳夙昇、許子軒、涂亮聿、吳夢綸、賴祖羽、白玉萍、王家祥、程可欣、吳志偉、蔡宗舜、藍世傑、何雅玲、趙建霖、陳秀盛、張嘉霖、林宛帆、周樂修、錢勝傑、楊心怡、黃馨祥、王嬌江、樂志瑋、童白佐、盧家華、黃柏茵、吳俊霖、陳佑虹、王雅婷、姚興沛、梁誠寧、張慧敏、顏佳蓉、邱奕添、賴翊依、林思蓁、顏雅琪、麻建輝、魏俊銘、蘇冠廷、李淑威、張瑋慈、吳建成、黃柏新、陳皓宇、李豪冰、白倫寧、張志斌、葉美君、賴琪修、陳雅玲、羅培琳、王宇軒、許晴蘭、陳逸仰、呂馨慧、邱宇緯、陳瑞華、王明原、張右學、張凱孝、吳柏婷、鄭家豪、賴偉倫、鄧智傑、蔡秉昌、柳國華、王祥谷、吳坤恆、徐俊豪、李鈞法、陳柏堯、傅育德、郭千慧、陳英法、韓惠珠、陳俊賢、周健貞、陳珮玉、李怡亞、蔡欣宜、劉玉伸、林育亞、王詩娥、林俊傑、張彥翔、李佳穎、黃卉惠、許淑倫、許孟儒、李政宸、蔡怡菁、鄭承恩、劉俊群、黃志啟、夏明杰、張彥璋、陳展洋、楊弘純、陳淑音、游憲修、黃廷方、柯筱涵、黃毓宏、徐偉玫、梁苡心、林怡文、翁純添、謝夙琪、趙家維、金郁婷、陳鈺婷、張桂雄、黃婉雨、羅冰芳、吳佳蓉、唐妤書、黃嘉恭、陳玟達、謝雅玲、祁景音、謝群胤、柯巧茵、方木慧、蘇樂坤、柴振豪、劉淑玲、張昕珊、黃士茂、暴邦貞、金憶新、林玉善、黃彥昇、楊韋依、藍雅竹、黃有辰、蔡宗靖、孫凱鈞、陳韋易、林子法、李孟恬、柯珮甄、李淑玲、楊雨芝、賴美娟、黃淑剛、黃怡婷、蔡睿、吳淑和、楊韻博、柯夢緯、林聖方、劉于倫、陳立茜、陳瓊揚、王淑婷、張淑萍、張台喬、方雅玲、趙淑梅、陳宗隆、周南音、周佳琪、吳淑芳、葉尚鳳、林志強、陳映桂、許剛亦、王陽麟、張嘉芷、李孟珍、許偉翔、黃佩珊、林義英、翁智文、謝先帆、謝平堯、謝耀喬、高鈺雯、林彥良、吳家佑、陶怡婷、蔡茹胤、程富秋、莊政儒、劉靖吟、黃佳蓉、李淑玲、陳禎陽、趙宏偉、池禎士、林茜虹、潘竣傑、賴為盛、劉琇琦、曹旺吟、王丹亞、徐文馨、林怡真、蔣玟傑、奚亮駿、黃秀娟、陳文福、王育誠、張英強、蔡俊紋、陳政哲、吳健豪、林雅揚、賴宜靜、陳雅伸、龔淑慧、陳秀玲、胡文馨、楊俊杰、陳江佐、陳玉凱、袁怡菁、董劭忠、羅彥宇、林素城、曲孟財、陳巧花、謝堯旺、陳慧毓、陳慧諭、郭瑩郁、李吉全、楊佳怡、蕭盈友、杜志平、詹雅雯、葉瑞夫、連姿萱、洪昇清、陳盈美、王有凌、阮盈如、夏淑真、王有火、王淑芳、謝建春、林禎學、馮玉娟、張儒人、翁星信、蔡明勇、劉漢佐、林鈺均、李孟雅、梁翊玫、李佳桓、林雅玲、徐可欣、酆仁妍、黃美中、謝姿吟、王冠廷、林雅琇、孫國明、蔡宗峰、林冠宇、林安琪、伊淑惠、林雅旭、黃峻鑫、張盛政、宋喬洋、曾泰娟、張美惠、吳俊茵、廖孟軒、涂學樺、林淑美、林敬琇、謝宗翰、陳智翔、貝啟年、潘宏儒、唐明杰、蔡宗人、彭勝傑、童先傑、呂映韻、謝依孝、張麗珠、方佳夫、童重勇、吳星盛、黃子昀、李綺俊、陳惠茹、黃玟亮、陳士恬、王怡璇、金慧婷、陳致秀、張承雪、蔡傑寧、弓克齊、袁茂君、林子書、儲守富、黃麗中、蘇人發、丁一傑、林家岳、李琳幸、吳啟盛、陳彥宇、陳雅慧、陳君香、柯兆妏、劉冠宏、李文欣、王詩涵、梁吉芸、王邦勇、陳天翰、張竹陽、陳惠珍、彭林劭、吳宗芳、陳鈺傑、彭成泉、徐耿利、張建霖、曾婉婷、方胤秋、楊沛麟、李美任、賴怡威、陳永勇、黎彥志、林子瑞、楊忠發、陳怡玲、劉建良、林秀安、王如軍、趙元睿、鄧胤玫、蔡惠玲、王興忠、莊雅萍、謝姿穎、陳靜雯、黃淑勇、陳辛萍、張韋成、蕭麗琦、賴泓花、廖婉琦、李家瑋、錢彥儒、楊俊凱、李雅萍、林曼花、張玉紹、柳薇宸、李佳欣、金鈺富、謝佩雲、馮勇志、林淑芳、傅奕君、鄭雅婷、賴宜靜、吳秀慧、陳振東、邱勳佐、楊鈺宸、賴家豪、洪淑謙、陳金宏、陳俊均、邱晴岑、林木岳、謝孟修、王智群、趙宗韋、李宜岳、陳佩韻、陳世任、李志瑤、賴雅婷、陳孟修、陳雅君、李孟倫、潘紫雄、王明惟、洪倫寧、鄭宗翰、魏淑娟、胡玉婷、吳宗康、黃俊榮、滑歡易、陳宜君、吳珮君、陳佳臻、楊薇星、杭昆霖、陳鴻正、侯柏廷、李常珍、林儒誠、林育沛、鄭宇以、許仲儒、鄧賢玉、鄧雅南、林冠妃、黃昶裕、張群容、姜淑娟、王峻萱、楊怡珊、易文君、倪育菁、顏易韋、蘇佳松、陸冠廷、鐘柏翰、宋右政、李展姍、於馨水、蕭士孜、王隆念、胡妍火、郭貴雅、林威任、沈冠宇、林士伸、王雅竹、鄭妤新、陳佳霖、蔡宗真、陳清仁、曹佳玲、林立來、黃靜宜、王明義、王俊杰、金維萱、張昆毓、朱怡君、袁佩如、林靖雯、李惠玲、詹昆欣、劉珮瑜、劉文婷、鄭昭智、唐怡君、黃映軍、楊倫謙、林筠佳、陳其星、林育隆、盧文君、黃俊逸、黃瑋婷、蔡惟孜、劉筠政、鐘吉州、黃香君、張丹樂、戴美惠、林哲瑋、張瑞智、王淑娟、林孝臻、蕭竣辛、張維欣、魏少希、謝旻秀、黃雅音、張凡軍、許美文、喬歡柔、張秉蓉、林必邦、許弘昇、林佩琦、林建一、楊凡蓮、姜淑貞、林淑順、陳漢尹、蔡秉一、羅佩珊、黃仰吉、黃威廷、吳翰薇、林仲然、林嘉正、許幸方、葉宛君、張珊恆、魯秋燕、羅政廷、張雲貞、劉秀禾、陳瑋貞、江佩如、趙憶妤、賴家豪、黃俊謙、何建成、賴平希、許莉宣、彭岱胤、李佳慈、游伊燦、余子翔、李宗翰、陳國華、蔣宛玲、林淑霞、杜雅音、孫鈺桂、馬榮玟、梁夢佩、黎春智、陳惠珠、侯素華、王郁堅、蘇玫吉、許雅慧、黃智昀、柯與聖、林彥雨、王禮函、林展其、吳政凱、林政真、黃筱峰、陳維育、黃智易、蕭湖財、李志維、廖家康、王佩芸、羅純寶、林志輝、江羿惟、童玲春、許琳卿、黃若興、王怡伸、張雅竹、許怡君、鄭大佐、王佳柏、何芳如、吳怡雯、袁坤玲、陳冠南、賴惠麟、陳政廷、趙羿凡、王耀中、陳彥廷、鐘智翔、陳依育、彭昭州、張宗穎、林靜伸、袁真潔、溫明慧、陳婉如、李昱宏、黃雅雯、馬修翰、劉金鳳、楊惠雄、鄧宗禮、曹呈幸、張雅山、傅家瑜、陳宜珊、吳淑娟、黃雅芳、徐夢琳、李佩珊、戴雯銘、李依善、林平辰、彭宇翔、馮延生、徐漢香、朱佳玲、黃莉雯、林兆谷、黃郁利、洪雅雯、林孟軍、蘇巧喬、許莉婷、彭靜宜、張苑峰、陳志成、陳家瑜、周佳穎、潘初俐、劉冠宜、周志城、朱科其、張晉雄、楊婷婷、方宜蓁、家佳穎、陳美玲、黃靜怡、周欣穎、黃尚憲、藍亦鑫、郭映強、李柏吟、林正沛、王肇幸、陳薇群、崔禮仁、方婉誠、郭憶典、盧怡星、王怡其、陳仰芝、林郁剛、陳彥洋、林韋伶、王淑軍、趙蕙尹、張景孝、張瑋彬、劉若愛、陳佩禮、葉怡源、蔡佩花、蔡佩奇、倪智翔、王菁鈺、吳宛男、王鎮延、於可月、周彥儒、許耿亮、陳萱霖、張志豪、李嘉妃、林子翔、陳信蓁、曾友典、黃淑玲、王佳霖、吳勇沛、宣勇心、張香火、黃珮桓、陳瑋婷、李佩琪、朱恩德、戴怡君、林淑其、林佩玲、許家聖、陳長原、吳江湖、袁紹盈、李俊南、許淑婷、陳品欣、林淑娟、郭燦德、彭士南、金綺沛、張雅輝、韓喜明、陳火喬、楊盈君、杜中云、潘信城、張倩季、林宗汝、方郁倩、黃詩帆、林芳瑜、高介冰、陳慧玲、張佳季、陳虹梅、林怡君、楊姿蓉、張展恭、吳文豪、儲振宇、顏吟盈、林佳美、林思翰、郭光倫、林丹竹、林振瑋、祝筱佳、楊惠妹、陳秋燕、邱新輝、陳雅惠、林智昕、黃萬學、施振啟、王枝恩、陳婷謙、鄭慈恆、王丞婷、王郁文、劉雅慧、夏儒辰、張育傑、馬宇軒、崔瑋婷、李依潔、黃馨恬、林俊蓮、林于宣、張婉瑜、許芳博、黃皇蓮、沈奕翔、賴千惠、周梅政、陳惠一、蔡璇屏、劉淑真、杜雅云、張志泉、楊江丹、邊緯珍、李冠茜、孫禹忠、王馨白、宋宗達、郭若林、楊婷賢、許名中、王揚儀、林玉海、陳英文、劉俊成、謝賢倩、陳胤忠、蘇先男、林靜達、鄭介彥、高怡君、袁苑原、洪軍琳、陳宥伯、李國瑋、沈歡儒、謝詩涵、陳信凌、葉培義、楊聖寶、王瑤毓、高明輝、戴詩樂、丁珮如、吳秋甫、蔡東發、柯馥綠、楊紀婷、王健美、蔡胤書、陳怡君、傅菁妍、金敬輝、蕭惠君、黃信喜、李偉智、馮初泰、簡家瑩、陳俊安、涂志軒、王雅鈴、林美新、龐依萍、呂仁傑、呂雅婷、孫佳玲、方桂威、羅羿彬、劉怡偉、曹進弘、徐芳弘、林筠沛、吳彥儒、陳泰美、何祥彥、曹淑惠、王婷婷、陳俊毅、張羽舜、姚清樂、吳俊妤、曾珮青、王家宏、駱哲豪、吳宜修、王智智、林玉玲、吳吟名、王士瑋、楊家良、冉心盈、林惠如、王竣傑、張瑞佳、梁亭一、張小豪、楊禮然、李怡禎、林秀珍、吳銘聿、陳昆霖、賴麗岑、吳馨迪、黃群昇、李珮元、吳承亦、藍貞書、王財安、趙雨儀、李冠宇、劉文瑜、陳彥珠、鄭義娥、林書瑋、陳呈菁、李婉合、成雅芬、王彥君、林如穎、謝俊嘉、周嘉玲、吳晏來、梁家慶、游峻香、黃建亦、邱琪亦、涂至蓮、陳芳如、黃琦正、林雅雯、吳秉弘、梁向瑞、魏紫白、郭寶宣、許沛夫、林逸夫、林靜音、盧文傑、吳淑成、謝明昌、顏建宏、陳伊宸、蔡秋霞、曹麗娟、劉玉幸、蔡有興、張建智、李明軒、施偉哲、程美君、林懿琇、戴孝容、蔡佩璇、謝宗盈、黃茹雪、楊怡伶、楊博仁、張惠雯、吳家瑋、李孟山、李婷婷、李曼旭、謝鎮迪、陳嘉謙、魏于珊、戴岱歡、張淑如、許姵岑、潘仲恬、江竣傑、胡則火、李佩君、陳怡君、張佳琪、顏依婷、翁欣穎、陳思婷、張昀辛、陳志堯、吳佳伯、郭晴芸、張惠年、王美惠、林人豪、黃思恭、崔彥璋、李碧亨、黃筱凌、吳淑如、陸凱倫、張思貞、高柏鈞、林婉瑞、劉仰蕙、傅婉玲、鄭宏達、鄭香任、沈雅雯、連力博、陳佩珊、王右興、連志豪、陳婉治、王子揚、王建成、陳威妮、劉怡佩、昝愛映、劉思涵、黃莉禾、王潔全、丁馨儀、黃昱迪、陳于珊、蕭育維、葉嬌妤、陸林銘、朱柏宇、楊智凱、牛嘉鴻、吳翔威、何淑娟、賴思婷、柴盈君、高孟哲、陳聖傑、林睿蓮、林怡靜、林孟易、陳國平、賴志偉、姜欣怡、朱苑蓮、張哲維、吳怡靜、張珍枝、陳淑敏、林柏豪、王佩樺、潘成歡、黃國慈、陳希青、蔡佳萱、彭火光、張冠廷、林漢迪、張怡君、王彥志、毛泰禮、鬱雅禾、吳泓智、金進瑜、李木軒、林冠志、梁心怡、蔡綠仁、張怡萍、黎建宇、高欣宜、陳維倫、黃文雯、彭志宏、張怡君、邱禎賢、許育綠、謝美珍、許文士、陳碧亮、郭向福、黃映白、林淑真、劉台強、張馨茹、鄭賢芸、翁信宏、張志易、詹孝州、林秉勳、陳淑娟、林雅惠、傅婷奇、林慧書、李俊茜、劉宗翰、陳姿軒、謝淳邦、蔡宜喬、何淑珍、劉雅茹、李詩涵、蔡佳州、勞振恆、張妍裕、鄭政儒、王昱靖、韋千姍、林明輝、黃玉禾、郭翊虹、蘇明杰、王秀妏、羅宜潔、王珮瑜、方緯雲、張政哲、謝承陽、陳尹月、翁天淑、謝容航、陳金柔、張盛妹、柯宛蓉、崔承穎、鄧夙仲、陳政廷、蔡博紹、王東山、杭佐邦、申茜雅、林威廷、許美珠、劉志虹、陳江樺、嚴明輝、林明瑜、袁寧禎、賴琦學、陳建瑄、林璇揚、謝文春、黃雅婷、王彥雪、鄭文方、張嘉云、張鈺婷、李柏原、黃曉玲、李志嘉、葉怡君、宋瑋玲、沈大鈞、劉美玉、鄭方杰、駱定仁、李建銘、李宗誠、郭維倫、鐘仁傑、吳宜娥、劉延芬、吳宗以、張雅婷、廖雅婷、劉世群、陳美君、林立俊、朱莉典、周守紹、林逸哲、賴婉達、陳韋妹、杭仕宸、林玉任、鄧瓊慧、陳盈偉、馮婉婷、賴小依、陳康孝、林政亦、侯建德、沈喬傑、黃靜怡、吳淑一、張肇歡、傅雅惠、駱語忠、黃國松、王依偉、孫惠雯、李美容、陳昱雨、劉岳亞、王軒豪、曹左雲、郭偉智、陳伯鳳、陳詩娟、楊雅怡、李凱鈞、劉歡治、賴英宏、鄭欣瑜、藍婉菁、李江雨、姜文傑、林尹亞、謝莉婷、吳君豪、柳芃歡、謝育誠、李樂冰、魏靜宜、李欣麟、蔡依恬、謝炳柔、林思湖、毛承翰、王智翔、項麗娟、秦彥霖、梁亭緯、錢宛臻、鄭婉育、李佳雲、陳均裕、李添德、陳家妹、劉心星、林書秀、張佳慧、李婉珮、袁宜善、白敏憲、陳宥延、林瑞君、黃家軒、郭懿琦、黃意婷、牛俊宏、謝季薇、朱介勇、藍俊毅、黃文娥、藍辛丹、張坤哲、邵德柔、蔣恭侑、李竣珠、劉雅琪、劉家瑞、陳健豪、王志威、賴俊憲、陳智芬、劉益茵、余智彥、張香昇、吳予樺、蔡美珠、陸怡雯、林菁旭、許瑩英、童浩林、邱綠妃、楊姵怡、郭志成、劉水丹、陳瑋順、楊冰淳、蔡明凌、李美玉、張雅奇、張靜如、羅淑卿、李志翔、黃為俊、張夢奇、黃昱凌、張淑玲、強丞淳、周佑仰、許純宇、李喬清、邱昇智、林珈雯、黃世萍、鄭凱雲、黃隆鈺、周惠任、詹承萱、毛采仁、蔡政鑫、邱瑞延、張怡文、趙玉燦、謝怡伶、陳彥岳、毛瑋婷、郭民孜、蔡采康、林筱婷、潘安江、郭敬妍、王綺南、陳宥城、張維軍、王定舜、林惠雯、楊美玲、林曼屏、劉玟中、劉佳慧、劉士凱、周喜順、鄭文婷、張惠妹、柯勝孜、陳群雲、蔡孟倫、林國紹、楊仁杰、黃鈺桂、吳白任、吳學凱、張韻昀、曾俊嘉、張佳伶、郭竹其、陳彥君、王淑迪、鞏昱貴、賴士豪、賴婉婷、吳綺卿、唐慈舜、陳思陽、廖家良、李宜靜、許綠謙、金育如、吳宗穎、劉威忠、陳筱汝、林書映、陳家瑋、鄭雅菁、簡志樺、徐聖傑、袁珮雄、李麗美、唐宛儒、李惟亨、劉馥馨、曾家豪、屠智富、郭婉如、陳季興、林盈君、邵雅芬、馬俊廷、鄭婷、陳明哲、葉念華、戴乃文、林怡靖、林智翔、羅維倫、劉安堯、陳家盛、許承翰、李郁琇、王玉雯、陳堯娟、黃惠雯、石人聿、張妙南、朱淑慧、吳麗美、張怡成、黃秀玲、陳有愛、杜柏翔、楊冠廷、王淑芬、陳家偉、劉詩婷、馮筱婷、傅玉華、陳柏成、王千惠、謝淑萍、黃山東、賴怡安、黃俊蓮、王淑惠、李欣福、李廷一、黃世昀、李政妃、張秀玲、張智男、溫雅慧、詹若珮、陳卓士、謝芷珠、陳牧青、楊逸群、陳思婷、鄔奇佩、洪政憲、楊建宏、王婉瑜、謝政芳、李國易、陳萬穎、呂瑋玲、黃建意、陳志中、胡翠雲、劉懿緯、陳淑珍、洪淳文、劉怡靜、陳彥偉、黃木山、張宏達、黃彥安、賴韻昆、方桂彬、張惠婷、廖偉霞、張建文、劉建群、楊俞喬、連怡萱、黃賢亮、鄭子豪、李瓊芃、林秀冰、蔡宗平、平亞吉、李昆霖、張惠白、張靜如、林君佳、陳君念、許建輝、蔡胤文、黃怡夢、黃文龍、洪毓嘉、林敬意、王惠雯、王傑純、林瑤虹、楊欣潔、吳舜文、黃志慈、林杰發、潘山雲、鄧志傑、杜耀茵、謝佩陽、謝彥博、蕭林蓁、吳志成、李惠雯、蕭育燦、林怡芳、李素豐、鄭進穎、韓國宣、祁宜珊、吳奎哲、高典夢、李宜宏、羅雅筑、林淑真、曾儒州、郭鈺婷、蔡宜富、方耿勳、王秀彬、林聖玲、郭瑋婷、林雯緯、張琳年、白佩成、賴奕君、李健如、陳志宇、李淑能、蘇佳玲、林珮瑜、周培倫、李文彬、陳興靖、孫佳琪、謝曉薇、鄭木幸、黃志書、吳靜怡、張孟儒、黃與財、陳育財、黃書瑋、柯宥睿、陳軍瑤、郭冠伶、鄧順梅、李玫學、楊雅玲、宋仕茵、沈柏良、李舒涵、陳先宸、陳美玲、林家茹、陳海依、張佩芬、楊柏翰、林怡婷、吳志孝、陳慶昌、蔡美華、林怡伶、李秀芬、翁恒鴻、黃可琳、張鈺婷、鄭建夫、陳建生、楊昶花、劉惟睿、韓嘉惠、游怡君、李瑋婷、梁堅柔、許建宏、王佳慧、黃憲傑、蔡美華、陳俊良、吳雅惠、李宗陽、陳雅茹、林文傑、吳家豪、黃孟儒、馬歡穎、郭雅妮、謝玉華、徐枝屏、楊淑偉、吳惠宇、陳佩珊、柳苡沛、曹宗水、李宥江、陳賢寧、林詩名、劉世偉、陳逸霞、張嘉芷、張中菱、張家維、陳竹玄、賴翊揚、黃慧君、謝怡樺、李俊凱、陳俊州、倪鎮慧、葉士賢、林志新、侯怡欣、陳家琪、謝佳義、陳宜心、陳彥富、曹淑玲、曾雅婷、鈕志杰、藍水翰、鄒志鴻、劉志鴻、郭家真、謝威廷、王偉能、楊依蓁、李佳燕、毛慧娟、李佩芷、席欣怡、方芳瑜、蘇維哲、程芳禾、曾泓洋、周榮佳、連佳儒、邱俊義、張雅婷、張毓芬、溫穎凌、顏隆龍、顏旺火、趙呈柏、韓莉婷、王怡君、鄧至康、葉巧恭、林枝利、陳靜能、蔡冠中、林芃智、陳雅茹、楊慧郁、鄭柏成、陳佳伶、吳秀芬、連祐枝、張筱婷、曹雲順、陳曉玲、王詩希、吳亞人、黃家屏、賴山其、溫郁翔、林華鈺、陳芃宸、侯羽禮、楊佳琳、李怡君、林冠齊、張淑真、謝欣蓮、張惠臻、馬翔紫、楊麗芬、柯順喬、倪智強、林建弘、蔡吉謙、涂燦季、周振善、張忠嬌、黃常屏、蔡群香、林書霖、張佩君、許怡如、林郁婷、陳柏州、王宜妃、李佳穎、林士瑋、吳惠以、楊志妮、羅潔娟、潘怡君、張怡臻、謝惠婷、王佩怡、許佳霖、黃子偉、楊珮君、夏孟穎、陳韻君、王俊廷、黃淑萍、林思中、陳文俐、林上花、別麗君、陳雅梅、吳秀玲、張倍君、曾亦成、黃淑貴、鄭瓊友、曾友弘、王俊琬、王姿輝、陳紹林、許士賢、謝力堯、吉玉娟、施佳玲、姜茂能、張慧玲、陳承翰、王欣怡、黃凱婷、林皓義、陳美紫、韓怡靜、李則聖、吳怡靜、呂庭康、劉秀英、黃品軒、徐惠妍、王俊福、黃柔念、張曉薇、吳建瑋、曹辰淳、楊舒禾、吳美惠、林姿其、徐偉廷、劉志豪、黃文婷、王大奇、童千育、黃百聿、蔡志琪、戴慧玲、蘇國華、戴哲維、陳添明、楊為桓、程新玲、許耀中、陳凱婷、吳佩雅、鄭怡靜、羅昆隆、陳淑光、陳昀佳、孫燦丞、許韻如、林思冰、陳佩平、張柏士、王雅萍、夏育春、張俊成、林幸博、鄭慶生、賴佩珊、黃麗陽、黃佳昆、陳志山、林薇宣、黃珮雪、杜冰信、楊人奇、林韋枝、許智凱、鄧建勳、劉季修、黃康能、蕭文傑、吳欣宜、張家齊、林雅堯、趙秀美、劉珮定、曾惠玲、郭玉芳、劉美惠、張志以、劉永紋、吳政瑄、方旻怡、林文彥、吳有函、林佳芳、蕭辛駿、賴雅嵐、蔡宜貞、林秉胤、周豪源、方紹星、潘又邦、張博博、王漢梅、黃子傑、黃瑋財、張毓冰、張俊佑、陳韋諭、林建成、邱東舜、榮俊韻、周宜民、李虹來、李聖文、許政峰、林思源、鄭宇香、毛舒婷、徐彥宇、黃哲銘、黃育儒、杜良菁、溫小虹、黃建能、姚淑婷、陳志明、林裕昀、林淑真、吳淑萍、楊志雅、李昱廷、陳志堅、黃秋萍、蕭盈君、陳竹康、詹桂柔、周喜一、陳信帆、楊邦博、陳柏勳、林筱侑、游美惠、田惠雯、劉惟鑫、鄭淑芬、錢盈君、楊佩璇、林志翔、蔡佳真、陳耀德、汪怡禎、王逸凡、林士蓮、吳宗士、林詩岑、謝曼妹、時左霞、陳名江、毛易俐、林雅惠、蔡婉如、陳宗翰、魏淑娟、陳佳蓉、葉誠宏、吳宛仲、張中榮、王茂蕙、劉璇綸、席筱涵、郭嘉文、路昌文、張智傑、黃傑聿、張凱映、王怡孜、陳南海、梁麗雯、蘇淑芳、張筱涵、許蕙梅、陳永定、張佩珊、謝佩華、鐘偉倫、李姿伶、賴真靖、楊揚駿、蔡彥廷、姚香君、傅俊賢、宋佳穎、賴榮文、曹致遠、黃彥宜、陳萬馨、袁軍欣、蔡孟以、蘇方雅、周雅云、浦堅斌、吳明念、吳佩天、張如花、楊允亮、柯姵君、謝如怡、黃志文、吳慧玲、謝麗雯、陳宗威、黃學明、許宜欣、梁宣寧、劉彥良、王筱達、戴宏貞、鄧怡如、韓品歡、吳雅玲、李柏廷、劉恆名、鄧紫亦、陳美佳、宋景珊、王翰妃、趙樂方、鄭威宇、陳曉薇、邱若翰、吳石丹、侯維倫、彭昭陽、江啟良、吳秀美、劉佳欣、甄茂賢、李佩珊、陳秋生、林麗邦、黃怡亦、許燕紹、姚佳霖、蘇彥良、王信宏、邱鈺雯、吳偉婷、張瑞裕、簡芷彬、李怡泰、朱淑婷、蕭宣丞、林詩念、林典凱、涂志韋、余燕諭、黃秉白、鄭嘉琪、陳致遠、王嘉惠、賴俊虹、林宏紹、陳弘茂、吳佩蓉、黃玉真、方玉婷、林秀琪、藍剛銘、鬱容峰、沈瑋婷、王文馨、陳俊珠、葉逸凡、林英傑、李宗翰、謝育萱、謝皓翰、梁維哲、林奕洋、鄭心怡、喬雅芬、洪予平、田盈嬌、連子帆、張家谷、吳孟心、陳致汝、王志臻、林宛真、黃莉弘、李國啟、鐘嘉文、劉啟彬、蔡台憲、陳明凱、王凡均、鄭鈺婷、王智人、吳明秀、劉淑真、吳珈定、徐偉光、吳宛平、李佳慧、江嘉祥、張士弘、潘旭倫、林文正、張之春、楊月妤、黃雅雯、王芳岳、劉清人、謝武財、陳智芸、孫秀蓉、劉哲嘉、張雨發、林靜怡、張涵花、葉嘉易、蔡百臻、張佩君、郭立梅、侯俞亞、李思翰、陳惠政、李琪財、陳冠宇、張俊傑、劉少芝、林家銘、張靜宜、王兆亨、黃合冰、奚艾杰、林瑋凌、羅義聿、鐘亞琬、吳明輝、楊婷婷、溫琇恩、許昀華、黃偉倩、馮軒豪、徐怡憲、沈建屏、陳冠虹、張奕君、徐毅綸、蔡政岳、李民姍、李庭心、錢靖念、賴育儒、林泓岑、王財琪、陳韋伶、傅雅文、黃彥云、李順燦、陳彥廷、陳玉珠、賴明哲、毛彥文、張康薇、張剛博、傅琪韻、包伯孝、侯于珊、李秀娟、涂秀玲、張孟宸、杜聖傑、蔡家弘、李國鴻、潘佩璇、張靜怡、陳致勇、林淑芬、張淑玲、曾芸華、沈吉佳、劉冠琦、張卉昀、林子源、蔡綠新、方偉成、盧士德、柯孟儒、林宜山、朱婷婷、黃卓利、郭毅睿、黃毓和、張克恬、王聖一、張士銘、游聿嘉、項詩雅、馮亦昕、陳珮法、陳怡靜、陳建德、林俊霖、程歡睿、張枝添、陳玉玲、許依婷、李政哲、余政勳、張上玄、陳亭成、陳明杰、侯晏廷、楊智翔、劉蘭玫、楊景堯、黃曉玲、高恭友、王文馨、蔡明東、江佳來、劉雅軒、陳筱雯、李君夫、楊京喬、陳宛一、王俊傑、鄧佳純、蕭韻如、張彥文、陳政辰、楊金妮、陳凱翔、許智鈞、吳千伶、康姿君、蔣易侑、郭俐萍、吳青佳、吳靜雯、張孟勳、楊哲維、黃淑倫、王幸妮、黃哲宇、蔡宜琦、謝佩明、黃雅行、郭怡孜、夏劭季、余財湖、李玉娟、李仕寶、杜羽平、林政福、翁其夢、李珮玉、李玟群、陳怡如、楊佳軍、廖上芃、扈清城、趙佳蓉、劉惟順、黃淑旺、吳怡筠、連怡如、宋淳蕙、曾彥利、王怡佩、李姿源、孫辰延、吳仰萱、陳郁瑄、李依瑞、張雅芸、藍冠宏、馮惠茹、吳佳和、陳瑩東、楊淑強、蔡鈺定、林劭天、陳雲慧、陳敏凌、王聖明、陳永苓、溥富珮、蘇瑤軍、賴竣傑、吳佳蘋、姚淑昀、蔡原宇、余介蘭、馮香白、吳韻婷、溫佩蘭、謝純源、張子芸、湛正偉、楊佳慧、蔡維倫、曹佩芬、劉伶燕、張宛蓉、張維倫、王建佑、郭怡君、陳建源、李鑫、林漢緯、陳予富、陳思香、賴筱芸、張奕廷、蔡明玲、吳家豪、朱佩玲、郭志豪、林秀彬、王昌雅、張珮瑜、潘淑貞、楊凱茵、池盈如、林育德、李昌星、禹佳慧、張家宏、吳協樂、林翔亨、李宜音、何承穎、林慧珠、張維屏、施旭如、林一珍、周宜臻、謝家豪、歐大鈞、吳思芝、許嘉志、張伶惟、陳瑋博、高青臻、馮允妮、黃郁原、李欣怡、張建宏、程仲能、林佩芬、葉和年、駱志文、林智仁、侯佩芬、劉佳真、陳鈺婷、王建智、郭政哲、吳怡均、陳睿文、劉彥宏、汪彥智、吳彥璋、杜靜怡、陳懿諭、趙廷香、陳琬婷、謝佩堅、張鈞茹、黃佩儒、許邦良、金明哲、吳子芸、吳柏伸、王貞儀、徐永芃、蘇馨儀、黃麗人、姚芝音、周筱涵、蔡欣怡、彭柏成、吳啟樺、許牧佐、陳亭辛、林冠全、彭尹宣、林真瑄、楊紹紫、陳昭念、楊凱文、劉惟琦、王思州、李孟民、林佳蓉、陳秀傑、韓慧玲、吳盈秀、魏俊宏、陳雅萍、陳文亞、陳聖傑、王士薇、陳怡文、張哲行、胡必利、陳合美、楊怡君、李如靖、陳靜芳、郭家豪、呂蓉泰、陳文東、劉桂霖、賀伶馨、楊宗山、游倫儒、羅宗儒、林恭俊、周蘭瑩、戴倍恭、吳佳瑩、李俊如、夏立玟、劉季威、林明忠、昝雅婷、謝琬盈、王小麟、蔡明仲、陳佩蓉、馮竹強、謝友春、程奕恭、蔣欣典、林宗富、季孝胤、倪軒豪、劉晏貴、張文傑、王憲歡、郭彥志、駱雅琳、陳玟行、李雅萍、李彥廷、吳俊賢、林芳宣、洪淑俊、林芷韋、徐志宇、陳家恭、胡翊琇、梁禹紋、張博正、廉奕君、林念樺、陳延斌、吳雅雯、柳欣怡、陳雅雯、王益斌、鄭佩琪、吳政凌、陳冠柏、鄭柏鈞、程泰平、許雅珠、曹秀哲、陳盛海、姚雅惠、陳堯寶、陳瑞泰、陳怡萍、溫介桓、賴慧君、黎林禾、陳俊賢、阮舒義、李淑娟、傅竣傑、黃欣怡、連雅雯、林淑平、黃鈺谷、黃惠純、林怡欣、王筱苓、陳兆柏、張建銘、賴詩孜、楊馨東、張偉倫、陳嘉勳、劉子軒、傅珮瑜、曹喜明、林俊柏、王玉婷、王建宇、林維倫、夏治憲、陳育瑋、蔡雅文、謝佩州、劉俊法、林喬亞、陳辰竹、吳金喜、朱昌然、黃子鑫、林宇軒、陳若宇、李志文、黃千靖、林文君、徐宗憲、林隆甫、陳曉均、周素文、鄧佳琪、柳梅男、郭中蘭、洪姿君、蕭辰正、賴志麟、蔡志豪、王仲易、周世豪、王偉昀、洪威廷、黃俊傑、黃信宏、張函岳、陳惠欣、張淑花、吳婉玲、吳承翰、劉孟勳、簡湘臻、林逸光、楊哲豪、林怡斌、吳佩芸、陳宜秋、劉雅萍、張修興、林鈺亞、呂家豪、張榮旺、羅憲玟、劉璇蘋、陳紫盈、李佳以、李友松、黃敏妹、呂佳恭、徐胤誠、林曉堯、王智傑、毛儒妤、吳靜怡、吳佳男、吳書瑋、陳淑珍、張幼人、廖俊賢、蔡筠虹、李又菱、陳紋薇、謝文彬、盧婷婷、何恒維、黎彥伶、陳敬紫、吳佳芳、孫培辰、陳冠玉、李建緯、陳勇夫、吳雅慧、楊孟修、吳淑敏、溫振瑋、陳冠勳、謝欣怡、陳柏謙、張冠勳、王麗英、趙倍順、楊新容、周麗喜、王梅仁、陳麗惠、趙小新、張允志、林思穎、王于婷、張英萱、吳志如、沈姿蓉、李法憲、李心怡、陳姿宸、沈翔忠、王婷婷、林宜芳、陳火恒、蔡重英、何采惟、黃國威、鄧堯白、涂乃劭、趙婉婷、張景啟、王秀秀、林佩白、林智超、蔡春卉、陳映志、鄭靜清、林姿映、鄭新臻、林景琳、黃筠美、汪百生、黃雅茂、王馨文、林玉杰、張志穎、賴宛名、王素貞、孫政翰、趙允燕、游家宏、袁美玲、韓美惠、王右美、陳婉婷、楊璇寧、鄭雅君、張邦妃、魏潔真、王佩君、何怡珊、孫雯芃、王柏東、黃哲瑋、陳萬岑、李怡發、王秀霞、鄧怡璇、鄭嘉宏、洪昱杰、陳昭信、張馨慧、林育宣、彭健銘、李俊宇、吳文迪、盧綠平、梁子軒、林賢虹、曾竣貴、元怡君、王子揚、韓郁雯、姚宗穎、楊建安、汪皇羽、江柏廷、楊宗雄、洪邦均、林宜以、吳得貴、劉香君、梁宇發、李嘉愛、趙雅惠、羅明杰、李美祥、林曉雯、林碧映、張琬群、黃依萍、黃博仁、黃淑惠、陳怡君、鄭嘉玲、楊怡萱、蔡佩筠、倪雅雯、邱淑芬、劉鈺雯、蔡奎恬、丁協斌、蕭亦汝、張行筠、魏志定、楊意其、陳韋龍、柳嘉依、林韋鑫、陳為群、王俊霖、鐘宗翰、胡香雅、陳佩萱、柳婷音、陳信辰、王婉婷、黃國玟、符孟涵、蔡育如、吳秋一、陳堯迪、趙佩君、崔宗霖、杭兆名、張冠丞、朱春弘、陳郁羽、黃佩玲、禹雅琪、張筱花、李君音、盧倩燦、謝揚誠、陳思旺、蘇堯憲、鄭姵君、儲友易、李希海、彭淑君、劉佩蓉、游信紹、陳盈卿、林維妤、黃珮君、陳芷惠、蔡政雨、黃婉君、陳明儒、江家良、盧宜蓁、姜牧岑、黃怡蕙、柳雅玲、翁菁洋、王先全、潘靖雯、涂偉誠、林靜屏、溫俊達、廖俊明、江傑年、楊凱淑、謝俊逸、吳書豪、邱芃傑、李筱民、黃淑愛、張郁軒、劉雅萍、王定如、黃舜仁、吳又岑、劉玉音、蒙鎮宇、陳慧亦、杜士紫、汪雅婷、張嬌辛、吳建盈、蔡夢志、吳宜軍、彭肇禎、王育儒、陳與卿、黎彥辰、羅柏翔、鄭冠宇、林素怡、葉夙善、陳真傑、朱貴裕、陸心怡、陳俊安、潘歡玉、林宏善、林必吉、林建泰、陳偉桂、林怡伶、鄭純寶、張維湖、李人豪、簡孟倫、楊玄璇、陳宗憲、王麗輝、蔣柔佳、黃裕木、王子傑、吳倫念、魏筱婷、傅秀琴、黃明哲、李家季、廖麗娟、陳俊宸、楊哲維、阮筠發、陳惠茹、謝揚宣、梁仁旭、周淑惠、趙耀文、彭玲惠、王怡旺、林鈺禾、林麗美、涂智順、楊志謙、謝淑慧、歐陽琪榮、賴志成、曾彥廷、黃淑君、宋志遠、吳新聖、吳秋星、陳芸韋、姜奕苓、蔡思涵、郭子翔、楊惠珠、洪林軍、林芸學、陳麗明、趙廷岑、陳昱汝、張慧君、陳梅容、阮玉萍、張莉沛、葉元菁、楊茹依、王明人、蘇松桓、黃怡心、王佩君、吳雅婷、柳智廷、林羿瑤、陳姵君、陳麗人、杜淑梅、劉怡君、張勇志、方姵誠、諸俊穎、曾恭鈺、陳怡芬、陳佩育、駱宣玄、崔雅茹、陳嬌忠、陳嘉恩、鄭振宇、廖怡如、李弘凱、謝佳旺、張智郁、賴清嘉、吳明意、陳胤芷、郭修儀、陳佳帆、鐘欣怡、陳美湖、王岳坤、白茹恆、蔡佳亨、陳學霞、周佳書、袁杰苓、陳美珠、陳筱雯、吳智翔、陳俊銘、李正亦、陳怡靜、陳文珍、錢志瑄、蔡忠民、黃均任、李美玉、黃于治、陳芝瑄、蔣修秋、游子芸、張夢哲、宮佳慧、張思南、王子芸、吳秋燕、王昱琬、賴俊達、李欣儀、陳志萍、黃舜名、謝慧萍、陳怡婷、陳俊年、黃乃育、羅雅鈴、林立怡、楊介欣、吳秋娟、邱博鈞、黃淑修、邱淑婷、甄夢歡、蕭怡禎、謝歡燦、程阿蘭、陳慶雲、張伶柔、孫佐然、李子鳳、蒙興愛、陳成茂、曾冠儒、彭志穎、王雅婷、賴鳳迪、程紀姍、賴靜宜、沈泓穎、陳恭芸、陳宗霖、林郁其、陳彥人、韓俞杰、吳思妤、施鈺婷、陳雅玲、林坤廷、林之霞、何怡婷、袁芃宣、葉偉成、王玉宣、賴歡瑋、李秀青、林靜昇、王志翔、陳冠侑、溫宗穎、謝文梅、陳平元、張中喬、趙祐誠、梁佑丹、林宇軒、呂士銘、謝豪妃、熊軒郁、郭和孝、金婉如、曾思賢、童品威、林惠雯、王佩樺、王志宇、吳軍軒、杭柏豪、張韋華、陳得正、簡佳慧、王佩君、蔡恭俊、陳莉喜、吳佩生、李思威、王志宇、陳立秋、吳芳劭、王俊德、郭珈竹、陳依侑、許必水、林育法、蔡威廷、徐裕翔、吳孟合、黃郁英、陳于倫、吳美鑫、陳家人、陳瑞柏、劉怡如、王卉凌、林威盈、鄭孟軒、葉韋伶、張勝恭、黃正帆、劉揚方、黃佩玲、陳夢琳、陳冠芷、蔡文婷、林宜真、黃寧岳、毛治恭、闞守純、林琦光、傅儀鑫、張麗君、李淳齊、陳智超、林秀揚、楊昶其、王盈甄、許雅晴、劉麗娟、林皓谷、林雅強、鄭吉鈺、徐美玉、林志芷、陳威任、韓裕尹、葉建緯、王建宏、謝佩希、許景卉、郭佳蓉、李彥廷、郭旻法、劉昱宏、陳雅鈴、林智偉、林智德、藍宗惟、諸鳳珠、喻至竹、鄭陽賢、吳淑珍、王舒婷、黃蓉延、劉詩婷、趙苑吉、王冠宇、柯彥廷、許佳恒、蔡胤傑、金怡妹、鄭凱慈、陳火辛、謝文帆、陳明恬、張筠奇、湛郁翔、吳家昇、吳松枝、許亞茂、吳淑淑、黃偉玟、李怡仲、白光淑、張美雲、程宗翰、林俊逸、貝雅惠、戴雅芳、楊亭維、劉孟君、林彥良、曾秀美、鄧淑芬、明金姍、吳文惠、廖光慈、吳傑芬、賴善韋、林秋智、簡威宇、林雅雯、楊佳雯、蔡政軒、姜怡伶、黃重發、陳美玲、陳佑冰、施苑歡、丁貞文、林耿穎、陳淑韋、邱晏文、張慈康、陳智筠、王麗宸、陳玉輝、胡奕翔、洪淑元、鄧佳怡、曾崇生、王孟君、黃軍屏、吳筠恆、黃盈甄、周明儒、吳品嬌、謝協琬、陳家俐、陳逸治、夏忠典、黃韋廷、吳志偉、謝義剛、張鎮綺、鍾明竹、陳昱瑄、陳信勇、彭俊宏、黃振瑋、黃皓淳、鄭志軒、許幼香、廖仕康、黃淑惠、鄭倫心、蕭偉智、蘇國華、駱憶嬌、許惠強、丁雅惠、芮冠雄、李宜謙、邱松富、李之寶、林育昀、張志遠、郭姿穎、游金娟、林芳蘋、羅怡伶、王冠宇、許承穎、陳志明、蔡昇真、蔡宜萱、謝婉婷、施惠君、鄭瑋倫、李佳雯、王珮雯、黃倩義、孫佩珊、林韻如、趙茂榮、劉秀德、姜雅琪、彭燕儒、黃秀白、賴彥元、蔡卉均、許宗和、鄧科定、阮蕙妃、王芃學、許雅雯、黃茜妃、袁怡婷、王淑忠、楊智貴、陳祐誠、劉美堅、林素芬、張泓凡、王吟學、傅國珮、劉啟緯、林嘉人、陳怡靜、黃建揚、黃俊嘉、楊舒甫、葉儀新、劉廷宜、魏冠志、葉郁婷、陳牧堯、張素芃、謝松航、李真裕、王俞人、麻景翔、蔡俐皓、鄭為原、黃美達、王景翔、戴蕙博、許柏豪、李芃誠、蔡嘉泰、盧雅雯、蘇家欣、王愛月、陳淑琴、吳建成、方幸謙、楊舒來、鄭彥良、黃柏廷、黃心均、李雅孝、蔡為亮、陳志珠、黎展喜、張靜怡、陳依萍、鄭懿珍、黃郁祥、吳雅鈴、謝軒育、黃瓊盛、王怡靜、林展穎、林義怡、沈政達、陳涵良、陳政雯、張秀慧、吳俊玫、賴慶海、溫哲瑋、曹雅婷、劉家慧、張玉泉、蘇姿穎、蔡佩岑、黃志明、郭士瑋、林文麟、劉佳芳、王怡亨、李雅芬、蕭松貴、陳人順、賴佳燕、林婷婷、李憶雲、錢伶瑤、陳祐昆、連炳海、林怡禎、王玉宜、林博鈞、陳秀淑、冉堅裕、杜宗諭、賴智鈞、楊曉孝、傅禹海、鄭毓凡、袁子軒、林佩斌、吳孟勳、賴秀美、林映姍、林家民、吳秉昌、蔡嘉智、汪家瑜、楊佩君、吳維倫、彭剛城、鄭廷桓、陳宗淑、鐘珍生、林文妹、黃添蘭、楊則妮、許志芳、張旺綠、陳彥菁、陳中諭、陳佑劭、張侑怡、張玉婷、馮敏琬、宣傑念、程湘婷、黃蕙梅、李永任、柯哲瑋、葉明翰、吳雅宸、錢怡靜、李常杰、張育維、鄧雅婷、景俊吉、傅素容、李怡真、姚憲瑄、毛家維、吳一婷、蘇宛臻、高振豪、蔡佩玲、謝竣謙、黎凱倫、蔡丞鑫、徐哲維、林書宜、陳品治、吳怡文、傅協安、吳志威、黃宛儒、林俊諺、明智妏、謝珍龍、張志杰、吳宗秀、羅宜靜、朱韋成、潘智仁、李豪全、涂有綸、吳晏麟、謝綺火、林郁城、葉建宏、黃國彥、劉政宏、陳詩婷、林介樂、吳文雯、陳志林、張杰伸、謝昕正、連美君、賴雅雯、錢珮君、郭佳慧、宋佩珊、楊儀伸、劉冠虹、林耿音、吳海君、周馨文、張文紹、張美君、張淑婷、鄭瑋婷、林婉菁、張凱倫、陳靜如、陳剛財、蕭家弘、蕭雅嵐、曾素貞、黃婉瑜、王淑偉、韓彥霖、蘇皓佳、蕭思穎、師昌慧、楊世傑、施玉玲、趙人豪、楊胤菁、程莉婷、張君念、陳靜生、高俊豪、蔡宜雅、陳文念、王君鴻、陳梅蕙、唐佩珊、吳俊逸、陳俊念、張雅婷、傅小春、張憶嘉、陳淑貞、王惠雯、陳惠雯、劉耀中、陳友合、林維倫、王冰恆、馮志文、劉家宇、鄭延蓉、許柏勳、陳永淑、施聖文、李宛儒、駱家瑋、黃淑貞、柯承恩、林雅雯、陳永軍、陳柏美、陳成榮、楊勳雄、陳美城、王苡義、施雅筑、黃千瑜、蕭偉哲、楊瑩賢、傅亮、許志卿、李建昌、郭竣霞、李姿恩、吳永希、林舜書、林祥航、郭佳慧、王睿正、賴慧萍、蔡儀綺、黃佩君、李珮如、張蕙政、郎淑貞、方俊琪、陳俊凱、夏韋廷、林良憲、耿彥志、李雅鈴、黃苑卉、李吉人、吳重瑤、劉美玲、劉彥泉、林明倫、楊旭春、葉鳳珠、楊雅昇、王奎愛、鄧美玲、丁美玲、葉逸妍、吳宜佩、王克劭、吳慶士、吳盈秀、吳佳慧、蔡明琦、賴美君、吳宗蘋、傅尹奇、黃松羽、陳于庭、劉南季、李火凱、張心怡、李容季、王建豪、張旺卿、蕭珮婷、林雅雯、曾婷婷、劉佳穎、周淑惠、洪芸發、劉怡萱、駱山杰、王榮秀、呂佳迪、劉欣怡、梁明儒、溥怡君、廖珮姍、吳建宇、傅緯念、邊雅琳、姜成光、黃啟俐、宗彥茹、胡美雲、許青季、張孟哲、黃偉茵、葉怡君、連邦依、陳珮斌、簡志佳、王淳宸、駱玟紋、李冠忠、羅哲榮、別怡君、黎亦瑩、林柏吉、彭心怡、吳亦成、李欣怡、黃于庭、陳惠美、陳惠甫、陳秋香、林正毓、許佩君、陳柏成、吳思宣、周佳蓉、魏柏豪、李思迪、侯宗達、張白儀、葉韋志、黃玉婷、黃凡翰、楊妤名、李岳睿、李偉哲、郭希松、林哲利、楊亮洋、蔡淑威、謝喜軍、陳凡倫、孫盈如、黎竹吉、楊民泉、貢愛輝、劉育霖、黃信瑜、周水鑫、吳淑芬、陳淑舜、李宗憲、龐芳儀、黃依婷、楊雅芬、吳文宏、吳佳易、謝雅芳、張妙萱、張怡君、李怡靜、王璇佩、李明恬、王明儒、陳靜怡、曹婉玲、黃靜怡、蘇振春、郭忠宇、劉修娟、洪偉來、黃奎雅、潘英玟、申與利、張玉勳、詹佩珊、李瑩原、張永勇、林怡雯、浦思翰、鄭承侑、袁意霞、林宗仲、傅明念、袁琬惟、陳冠恬、林國惟、沈琇雅、蔡馨儀、陳君鈺、劉雯婷、賴宏樂、陳淑芬、陳曉松、陳京瑄、蔡孟康、陳麗娟、黃慧惠、劉珊星、周育儀、林惠敏、錢浩如、邱婕妤、黃冠帆、劉少凡、許琬佩、陳萬宇、吳名仁、王敬瑩、崔宜凡、李慶慧、黃淑如、葉佳儀、連國林、李孟怡、黃真萱、陳欣天、李希佐、陳雅雯、胡珍妤、茹怡廷、林凱琳、黃靜春、連宛清、黃威廷、溫惟雯、王慧娟、張子豪、鐘方蓉、張宜珊、談惠敏、郭靜芳、謝鳳沛、李淑禾、林揚玫、何耀哲、李怡蓁、戚雅慧、林孟奇、姜珮如、楊宗木、藍千惠、邱淑芬、楊思妤、陳克希、洪偉隆、韓典依、葉思妤、林嘉宏、潘俊安、張馨喬、李雅妃、韓家弘、蕭佳蓉、駱順輝、張雅娟、施俊霖、黃幼堅、戴凱文、黃展玉、夏寧美、黃信桓、林佳玲、李新綠、楊中岳、姚京木、許惠宣、蔡宗星、張翠季、趙海愛、黃珮源、程哲豪、張友瑄、陳美珍、方家銘、鄭羿雨、彭軒辰、繆筱婷、陳韋廷、施緯軍、蔡聿嬌、蔡孟璇、鄭凱旺、馬惠茹、張偉倫、謝靜怡、祝國維、楊意蓁、陳怡伶、黃郁伯、謝明舜、洪惟貴、楊昱宏、吳志明、龔永雅、葉佳慧、賴美惠、阮卉青、楊火芬、曹力天、劉辰文、許淑玲、林郁伶、巴靜雯、劉姿蓉、梁宥方、吳豐雲、黃鎮宇、陳志宇、張則天、吳佩韋、賴儀彥、吳淑貞、陳威倫、楊淑娟、林韋志、張雅茂、王承南、吳昱宏、李麗芬、冷芝迪、彭舒涵、陳志銘、王文義、王舒然、錢淑貞、許雅玲、黃建法、江雅慧、施雨麟、梅彥霖、李姿士、倪昇嘉、吳淑萍、黃聖峰、陳雅靖、郭佳琳、蔣雅萍、許芳儀、林雅芬、王可妏、李家豪、羅思賢、陳金然、蔣于真、王潔昇、彭雅娟、許欣怡、胡威廷、李佳珠、陳祐城、白嘉貴、張可恭、龍晏昀、鄧雅芬、黃秀娟、劉信宏、汪玫雯、戴建彰、張法奇、胡政廷、王丞亦、林瑩睿、曾淑利、潘佩樺、張又珍、陳志明、楊俊佑、潘隆輝、杭淑芬、楊武卿、曾曉萍、方育柏、甘文華、黃雅惠、周凱元、衛美雲、藍元祥、黃文宜、李嘉威、楊清甫、楊志宏、何惠珍、陳祥淳、周卉岑、楊翔仲、何舜文、陳麗珠、吳宛蓉、駱雪音、李宗孝、林子菁、溫淑貞、陳先翰、馬佩蓉、陳宏斌、張蓉容、謝韋秀、陳亭甫、徐韻婷、錢育玄、翁志嘉、何凱翔、冉海妃、嵇湘皓、趙財妏、符俊逸、簡怡璇、陳惠雯、張筠廷、楊恒緯、錢常民、蔡雅萍、張健汝、吳秉雲、馮羿云、劉江興、黃文誠、楊素桂、黃佩珊、許麗雯、吳怡君、王思東、陳正宇、謝佳陽、黃建正、洪群禮、吳恒堅、施麗美、龍淑君、汪士豪、周婷仲、陳韻婷、林志云、黃俊佑、盧志杰、吳家瑋、黃賢仁、林宜萍、張怡季、趙雲玲、吳彥智、連建德、葉昀隆、孫依婷、林木士、許淑娟、蔡剛妍、徐玉娟、王家豪、張延喜、涂喬信、張佩昆、陳雅鈴、施育德、吳鳳柏、方靖洋、陳靜桓、林俊妏、唐鴻城、陳宗憲、黃詩婷、葉祐忠、程奇廷、黃雅虹、黃建瑞、呂孟穎、張海蘋、王志明、黃翊玲、陳秉昕、曾淑華、吳英怡、吳月旺、賴慧萍、陳鴻瑤、賴淑貞、吳宜吉、蔡雅玲、陳卓方、陳正信、葉俊賢、李建惟、楊書聖、蔡政琳、張伶佩、黃薇政、蔡莉俐、袁宣宇、謝文彥、郭明慧、李舜光、楊秋偉、吳文馨、陳逸羽、王冠霖、林承斌、黃聿辛、沈名伶、林慧喬、李怡合、蔣興愛、李姿君、陳彥琳、王家瑋、彭嘉祥、陳美宣、蔡亭君、林舒廷、王祐誠、趙淑華、涂典帆、袁竣芷、吳慧敏、宋佩能、高木名、謝元汝、柳益諭、陸宜芳、魏惠敏、曾竹蘋、施晴政、趙健瑄、黃佩穎、鄭昶學、柯家偉、劉淑貞、曹雅琳、陳柏茹、吳珮如、王家齊、鄧素甫、楊又如、湯恆堯、劉家威、白佳靜、吳富瑤、郭又毓、袁惠婷、徐辛茜、林儒雯、黃淑惠、蔡雅婷、宋惠文、吳庭昆、謝欣謙、周詩涵、孫佩君、蘇映娥、陳韋伶、徐智傑、王思華、陳韋豪、賴家榮、王初音、林家瑜、林豐斌、方明翰、李雅玉、林南瑩、陳辰睿、王瑋婷、劉修蕙、趙宛侑、陳嘉惠、韓慧君、王士銘、張志綸、葉柏勳、李元財、駱鴻火、秦其林、劉志豪、蕭耀文、賴祐誠、許耀德、鄭子名、盧光倩、柳上霖、李雅萍、陳萬易、陳奕君、柯千慧、林雅琪、林明湖、李幼志、陳雨琬、李彥儒、程怡珊、王玲輝、林柏利、蔡建銘、許喬邦、沈夙霞、許佩蓉、毛玲香、姜方瑋、湛苑平、王鈺云、鐘淑婷、柳士瑋、溫以育、黃淑香、王昌辰、謝欣揚、李怡剛、張韋廷、昝雅萍、陳文琪、張姿蘋、蔡佳苓、陳姵恭、陳怡靜、黃瑤賢、宋木哲、何智偉、黃芸彬、王承岑、張怡君、陳文傑、黃雨星、陳嘉凡、李孟妤、闕宜劭、鄧佩珊、許嘉辰、王詩華、林琦旭、江婉君、高劭州、江佳姍、吳承香、劉邦軒、謝珮杰、李怡皓、蕭雅昌、吳尚銘、黃雅茹、張至新、陳威臻、楊靜怡、陳欣孝、劉彥廷、林牧孝、涂月全、彭長一、劉文彬、蔡佳蓉、余翔妏、王怡如、徐嘉蘋、劉麗芃、夏鈺婷、錢鳳珠、邱南法、張儒羽、胡健銘、王偉傑、馮哲維、林純佳、林睿霞、黃林恒、林政霖、王盈如、鐘辛喬、潘凱郁、曾逸凡、陳宗達、盧健銘、李旭和、楊鎮琦、陳佩柏、馮吉芳、趙少民、陳星佳、張劭志、吳容霞、黃耀德、陳淑芬、王韻依、吳美愛、陳珮雯、王羿昇、曹淑娟、陳堯梅、吳得心、陳左茵、張智玉、賴明儒、潘立維、陳湘幸、周淑珍、呂仕君、李可琳、謝典辛、王劭光、陳永誠、沈怡婷、黃祐宣、廖嘉祥、李建惠、王鳳城、吳明軒、王之枝、曾淑裕、楊怡君、陳皇璇、周其禎、黃馨儀、林宛真、廖珮人、張協方、涂玉鳳、劉品宏、張淑萍、姜晉福、張博天、陳盈紫、連湖潔、陳孟倫、傅浩一、劉昌泰、郭晴孝、戴夢郁、陳建財、姜義平、霍禮睿、陳雅婷、陳至淑、王奕翔、傅娟、陳佩能、劉芳威、林彥瑜、黃友辛、郭怡君、陳于妏、吳韻如、冷威柏、林宛臻、郝沛福、魏禹伯、涂妍嬌、劉偉成、簡初潔、張仕茂、高宗白、蔡宜菁、郭淑映、郭怡伶、林馥儀、吳玉梅、黃俊男、戴巧意、陳幸達、洪辰心、許柏弘、陳雅惠、張瑞剛、張嘉俐、毛奇姍、周紀鴻、杜瑋婷、劉哲宇、趙怡如、廖安火、彭家榮、吳慧玲、蔡孟安、劉雅惠、張舜文、謝哲全、王儀紫、史萬年、陳紋仁、蔡政萱、吳勇亦、游曼倩、陳仕書、謝竣歡、陳健鈺、荀又添、朱明宏、邱玉行、張玉茜、林佳貞、楊孟穎、林佳劭、謝欣劭、李俊純、杜政儒、陳明琴、黃中學、莊惠法、謝鈺婷、鄭俞富、劉哲榮、孫侑忠、張佳州、胡志穎、杜剛吉、潘涵偉、連喜行、張文凱、楊惠珍、陳秋燕、蕭威雯、黃嘉以、吳彥宏、樊偉哲、羅琬韋、王柏原、楊雅慧、鄔薇堯、侯芸琳、袁苑全、張哲定、袁建萍、陳雅松、張志雪、姜恩昌、賴姿妤、陳佩白、王俊豪、鄭旭欣、廖珮屏、陳玟珊、高雅惠、陳富琬、楊子軒、林政廷、王筱夫、黃雅婷、蔡子傑、羅欣穎、毛彥慧、鄧吉盛、黃重燕、林彥綸、劉青淳、林秋萍、鄭穎賢、林潔雄、沈淑珍、陳桂妃、林山盛、蔡雅恩、陳慧敏、洪明慧、柳翔義、彭淑婷、楊富芸、陳羿亮、郭群花、涂文君、張詩婷、蕭宏達、林于婷、黃慧人、林冠恩、陳佐惟、林右喜、陳智凱、林瓊慧、李天佩、白秀英、謝郁婷、陳儀柔、林雅男、吳宜陽、顏俊欣、吳秋麟、李曉萍、李明安、吳金群、龐弘發、賴雅馨、陳雅吟、彭婷仰、王怡屏、趙景劭、黃巧易、簡美慧、周宇妏、林偉倫、唐筱婷、黃慧茹、黃彥翔、蔡明軒、茹偉傑、劉佳琪、藍家瑜、張易舜、周景芬、趙健峰、蔡彥宏、賴惠玲、王俐洋、宋旭韻、楊宗憲、曾韻如、許哲貞、曾貞賢、駱雅婷、吳文真、謝淑貞、蔡秉然、賴俊琳、陳淑君、黃怡慧、孫彥儒、張靖雯、張宜真、胡岳強、許佩芸、曲柏成、吳苑春、彭雅筑、簡淑君、韓冠儒、陳淑娟、高雅玲、林寧琪、胡舒娥、湛俊廷、楊佳蓉、張正偉、王彥博、陳亭修、連哲瑋、張紀琳、顏政宏、姚舒月、黃智柏、郭雪陽、彭柔緯、白佳蓉、張貞玟、吳怡萍、林惠珊、唐韻婷、林曉竹、李宜真、周書瑋、林郁恬、林台彬、何惠珠、阮旺芃、黃瓊州、楊力妍、許清哲、吳士欣、黃嘉聖、林佐瑋、洪牧智、陳伯愛、曹順福、游雅文、楊皇禮、趙哲瑋、張馨文、賴翰、陳怡希、吳秋星、路學綠、崔雅琪、李欣怡、許冠良、黃怡靜、潘夢漢、劉健銘、黃冠志、涂禮吉、賴瓊映、林映榮、高佳慧、蔡宜仲、王朝辛、黃建松、詹淑敏、蕭俊霖、黃琬婷、葉純仁、胥虹菱、孫柏恭、周竣柔、陳易蘭、袁曉玲、胡玉霞、鄭易季、劉怡禎、謝義嬌、趙家祥、李靜蓮、王怡珠、張揚吟、陳重夢、徐軒豪、黎冠儒、唐陽芬、汪璇成、陳文富、余怡萍、皮信宏、楊仲和、劉佳玲、鄭凱亨、蔡心怡、周明翰、黃彥苓、蔣偉哲、陳雅其、王如天、祁漢嬌、鄭松雪、李柏毅、李秀成、陳翰行、藍志宏、黃嘉偉、成佳穎、顏敏芸、周慧君、呂桂茜、張淑瑞、金哲榮、楊仲裕、陳祖善、吳瑤丞、夏憲睿、蘇妙然、陳婕妤、溫惠如、楊詩駿、張惠喜、柳恆愛、陳國清、李佩英、沈彥宇、賴佳蓉、李柏雲、蔡筱婷、金瑋芷、楊博勳、林冠珊、姜禎璇、劉玉劭、夏先群、李彥喬、游婷婷、吳俊彥、李雅婷、鄭鈺良、林文華、高昱廷、林家宏、孫佳慧、吳珊書、賴慧君、王德湖、林哲豪、陳玉梅、邱綠平、林敬凌、賴怡臻、郭力良、劉俊翔、王俊彥、黃常妮、朱家豪、李承翰、黃于婷、連如君、周孟儒、蘇雅芳、黃文琳、沈哲瑋、連士哲、蔡政宏、黃國瑋、姜綠凱、李凱聖、林建豪、許貞儀、王嘉亨、顏右琳、林憶學、陳盈劭、陳彥樺、吳佐倩、吳仲貞、陳慧妹、王柏靖、林惠珍、童亮穎、袁詩婷、卞定利、王子芝、蘇哲榮、張惠玉、金先潔、賴琪茹、王智易、沈世傑、簡彥廷、高芝沛、黃俊能、吳哲維、陳湘婷、林先杰、朱承茂、陳奕亨、高學合、周俐恆、謝元和、鄭若忠、陳宣銘、連羽梅、陳孝儒、劉宏芃、謝佳偉、阮佳琪、宋恆妍、詹雅岳、吳采幸、王佳霖、潘志嘉、陳瑋博、何少志、汪宛薇、郭怡如、周予昌、張慶旭、鄂禮禎、王明倫、王美惠、李心芃、黃智桓、鄭善羽、李美珍、方宜君、吳瑜霞、林慧琳、張婉婷、林育廷、王柏光、陳金鳳、楊冠宏、潘法韻、黃成吟、陳左來、施茜雪、陳映桓、劉佩雨、曹蓉誠、林柏芝、吳雅婷、黃詩旭、李友廷、黃彥文、張正枝、陳志倩、白玉娟、王怡香、方秀琴、林瑤汝、王金柔、鄭慧茹、林思亦、陳宜靜、連興以、陳美春、王家宏、劉彥蓁、楊允善、陳敏岳、張必薇、鄭伊行、紀軍合、徐雨桓、劉佳發、張佳雯、汪銘德、顏舜威、張淑真、歐陽淑貞、李寧華、楊語以、林欣星、盧宥鈺、王淑芬、王怡婷、潘怡文、黃旻嘉、李國英、彭育菁、蔡茂福、邱亮桓、黃淑亦、賴建花、陳克弘、鄭淑真、茹佳穎、蕭怡雯、柳尚菁、邱孟涵、涂俊賢、楊映杰、吳佩英、吳惠玲、高漢妃、黃柏城、黃怡康、劉惠茹、劉冠迪、黃詩法、張家原、王延友、吳宜靜、蔡佩萱、利真毓、林和元、陳馨卿、鄭瑋倫、唐京蓮、胡雅雯、李美智、李俊毅、葉慧玲、黃涵妍、陳佳玲、蔡文豪、武家慶、陳國一、王威任、陳思穎、林明輝、陳香君、周俊逸、阮詩涵、陳逸名、林育瑞、李怡君、謝宛孝、王嘉雯、謝雯綠、林詩恆、陳雅臻、吳文宏、林彥姍、羅紫州、謝仁傑、高茜書、郭予宣、楊朝信、黃雅玲、王志生、胡慈清、葉佳瑩、侯世傑、王詩皓、賴宗毅、賴卓學、黃虹天、鄭宜福、王冠琴、黃麗珠、周文彬、張苑恭、曹瑋旺、李志豪、蔡蓉玉、陳麗靖、梁筱涵、林旺啟、吳必清、陳俊安、屈偉婷、劉倩潔、顏婷辛、賴麗芬、張昌容、潘育維、張原茜、吳俊宇、楊雅雯、曾佩珊、侯誠禎、張瓊祥、陳希儒、簡侑瑩、洪慧山、蕭喜妍、陳正恆、王郁苓、周典愛、陳韻婷、鄭長坤、陳欣穎、林君志、陳智堯、劉凱琳、簡俊毅、施昆尹、陳琪花、顏佳宏、陳瓊枝、張雅嵐、王志堅、黃俊穎、羅智文、林柏珍、林婉如、王宜君、李玟嘉、吳一白、陳玉瑄、劉宗翰、梁怡廷、陳榮甫、蘇枝琦、戴琬珍、楊靜宜、張俊瑋、林佑然、楊雅慧、謝孟念、游宣定、郭凱元、賴志強、蔡信宏、郭宗淳、楊美珠、黃威啟、陳晉維、張姿婷、陳正和、林宗仲、吳宜幸、郭玲仁、王育菁、賴曉薇、許吟禮、王俊嘉、謝雅玲、王季紫、謝南秀、楊巧友、吳嘉祥、徐嘉琬、方庭瑋、呂吟儀、陳欣怡、李佳善、鄭佩君、蔡宗岳、賴博文、林盈欣、張坤娟、杜嘉豪、謝明新、許長漢、林立華、袁玟亨、陳瑋蓮、王品翰、胡怡婷、蔡伊芸、曹美惠、鄧惠茹、黃星杰、吳惠映、吳宜真、胡肇夫、倪海奇、王嘉信、張政湖、洪國勇、李麗美、邊宜芳、黃淑芬、林慧人、賈士銘、郭嘉鴻、吳偉蘭、馮仁豪、洪雅婷、林嘉任、謝劭冰、陳彥廷、蔡新君、林士凱、程智仁、趙石慧、林柏雯、連嘉玲、施喬聿、魏珮如、王昱亦、王蕙雄、李家瑋、張志祥、邊佩琪、林怡臻、劉成茜、陳俞欣、王介一、蕭欣樺、馮峻鴻、黃俊正、袁財明、林惠如、白宜音、郭舒卿、李慧玲、劉宗達、楊雅玲、林慧人、徐元州、阮肇紋、郭雅泰、陳淑威、許文泰、楊茂玲、周靜宜、鄧依婷、陳名容、李春珠、郭晏禎、韓湘萱、張明輝、高科祥、傅英傑、黃政昆、皇甫仁睿、鄧昕彥、嵇與豐、陳佩啟、王嘉琬、鄭涵元、陳世靖、張琬婷、李冠青、劉偉銘、鄭宇博、林郁妮、李依婷、章佳玲、吳宗其、李宜倩、陳雅琳、韓瑋婷、蔣政俊、吳淑娟、林國麟、張淑慧、林哲以、嚴瑋婷、施佳蓉、鄂俊賢、陳惠婷、陳柏淑、黃寧幸、陳韻念、王俊凱、林麗雯、黃國季、沈雅婷、游哲城、梁佩榮、方淑珍、王子麟、李泰丞、陳怡靜、蔡賢菁、許聖珮、黃俊雯、黃國伸、葉于倫、張書妃、李山貴、王上卉、趙耀中、林倩恆、陳信妃、吳佩冰、許淑萍、馬侑揚、黃漢紫、林怡潔、蔡柏豪、許惠雲、夏美惠、蔡明靖、陳建智、許苡蓁、黃家枝、張政宏、姚庭瑋、周于婷、林逸旭、林士虹、王依婷、張嘉桓、劉政儒、謝弘育、林牧定、張宥修、楊吉伯、李夢聖、鄭鈺易、支駿哲、吳宗坤、王憲吉、黃欣宜、林思穎、王宗穎、周宗穎、韓婷婷、許智月、王仲火、徐馨文、潘俊賢、劉淑珍、吳佳姍、李呈陽、郭淑云、曾登枝、陳妍松、馮美玲、楊千潔、王建霖、陳安彬、張健豪、梅曉亨、黃欣均、劉國偉、李建良、林佩蓉、郭馨文、涂威廷、柯筱婷、王山瑞、白筱涵、葉慧玲、陳盈慈、韓淑漢、黃筱涵、王友云、葉玫鑫、張秀瑄、林士綸、黃曉萍、劉彥儒、楊惠桂、李俐賢、陳瑋婷、李惠文、吳俊禾、胡冠宏、周志翔、邱振豪、徐嘉宸、王怡君、林冠容、楊宜芳、陳采均、焦家豪、林銘鴻、吳淑華、郭郁婷、常孟君、黃承鳳、呂淑玲、張群寶、黃美映、黃婷定、楊宛美、周忠春、丁于珊、路士銘、吳明儒、劉碧純、王明揚、白可聖、林雅治、陳偉銘、周柔柏、劉竹瑤、王偉翔、黃玉谷、韓靜怡、賴彥翔、陳宗翰、姚慶竹、黃世劭、袁莉英、楊奎蓁、溥崇婷、蔡怡芳、趙卉珊、潘百任、張卓豪、吳建宏、陳冠聖、張智惟、林韋佩、吳怡均、許淑恬、邱郁蕙、吳郁珠、王朝潔、黃向倫、陳其一、黃佩君、吳孟修、林宏達、劉建安、張舜孝、林敬帆、楊火皓、林子傑、蔡一瑜、郭寶駿、廖偉發、王鈺婷、唐雅文、林佑梅、張俊宏、張承翰、劉怡君、潘政茂、朱育德、張英傑、賴蕙亨、黃冠宇、魏季元、吳于婷、林丹月、金惠文、陳昆哲、郭士哲、林芸群、陳婉清、黃玉侑、蘇郁麟、曾詩舜、韓俊翔、蔡辛宣、張宛芃、王家瑩、邱美玲、林育芝、陳憶純、李佳幸、周尹來、吳靜宜、溫明宏、王雅琪、廖祥舜、韓書瑋、鄭俊霖、蔣依潔、邢淑娟、林季妤、簡麗萍、劉琬婷、張伊憲、劉鳳念、王聖美、蔡宜隆、林峻柔、蔡怡文、周予蓉、林牧利、李佳宜、劉思賢、黃雅玲、關尚和、陳鈞良、陳秀人、連雅欣、楊曼俊、符華方、張佳穎、張景博、權妍為、楊宛蓉、趙興禎、陳郁涵、黎雅文、許惠華、黃于佩、陳翰怡、李明南、全吟婷、張筱婷、韓文凱、王詠姍、黃雅婷、李詩婷、王其伯、吳美義、蔡紫苓、涂敏芳、黃鈺秀、黃淑偉、吳宜潔、王志穎、張鈞誠、張峻揚、陸珮臻、黃水康、鄧啟芬、郭尚寧、鄭家慧、黃聖誠、沈怡芳、林雅惠、陳鈺能、陳艾泉、曲建豪、陳上定、韓士傑、林于婷、方韋廷、林欣亦、楊惠泉、葉維雨、蔡志軒、陳炳婷、黃瓊來、林素傑、王俊豪、林孟宜、黃佳純、陳慶海、吳裕翔、李佳穎、李雅婷、黃鈺映、劉博竹、侯家豪、黃淑娟、楊雅琪、林冠玫、蔡韋潔、黃佳慧、黃于恩、賴世偉、江雯惟、林彥念、楊禎法、李彥泉、呂欣宜、藍一鑫、陳仁杰、陳俊賢、李玉梅、張信玫、蔡雅桓、王石正、陳淑治、林建珮、袁賢來、王欣怡、林淑芬、劉玉貞、涂嘉亦、吳俊宇、林雅如、林靜陽、洪國慈、涂蘭竹、許晏盈、陳必輝、黃登玄、陳玉福、金豪堅、黃佩芬、許美華、簡國偉、涂雅婷、李瑜添、林紹治、李妍慧、洪國茹、陳雅惠、陳弘凌、黃詩慈、楊耀文、賴欣宜、洪惠鴻、蔡筱芸、陳志良、王志文、蔡晏芳、顏美玲、陳曉玫、王韻雪、鄧詩涵、吳政真、闕敬謙、林左哲、林鈺婷、陳陽歡、徐怡如、黃富月、張志賢、陳俊穎、杜孟軒、蔡雅然、廖婉慈、蔡海潔、吳彥智、王詩婷、賴慧洋、黃淑妤、陳俊幸、韓韋芬、鄭淑君、潘登恩、張偉銘、李菁誠、王清銘、林佩智、林俊彥、陳志琴、戴育霖、徐建豪、謝碧岑、方定燦、郭怡萱、尚振宇、王志宇、黃麗真、陳世珮、蕭郁婷、鄧俊銘、宋雅慧、王韻啟、游淑君、謝湘婷、鄧嘉佐、孫怡君、黃采青、曾偉哲、王怡欣、林振宇、韓詩涵、吳志明、郭珊喜、林雅辰、林志平、方得柏、邱慧茹、謝佑伯、衛雅雯、陳靜怡、簡蘭原、劉國易、林家強、胡佳霖、張明杰、林慧安、李淑琴、毛淑娟、童健豪、傅義信、茹宜潔、許雅士、陳歡男、李淑芬、陳柏杰、劉賢元、陳白東、翁紀禾、蔣家慶、鍾俊毅、徐竹卿、李佳蓉、謝宛宣、黃淑芬、林郁婷、蔣紀娟、潘政勳、郭俊傑、陳政軍、鄭雅婷、鄭惠萱、韓怡婷、吳仁杰、杜乃文、張上彥、林雅文、張怡凱、郭岳樺、葉雅玲、邱重堯、吳玉梅、杜貞儀、韓宛恆、曾俊霖、林吟珮、劉雅杰、盛雅婷、王嘉富、林彥霖、毛佳穎、張廷心、林政郁、楊秉侑、劉惠玲、潘榮柔、吳菁雅、鐘皓鴻、陸俊傑、林紫文、劉姿婷、毛宛儒、王于婷、林火泉、楊亞希、賴欣宜、管乃蘭、洪予利、張依婷、阮明翰、藍美娟、臧為福、李朝慈、廖怡君、張欣穎、韓毅琴、路家欣、姜均宜、蔡麗卿、臧永蘭、張玉發、林雅嵐、葉仁杰、袁瑋倫、吳欣均、許雅雯、駱幼鴻、金建彰、錢碧羽、韓鈺筠、楊紀維、酈昀菱、宋雅婷、高淑珍、許茹辰、趙嘉任、曾士弘、蔡建良、陳宏蘋、洪勳啟、王旭玫、林丞淑、廖宜君、李雅萍、賴玄珠、林鈺人、張哲龍、張文紫、黃奕翔、陳冠誠、楊林其、林于男、袁興勳、馬靜怡、陳淑君、楊宗憲、蔡朝一、林孟怡、張昇迪、林秋汝、王恭云、方堅星、吳雅婷、張明奇、賴俊杰、黃瓊士、許宜蓁、郭美娟、郭芳瑜、王先秀、林家豪、黃怡桓、吳毓強、胡美慧、張維鴻、李惠君、陳守綸、王祖政、張金鳳、周郁雯、林瑞雨、許信宏、吳佳順、劉廷音、馮明芸、張志俐、歐陽妍霞、蔡政菱、沈致妮、王秀書、高慧玲、郭皇學、邱志豪、朱珊彬、楊典倫、蔡秀娟、張凱婷、呂伶函、林義盈、陳麗雯、王宗翰、蘇杰緯、鄭宇青、蔡燕治、諸柏宇、黃淑蓁、林江璇、吳靜心、黃怡孝、危意妍、林俊育、姜新宣、張育谷、李容江、吳淑美、蕭雅月、王先意、闞英薇、黃必真、吳蕙峰、林亮清、張佳忠、謝國偉、林蕙美、林宣虹、陳威誠、戴豪勳、黃智成、姚雅君、楊小珮、秦紀勇、林明利、陳蕙揚、賴子芸、袁家瑋、鄭尚玲、張岳卉、許智卿、景京亞、楊亭萍、王淑惠、陳萬清、林耀文、魏孟儒、趙志偉、陳俊吉、許哲瑋、林建玫、查嘉月、方奕倩、曹怡君、陳婉婷、吳憲玄、張行玫、茅群胤、林宣妍、陸威宇、張景順、張恭郁、林秀雄、林建文、郭家雅、張凱輝、王志豪、黃水哲、傅俊傑、鄭堯伸、鄭凱哲、洪芳玄、趙元嬌、黃于樺、許宜龍、陳柏陽、韓怡伶、盧映勇、楊方卉、劉怡山、陳宥火、林義辰、邱詩涵、廖珮季、高芳如、黃偉誠、經珍琬、施玉婷、黃武馨、邱百亞、楊惠合、張俊吉、施子娥、吳偉倫、宋少友、黃佳燕、黃琬婷、馬涵夢、程得岳、王碧慈、張育云、姜邦源、陳郁文、陳姵岳、周佩芬、賴心文、楊惠文、倪金鳳、盧珮雯、黃瓊郁、林佳雯、吳俊名、蔡宜修、郭順娥、阮涵定、李宜臻、周俊念、盧君琳、陳韻宣、丁俊佑、韓凱雪、顏凱諭、王志傑、郭義良、張哲豪、簡詩婷、詹家豪、柳婕妤、李郁婷、劉雅文、陸駿南、杜于真、葉南榮、周佳宣、張財勇、陳子揚、趙羿傑、施信宏、黃冠中、周佳舜、蘇健虹、錢語丹、侯紹盈、黃文姍、蔣淑貞、吳筱辛、林建瑋、陳政裕、張婉源、楊舒岳、祝鎮宇、林可韻、徐金裕、李琳恬、陳俊岑、陳伊婷、張華琴、廖宜婷、王采昆、蔡思涵、陳姵來、潘冠霖、王建琇、陳嘉文、陳希蘭、張長芷、張惠雯、鄭苑義、林秀玲、黃信州、賴介聖、林雅玲、潘仁杰、陳惠娟、黃婷憲、陳瑞玉、黃珮南、黃志卿、林文馨、張炳函、林育維、黃孟修、林順妍、張瑋珠、黃綠珠、李欣玉、黃冠中、吳柏義、吳承玫、劉行恒、馮佳儒、盧海珍、陳君慧、于碧丹、張志映、翁姵君、傅建中、黃俊善、錢晏丞、盧琇汝、王育香、林惟瑜、翁俊霖、蔡美昕、杜巧任、張婷全、李丞旺、白雅婷、黃盈君、藍佩怡、林惠如、洪國惠、林建宏、胡美玲、曹子揚、黃冠霖、張進博、孫蕙佩、方惠雯、張江珮、蔡孟靖、林珮甄、廖偉謙、林怡航、高亭萱、黃鈺斌、周佳綸、劉俊賢、封淑嘉、劉淑萍、劉博仁、楊姵君、林宜汝、宋維東、林立恩、張雅婷、蘇建成、余婉菁、鄭柏成、林家芝、鄧耀文、酆依夫、洪俐聿、林淑州、蔡秀芬、林依蓮、鄭香君、賴雅芳、趙建宏、王書豪、藍良虹、姜怡蓁、唐旭航、王剛修、段祖恩、解孟君、徐淑娟、李俊修、張昆治、王哲銘、陳志如、蔣文豪、詹宛琦、楊恆喜、詹家瑜、郭俊良、楊淑白、楊嘉宏、林姿泰、淩依婷、陳昱宜、郭雅芬、宋韋伸、張尚緯、黃耀德、李易柏、洪雅芬、程南茂、魏元甫、林麗倩、楊姿妤、和孟儒、陳珮善、陳奕正、蔡毓宇、林志翔、謝穎真、吳怡君、李雪珊、毛佑萱、王宛真、伍文豪、許孟修、黃柏桓、陳彥仲、趙惠珍、陳俊銘、陳欣怡、葉堯辰、林雅慧、吳淑春、張紫薇、陳慧靖、陳佑侑、李曉玲、賴琪珠、陳智孜、林佳穎、黃翰瑋、胡祐誠、解星瑋、王陽榮、李明輝、蔡宜玉、黃珮佳、林嘉新、劉翊福、柯林盛、吳佩青、許惠賢、吳進洋、陳惠珊、宣佩珊、賴建白、王怡芬、黃惠芷、謝至秀、張語名、陳景聿、紀宗霖、林峻航、曹立年、昝慧萍、張水韻、陳麗恭、宋怡芬、高建中、陳靜怡、宋一瑜、黃俊傑、曹琬惟、杜吉豐、李明華、劉俊翔、張建中、黎偉倫、桑法威、陳佳怡、陳依萍、張偉婷、汪方亦、陳瑋毓、林立彬、黃文宏、謝淑娟、許孟雄、李偉為、葉人法、楊勇帆、邱曉娟、謝恭玫、盧琪恒、陳信凡、郭琬泉、盧倩筠、翁彥智、黃俊年、許筱婷、楊庭瑋、陳怡如、洪秋玫、陳家瑋、黃晏忠、游清善、黃軒美、陳冠輝、陳美娟、蔡佳儀、周智丞、呂舜裕、陳育法、陳方偉、張美君、鄭欣怡、錢怡安、林佳蓉、周得亞、陳威安、林依綸、李韋廷、陳金紋、柳博仁、陳佩如、楊雅達、蔡政中、張素銘、陳建良、韓敏怡、陳建輝、潘怡安、黃子姍、陳力彥、吳靜東、蔡孟啟、張馨芳、楊欣潔、林少玉、吳淑玲、吳柏哲、趙麗萍、黃美玉、丁雅惠、鄭勝燦、溫育霖、曹馨文、李俊亦、王怡君、陳明儒、蔡亭怡、陳芃以、李秀雲、黃玉玲、張茜泉、張怡君、陳依城、賴紀菱、楊愛海、顏昀瑜、李惟念、施維亨、張雅佩、蔡文輝、陳欣雨、李美玉、許韋成、周佩君、王翊洋、王依婷、羅淑娟、黃郁原、謝吉卿、方雅雯、吳美惠、貝翠達、林鈺娥、劉善雪、侯國瑋、鄭曜宇、蔡其琬、朱朝慧、馬彥璋、林禹倫、黃柏康、蘇綺英、李鳳珠、彭育柏、張木中、劉彥榮、李鳳文、葉雅茹、申淑貞、權雅雯、陳賢駿、陳珈堯、惠先文、李志惟、廉詩婷、陳燦萍、吳法勳、吳婉婷、金珮雯、郭俊年、吳宜秀、袁淑娟、何彥翔、侯月啟、楊俊廷、鄭家偉、施詩學、吳柏郁、溫雅茹、劉偉銘、潘曼毓、陳美虹、陳育凡、蔡秉源、吳以駿、陳韻如、黃智元、林康玄、許偉智、徐鈞玲、黃佳儀、辛瓊慧、楊文孝、黃怡安、湛雅玲、陳思涵、張祥怡、王怡君、林紫韋、張于庭、吳淑珍、陳心怡、林玉智、劉雅琪、蔡育誠、黃啟毓、楊永易、劉雅卿、鬱孝蓉、李明佩、陳智文、連順芳、湯曉皓、鄭曉弘、黃彥妃、楊佳瑞、林玲、陳新南、簡舜怡、洪琳岳、蔡堅劭、黃景凱、張睿伸、王昱廷、黃智堯、曾涵陽、陳韋希、陳祐翰、林向仲、黃舒典、王治康、吳禮山、施威任、張語元、翁麗芬、廣麗玲、劉愛睿、陳秋仁、林祐誠、葉舜文、高威廷、彭怡如、鄭惠貴、王怡萍、陳容卿、黃心江、楊定文、蔡進博、林欣萱、趙惠玲、張佩君、蔡于珊、林紋燕、黃雅芳、許伯揚、黃文豪、邱惠玲、賴右水、沈俊宏、陳大光、許豐宇、陳俊嘉、黃愛卿、崔珈祥、鄭元玉、陳瑤純、許雅慧、潘雅惠、李明慧、張文洋、鄭柏廷、彭宜婷、洪以邦、楊俊宏、陳芃啟、張美華、李軍毓、高揚甫、駱家偉、鄭怡君、黃文彬、沈美慧、林欣福、王淑卿、張則仁、李仰平、余恩書、吳珊亨、王子揚、彭嘉宏、羅哲嘉、吉志成、吳景蓮、高士凱、趙郁文、馮庭映、王美貞、王淑君、徐揚祥、曾一琦、金康惠、余吟貴、王聿慈、謝元善、葉朝娟、王偉廷、陳芸為、羅淑君、黃家榮、吳明慧、趙怡君、郭季任、李琳豐、張侑丞、黃禮誠、梁耀德、牛嘉玲、程育瑋、謝依屏、徐士傑、崔宛君、劉嘉瑋、張裕翔、王展裕、吳彥廷、陳怡君、黃旺尹、林偉銘、韓廷虹、郭淑萍、許祐禎、元博芝、陳信舜、陳亭竹、洪國啟、呂鈺婷、李淑芳、李俊毅、元百函、謝志豪、黃定茜、殷家弘、陳人豪、葉琬竹、呂守其、曹俊銘、林彥博、陳意萍、陳卉盛、林欣云、汪虹陽、楊致遠、林政治、郭左紹、錢涵如、邵佩君、劉育心、林家亨、王哲宇、黃俊霖、吳雅任、陳致彬、翁俊毅、陳威杰、羅俊宏、崔婷婷、楊雅珮、劉博忠、劉明麟、張佳青、張耀虹、葉雅文、駱鳳聖、李函南、陳佳卿、邱明瑞、夏清真、田家偉、楊凱婷、劉怡珊、施修依、黃國原、林芝芷、邱建宏、潘志偉、張旺青、林彥宇、曾家瑜、陳鳳昆、楊歡怡、鄭亭君、曹鴻昌、王姿華、陳奕州、黃逸德、黃峻馨、陳孟臻、徐珮珊、侯美惠、張俊豪、周佳伶、李珮瑜、王尹一、李美雨、黎家瑜、張俊宏、張曉德、陳宥智、張裕妤、陳聖傑、邱俊瑋、齊茹銘、曹彥欣、李怡雯、秦琳祥、李伊婷、暴宛儒、蔡惟婷、沈怡伶、張威廷、林佩龍、周志裕、陳韋伶、孫珠、馬東仁、駱浩人、潘怡廷、王宛臻、張淳松、黃雅茹、黃志南、陳思以、陳尚學、林盈君、金麗珠、邱智超、林仰琳、王惠美、貝政霖、李婉君、林筠旭、陳姿鑫、吳建娥、許貴邦、鄭凱夫、袁泓卉、黎豐美、陳明宏、王莉婷、陳妙白、賴淑屏、李文婷、林彥廷、陳嘉妮、邱恩娟、郎士瑋、楊侑美、王明儒、吳芝伯、陳巧依、潘函蓮、鄭致成、陳淑惠、陳盛禾、陳慧娟、陳紹秋、黃靖喬、林旭廷、杜白妍、林盈君、鐘彥君、王彥伶、林善志、吳孟娥、劉冠依、陳炳云、庾家瑩、張旺青、陳立偉、張志輝、溫宗全、曹馨達、蔣純琇、沈智傑、吳宜奇、彭培倫、陳詩桂、黃姵君、林玉婷、林萱婷、倪建瑋、林秀英、黃怡福、溫思維、邱宗達、黃善恭、張明哲、黃雅惠、王博文、陳堅妹、董竣傑、葉佩珊、江水添、邱俊弘、許彥良、陳士賢、賴建宏、江昶奇、陳秋能、蔡瑞松、張逸凡、陳得潔、王怡君、林淑芳、錢鎮珍、劉淑彬、張添城、薛昱廷、陳珈其、張香薇、林必能、李宗一、吳幸屏、許雅怡、鄭火隆、陳欣龍、耿婉嬌、林志軒、盧名鴻、張育辛、陳政恬、房佩珊、蘇昀柔、陳培倫、陳威岑、姚子鈺、侯于真、吳佳忠、林宗政、劉左雯、李雅雯、許台隆、蕭麗君、黃陽銘、丁德云、洪恩雄、陳慶富、鄭牧寶、趙崇恬、吳倫治、萬香利、林鈺筠、黃俊凱、劉添任、林士民、欒旭城、林品星、張家維、王萬寶、王雅玲、邱俊迪、張豐駿、賴怡山、陳千惠、陳志均、賴映堯、洪育菁、林立宜、方雅玲、趙凱琳、張玉容、徐昭然、蘇維倫、林淑沛、唐禎苓、朱怡靜、王姿剛、謝君吟、蔡嘉屏、林雅文、黎雅筑、黃亮萱、經郁婷、賴怡芬、許志亦、張翔菁、楊緯仲、李怡如、林季杰、趙伊婷、李婉岑、黃慧茹、林俊彥、葉詩雅、黃彥郁、周夢蘭、徐淑華、林勳喜、傅成穎、王嘉鴻、何涵霖、林子亨、張夙雯、施芸珠、賴宇翔、王家齊、郭育儒、林嘉惠、葛欣怡、何佩芬、溫雅云、許怡秀、顏必群、方左劭、李淑芳、林偉倫、張瑋苓、黃得豐、林萱瑄、劉育恩、吳柏定、林佩玉、汪希映、陳怡揚、呂先娥、楊均紹、林秋財、洪怡禎、馬惠珍、劉與苓、黃予菁、楊隆貴、蔡翔年、牛文彥、崔傑淑、馬怡君、王冠茵、黃珮燕、賴怡琳、林秀揚、鄭瑤鑫、謝苡秋、潘文婷、陳妙修、李意行、葉嘉玲、盧政儒、陳品文、王詩聖、浦詩雅、林湖萱、車尚容、黃美娟、黃玉婷、陳雅善、謝奕翔、林恒純、李亞春、王欣樺、許淑芸、楊建輝、吳柏成、郭雅正、潘宗翰、賴金鳳、黃偉恩、陳武男、方念來、張育洋、林嘉希、劉怡君、蔡雅啟、張靜芳、陳智偉、王世傑、蘇禎丹、黃政勇、陳乃文、曹俊豪、林育寧、賴玟君、姚行人、許綠英、陳健盈、唐哲維、李鈞淑、林智凱、陳怡珍、王思筠、陳嘉育、陳美劭、蘇智翔、鄭瑜昌、朱仲琦、徐俊穎、吳惠天、王詩中、吳志海、高雪峰、高雅雯、郭初妍、柳宗仲、趙偉傑、林文軒、林莉雯、陳竹歡、楊登綺、王素恬、陳雅堯、葉雅萍、林明彬、沈妙裕、阮冠惟、顏宥隆、許哲傑、孫舜梅、李嘉萍、張晏帆、張志嘉、劉育沛、李婷婷、陳家佑、陳國伸、陳和亦、王景翔、張怡臻、羅家慶、謝明琳、陳曉雯、賴裕仁、石懿琦、林慧定、陳明松、李淑菁、蘇冠中、皮美娟、李璇慧、鄭蕙然、蔡淑隆、郭昱志、陳君偉、李美冰、張筱婷、謝昆霖、洪淑茜、涂怡文、劉德函、範耿寧、林淑如、許致遠、吳初吉、蔡怡萍、吳怡君、張宗翰、龍登坤、蔡麗娟、黃于惠、郭惠婷、黃淑敏、童恩昀、林家洋、張淑娟、吳侑琇、李妍俐、王吟昇、黃秀芸、黃淑芳、吳淑芬、陳姵汝、張凱倫、林秋汝、黃冠以、陳雅婷、藍雅婷、許昇火、韓佩芳、許仕卉、李裕霞、劉佳容、王信菱、於靜怡、艾裕民、蔡慧君、蔡文天、李芃行、劉育啟、黃家雨、林俊憲、黃緯沛、黃政心、王文幸、吳宜富、陳怡潔、蔡寧星、韓怡君、簡佳琪、賴雅惠、蔡凱文、宋靖雯、陳義發、梅宗憲、畢冠中、林宜英、郭怡禎、劉怡郁、王聖亨、詹雅雯、李容歡、陳彥芷、李婷婷、陳慧芬、陳淑娟、侯家賢、張美惠、方美慧、周雅文、楊淑芳、楊松汝、吳博仁、陳柏華、袁浩樂、房世傑、劉怡均、林芳安、璩宛君、陳財龍、王淑婷、李丹雯、宋舜文、黃英傑、林志軒、陳韻彬、謝宜珮、王右天、項麗喬、陳玉東、王雅琳、林彥竹、涂登鴻、黃怡婷、林雅桂、黃淑劭、周家榮、王雅萍、趙玄云、戴惠雯、陳孟儒、謝恒銘、林淳剛、夏富宜、李志男、盧泓南、孫馨穎、謝淑娟、韓上凌、李明玲、童建德、李隆妏、黃清仁、林雅玲、李怡雯、葉金恬、黃睿皓、盧筠林、林彥剛、黃克雯、許崇福、何惠文、蕭秀娟、施緯義、張秀英、雲秋萍、黃婕妤、陳玉婷、張佳雯、胡怡君、賴姿蓉、王明宏、劉淑華、林瑋玲、陳仰定、黃家宏、陳昱香、陳曉謙、黃亭豐、陳益平、黃文欣、楊雅慧、彭佳玲、黃婉婷、楊艾斌、鈕育妮、張宗憲、袁軒容、阮惠君、張介傑、張典歡、鄭姿穎、柯辛為、楊妙剛、黃揚宜、曾肇舜、傅建德、李淑媛、鄭浩瑄、馮樂星、杜柏毅、黃彥齊、林家柏、許林芝、劉美夫、陳緯延、賴怡凡、李惠如、張宏瑄、黃靜琳、周剛仲、陳群琦、陳俊誠、邢方幸、林姿蓉、李尚年、李宗一、劉興能、劉佳儒、黃惠文、張婉婷、林慧博、黃明枝、王火昕、陳欣怡、賴威任、茅秀昇、涂珮甄、黃淑瑋、黃子隆、陳玟惠、張志雪、宋子翔、林珮瑜、潘慧侑、李玉華、陳倩芳、陳家欣、陳逸琳、謝雯婷、吳婉婷、張肇清、李秋萍、張俊毅、陳登蓉、曾華豪、黃郁妏、賴平學、趙士哲、李勇剛、劉銘郁、黃治燕、游郁真、李思毓、盧政翰、黃慧茜、陳嘉毓、黃致霞、戴于岑、蔡宏傑、謝聿雪、李承啟、陳莉儀、張政峰、彭欣芸、阮必茜、陳麗屏、劉聖蘭、吳佳佳、蔣以哲、劉韋伶、吳偉倫、施淑娟、潘陽育、陳淑堯、林淑娟、陳振航、林維倫、吳珮心、陳育元、陳品方、林美強、王岱任、謝俊安、李婉婷、黃美珍、李杰雲、陳建沛、鄧憶芳、夏怡君、丁欣潔、林思綸、吳雅琳、陳智翔、周秀娟、李呈玟、畢家豪、王蘭利、李淑茜、劉靜宜、郭儒一、張政然、別思芃、楊淑真、白青筠、連君易、張靜怡、賴修雨、陳政峰、王姿民、李文彬、陳仁城、劉恆雪、孫家豪、楊允汝、吳宗吟、楊詩婷、李冠宇、邱堯湖、柳冠廷、張水昕、王慧君、吳富岳、何法亞、顏方秋、黃子揚、陳威妤、王淑彥、張于庭、楊智心、林千泉、王雪治、楊協天、周佳孜、周怡易、程旺鑫、胡亭法、王明憲、張志宏、張美娟、唐國榮、陳怡君、劉乃芃、李雅琳、張玄瑋、李士豪、郭玉娟、陳秉欣、林科盛、李辛春、趙雅君、張怡樺、張江為、葉淑惠、李炳香、賴建靖、張嘉偉、陳建佑、洪蕙義、吳俊良、黃莉臻、陳鴻苓、陳貴紫、曹欣潔、余哲瑋、林家豪、姜群誠、曾靜如、羅伶倫、林聖明、許堯禮、黃怡潔、林玉均、趙英傑、鄭妤緯、陳詩辰、許晏蓮、楊姿吟、黃琬欣、鄭文欣、陳瑩心、潘勳哲、李俊豪、王柏琬、謝佳凡、甯明倫、林玉惟、劉峻豪、林芳恬、謝凱文、黎淑珍、雷大光、方鳳萱、李依山、張麗君、林敬松、戴琇哲、呂思賢、蔡政彥、吳宗穎、陳正瑄、張淑珍、黃其霖、陳佳心、王珮中、陳雅文、程淑真、陳育廷、謝佩均、林馨堯、李翰州、李政達、吳淑芳、蔡明杰、賴雅馨、李逸昌、林佩如、陳山穎、吳淑君、陳婉玟、顏星新、林百夢、王景豪、李宏儒、張維凱、賴千德、呂育瑋、劉樂儀、葉聖福、謝仁杰、李威德、陳怡潔、陳欣樺、唐偉紹、吳俊銘、黃劭芃、吳承恩、陳京秀、詹昕蘭、彭傑季、李艾妏、張雅俐、陳芝發、陳家真、陳尚聿、丁育誠、郭秀珍、聶呈諭、宋士豪、郭家維、蕭佳琪、楊家義、趙民桂、鐘紋男、宋惠易、吳靜霞、李詩貞、李于婷、林怡岑、黃怡宇、林秀綸、林奕辛、梁孟倫、謝辛馨、邱俊汝、陳文桂、馬國偉、陳柏沛、林詩琪、李禎燦、張雅惠、何佳謙、許木義、吳星真、吳岳貴、何卉信、黃廷鴻、童耀南、黃惠雯、蔡宗和、鄭鳳歡、馮于婷、李詩涵、張韋光、陳盈秀、陳惠龍、陳家瑜、蔡宗芷、王偉映、傅淑芬、林睿妏、王偉月、羅佩珊、馮苡迪、李健嘉、許靜怡、陳進韋、陳素發、陳昇然、謝雪禾、李家維、陳珮甄、林力春、王莉雯、連冠政、陳怡真、劉春君、蔡孝人、楊成宸、應美雲、高雅如、黃湖火、楊為麟、吳湘宏、蔡艾斌、李妍昆、林正康、宋宇康、李冠妮、廖偉杰、陳淑慧、李怡姍、陳振瑋、吳鈺火、李育吉、倪宗柔、林倍玟、林冠良、劉翊然、陳麗茂、王俊梅、吳富汝、李素芸、柯小綸、陳建月、吳京光、徐奕雨、吳佳政、謝香成、郭上恒、楊俊毅、郭任妍、李博鈞、林智仁、溫佐泰、呂阿誠、黃蓉屏、蔡怡伶、金民穎、江右謙、楊念和、阮淑琪、李麗興、呂仁豪、張博鈞、張琳啟、平柏青、駱郁婷、張妍添、胡坤瑞、王子星、姜宏淳、葉山彥、劉俊嘉、廖呈火、陳建志、姜士豪、鄭新苓、黃杰來、謝姿彬、連昱廷、林義劭、陳正峰、周志強、王星盛、鮑德卿、鄭雅婷、許中娟、林嘉法、劉湘婷、夏恒火、錢佩君、吳偉倫、李馨儀、黃志昀、胡育霖、黃麗博、李志謙、郭苑火、邱珍禎、陳冠花、姜紹雅、黃陽定、孫倍柔、鄧晏宇、黃旻政、王舜凌、黃怡如、蔡雪雯、黃宜萱、金立偉、潘定茹、弓怡雯、袁佳蓉、陳俊豪、吳怡萍、黃蘭玉、童宗憲、李宛齊、李芳舜、王均瑩、謝文君、黃紫水、姜宛真、陳宗雪、鄒維倫、李志平、蕭家齊、胡崇信、王隆亦、陳群生、鄭嘉鴻、林詩仁、孫靜賢、吳武定、劉偉智、劉怡樺、黃雅玲、曾靜如、何彥伶、王偉名、徐惠玲、劉蘭瑩、林育愛、劉智翔、柳世豐、李淑芬、吳佩名、郭志瑋、姚豪堯、夏毅全、黃一木、高祖菁、唐茜函、黃玲汝、高敬豪、劉語均、陳宗良、賴美惠、蔣秋妏、柯華善、林偉翔、郭佳琪、季仲發、張妍明、林家豪、許上喜、李思穎、陳馥宣、沈靜怡、黃志竹、方崇夫、劉雅萍、林易純、王慧娟、陳立定、林建玉、曾其白、王亦柔、劉美慧、葉哲綺、陳家宸、陳麗鑫、陳彥智、劉雅晴、吉冠霖、劉雅安、李鳳函、錢文誠、連惠文、謝佳容、劉劭亨、謝柏毅、王天明、謝枝玉、連平添、單江仁、鄧冠良、許啟一、陳慧玲、毛婉玲、陳亭良、葉亭希、賴郁雯、曾家瑋、姜定如、余梅群、徐華佩、林俊賢、金貞霞、林雅惠、謝宜旭、謝欣華、李秀睿、陳民凱、陳明伯、黃智翔、林淑惠、陳丹吉、黃怡蓁、韓宣軒、劉政憲、劉惠如、沈心怡、陳俊安、連于婷、蘇淑玲、蔡菁佐、陳俞念、林文谷、李冠良、熊家銘、陳杰舜、廖為生、陳智傑、謝佳伶、許明倫、劉玄菱、林俊賢、程慧萍、陳凱元、錢俊毅、李一岑、沈家維、馮姿婷、彭雅婷、吳惠杰、林淑真、黃淑娟、王禮合、周曉玲、潘天亨、翁友揚、林揚海、陳美君、高得嬌、吳瑩茜、宗天霞、王堯學、吳欣山、陳秀雨、顏辰琇、王苑雪、張淑華、蔡思翰、葉虹良、黃冠真、盧紫祥、藍彥廷、楊雅慧、蘇哲嘉、陳素星、黃淑娟、林宏達、林國瑋、吳予弘、陳虹成、李嘉恩、賴文傑、吳宜琴、蕭意美、任語花、駱辛鑫、王倩善、陳智雯、王力火、胡貞恬、許奕辰、黃麗妤、陳重真、溫欣怡、陳詩琬、顏吟英、蔡佩珊、黃彥紋、張逸喬、簡怡君、周婷雯、鄭鈺婷、李婉愛、林青生、張軒豪、陳致蓉、黃志枝、李任劭、黃雅惠、林漢旺、鄭佳琪、陳亦陽、李維夫、吳俊皓、李宛蓉、劉琳喜、林佐芳、張喬屏、劉彥宇、洪樂伶、徐偉俊、劉雅玲、陳婉輝、蔡佩正、藍則萱、項于秀、張鈺雯、王文希、劉宛真、涂武輝、徐琪學、陳育白、夏雅惠、鄭怡伶、姜雅慧、涂天財、方韋鑫、李彥駿、張秀博、劉宜蘭、王淑芬、陳瑞友、林羽法、鄭佩典、吳雅慧、蔡均妃、王弘恬、林昱怡、陳曉琪、林紋航、冷峻良、許雅鑫、吳岱慧、陳方秋、李俊吉、洪雅婷、李興弘、黃秀凡、蔡力帆、藍怡安、蔡威城、王新正、林秀玲、簡堅妍、丁富丹、張毓新、郭俊緯、陳惠雯、賴怡友、羅晏方、楊友枝、陳家辛、黃姿瑩、李敏蓉、蘇璇漢、陳洋、陳亭聖、冷宗翰、李百白、李欣寧、葉姿瑩、邱靜宜、陳莉珍、陳麗富、郭旻航、李佩杰、林美仲、柯禮清、童冰榮、吳又喬、倪憶儀、邱俊妏、黃世華、吳淑雪、陳君淳、敖介筠、藍明輝、鄭志杰、郭雅惠、龐琇安、沈茜中、黃志和、周侑霖、吳建智、陳傑德、詹艾映、謝家慧、王萱瑄、張宥平、王合陽、高凡慈、林國芳、韓志杰、黃曉嬌、陳淑城、童科恭、楊辛蘋、張佳燕、潘建輝、鐘佩君、何怡文、顏亭秀、蔡文啟、袁孟儒、陳勝傑、黃文富、周可幸、謝玉婷、袁靜怡、陳嘉雨、賴宣啟、陳美玲、楊淑俊、賴麗娟、許哲妤、李麗卿、賴建新、宋智歡、王美苓、李政紫、陳奕筠、陳宏州、周麗華、陳鈞南、王志聖、羅忠恒、許政翰、林俊美、蔡雅雯、王宗穎、鐘慧珊、蔡佩妤、周冠勳、葉君任、林逸靖、杜鈺婷、陸守民、高美音、黃士傑、劉冠樺、趙怡婷、蘇琬峰、李柔鈺、蔡維倫、李仁欣、吳雅秋、劉則聿、周小合、雷淑芬、鄭揚霖、黃育如、楊雅雯、施哲瑋、賴建瑋、吳雪仲、杜恆宣、徐士傑、謝郁樂、錢協恆、楊介雨、孫惠蓮、李東惠、王瑤興、金心怡、賴凡蘭、柳姿瑩、張惠玫、白貞儀、周燦禮、李桂吟、劉珈禮、湯詩婷、吳可姍、楊智士、黃慧茹、李哲華、陳文彥、柳育德、李泰峰、洪禹光、陳冠中、張聖東、毛珮龍、林曉昆、陳育琦、黃士真、周珮君、游亭君、黎嘉軍、李慧玲、樂聖文、王淑芬、劉文合、張雅玟、尹忠齊、周俊賢、張意婷、宋慧琇、林怡年、何和雲、陳柏毅、江柏翔、葉珮甄、吳孟仁、李宜宸、陳明育、李柏任、張世惠、曹雅君、曾婉君、賴成仲、陳智文、涂木誠、李俊宏、戴成豐、齊家弘、陳亭忠、楊姿男、楊建宇、張采來、陳雅惠、甄志瑋、袁怡靜、陳典宇、林孟琇、許以筠、黃淑芝、羅冠伶、鮑婷辛、林玉梅、林世強、張士典、林宜名、黃民妏、郭倫劭、黃彥恩、李彥福、王夢卉、王孟儒、陳美芝、李姿蓉、張景梅、陳東恆、程則洋、陳雅玲、林怡珊、崔志偉、蔡佳昀、楊凱堯、陳信芃、林宛玲、許惠婷、許智依、鄭宛瑞、郭怡仲、劉歡恆、梁睿成、雲雅婷、陳文惟、蔡建良、許雅修、郭紫凡、林君娟、葉筠希、林政祥、張佳慧、邱得萱、潘薇卿、王俊華、趙鴻軍、鐘筱芸、周靜坤、鈕駿和、王鎮宇、黃俊瑋、羅偉銘、溫偉為、林雅云、林雅竹、林士書、陳淑真、鄭惠弘、沈佩珊、賴容真、王琬婷、莊惠智、陳佳穎、陳建定、林淑真、陳淑娟、曾淑娟、張初任、袁萱行、陳曉紹、陳瑩定、左冠宇、林曉如、葉俊宏、張雅慧、陳彥星、鐘心傑、游建智、林子松、蔡雅容、曾豪強、蔡家良、李侑茜、陳慧辛、謝至瑋、徐志偉、謝惠雯、禹智文、錢于婷、孫漢君、莊惠純、謝欣蓁、王靜芳、林麗啟、吳志愛、吳怡虹、陳雅雯、陳家欣、林佩婷、周崇木、王淑文、洪聿昆、陳馨法、王志妏、蔡群學、梁之以、謝明雅、黃海善、趙振豪、陳郁合、侯亦誠、謝柏翰、陳上蘋、汪雅琪、邱志欣、張舒純、苗雅惠、陳俊毅、王郁雄、許宏達、魏與純、李國孝、林文歡、潘凱婷、賴美珍、蔡尹中、林士凱、楊佑芬、農盈甄、黃明儒、黃學惟、劉美雲、林冠珠、趙思妤、高世昌、黃俊洋、曾俊逸、龐可紹、林欣其、顏雅琳、林雅惠、劉俊明、李菁駿、陸冰玲、王建昀、吳志峰、陳麗君、劉天冰、黃彥樺、林佩琇、林佳忠、王淑芳、許友剛、陳瓊慧、黃郁冰、彭綠謙、黃鈺白、林怡均、邱建豪、徐純賢、楊佳瑜、黃火瑜、張世蓮、葉宏偉、游堯枝、馮貴坤、王怡韋、邱冠蓮、王典茂、謝依凱、周珮香、陳宣和、李冠宇、黃鈺竹、吳昇夢、童陽木、黃易蕙、駱宜欣、陳恩旺、林宗翰、林怡君、張采心、石弘智、林予善、周書豪、方俊憲、曹承翰、阮其綸、李淑娟、黃皓齊、許妍歡、吳恆龍、陳維倫、程國維、周珊綺、連昱宏、陳佳儒、李靜宜、郭有人、劉士傑、翁怡樺、鄭美娟、張瑞秀、廖怡君、陳仰奇、韓明倫、沈珮瑜、徐靜怡、林佩怡、王惠達、黃思紫、蔡珮瑜、楊雅博、蔡明虹、劉昇仰、林雅慧、柯姿凌、李美杰、黃珮君、陳綺南、饒姿伶、周進宏、吳博仁、楊秋萍、林芳民、楊淑峰、許瑩竹、張育瑋、吳羽睿、李峻竹、謝文南、戴鳳玫、詹美娟、陳健娥、劉信宏、葉岱柏、李玄月、鄭雅青、富忠翰、汪倫玲、黃昕燕、張宛玲、黃佑娥、黃沛源、張舜昀、陳俞輝、王琬士、陳玉威、黃俊毅、陳佩旭、簡恆宏、高威任、陳建修、賴孟青、黃文傑、林君良、王以綸、林秀季、吳家江、黃思翰、張志文、陳佳霖、黃以惟、張文欣、林育宇、毛鈺勇、王正姍、張軒延、劉冠瑄、張士豪、胡瑜韻、溫淑敏、翁志文、黃詩雅、吳湖鳳、吳姿心、李政芬、李百蓁、陳盈南、蔡宏剛、李明城、童昶雨、李慧萍、蘇怡如、戴琦羽、經美君、林雅俐、黃羽紋、盧芳鳳、李淑貞、張翔昕、陳麗新、錢祥帆、鄧展竹、陳政翰、劉文馨、張恒季、藍宜君、黃怡君、何軒豪、馮欣怡、黃姿琴、楊翰啟、黃慧君、張玄桓、陳憲昆、高惠君、陳君伯、李柏秀、廖水映、周志文、朱淑君、劉健任、楊智玉、徐丹花、曾雅君、劉仲鴻、顏玲泰、吳恭依、張吉意、陳冠福、蔡秉堯、柳佩君、洪美財、林玉玲、侯欣樺、連名威、黃憲恆、黃政宏、張秀芬、丁冠良、王淑玲、倪俊逸、謝凱元、吳建志、林志宇、張菁琦、盧宛穎、劉韻如、郭美玉、童乃一、黃怡靜、任又善、孫婷雪、黃秀聖、王筠華、楊君豪、陳欣怡、賴家慧、林炳玲、陳慧珍、陳姿人、馮一欣、黎枝天、劉苡宣、趙志明、李毓梅、王威宇、鄭淑婷、陳儀意、王奕君、謝建輝、林政剛、馮芃嬌、張瑞惠、林雅淑、李信宏、林彥裕、蕭浩儒、陳秋姍、王聿迪、劉妙亦、楊雅嵐、袁克弘、張松軒、李台慧、陳淑娟、呂昭合、羅俊瑋、姚淑娟、潘佳純、王孟燕、蘇子芸、蔡怡文、黃俊宏、謝健豪、鄭慶豐、吳美華、徐嘉羽、鄭茂香、陳協惠、吳佩玉、袁歡珊、林喬珊、鄭怡伶、方怡婷、陳欣雪、陳信英、駱真維、謝辰音、吳明秋、蔡群輝、蔡淑惠、張建彰、林惠如、駱慧玲、林宥秋、袁寧貞、劉宜真、蔡馨儀、黃阿郁、陳仁傑、郭怡君、李淑娟、梁怡慧、黃佳紫、林宏齊、李依婷、黃育穎、郭宗毅、蔣展怡、施惠雯、滕曉薇、葉怡航、陳家瑜、林育光、駱珮瑜、王順憲、黃玄妮、涂蓉琳、盧子翔、王星君、張雅梅、張佳雄、林正峰、韓予新、邱志維、韓鈺婷、袁雅婷、趙馨儀、夏雅琳、劉俊行、陳奕伯、蔡俊毅、劉文花、黃群妹、阮妍霞、趙志賢、陳彥柏、韓美慧、賴佳靜、賴美玲、鄧凱翔、李家宏、柳柏鑫、蘇雲山、張科劭、曾啟夢、李志偉、宋綺全、李台宸、鄭寶秋、王亭珍、呂世豪、張麗娟、王恆勳、陳瑋婷、陳韋亨、吳惠航、林靜玉、李靜怡、蔡琬婷、林書谷、許東旺、劉淑敏、詹惠玲、毛豐華、杜怡君、謝志文、蔡佩凱、李承姍、楊婷婷、涂怡伶、溫威龍、陳一行、李宜筠、姚枝孝、曹子伸、林宜信、吳怡如、劉勳政、陳欣堯、林向紫、曾淑群、林文孝、陳志陽、彭志豪、賴培琪、梁亦邦、楊雅玲、吳宗穎、王吟源、昌文傑、簡宜臻、姚奕辰、林璇為、林恆祥、林建真、黎侑云、張白美、林皇南、謝孟哲、邱茜琇、盛憶樂、陳凱琳、廉苡怡、王名恆、謝明心、吳鈺倫、林怡合、郭智鈞、尤志銘、唐雅婷、陳文泉、王凡書、王怡禾、張佳霖、柯學瑤、張馨儀、羅家豪、陳韋羽、黃竣傑、吳政茂、施宛蓉、李善樂、賴郁翔、吳智強、楊淑玲、吳玉萍、林承亨、謝采欣、陳麗華、蔣百夫、馮亮謙、倪雅婷、謝育儒、林財玉、張珮蘭、黃慧亞、楊奇發、游佳穎、蕭淑玲、陳宛妤、李彥璋、趙佳蓉、趙益亞、楊萱芝、吳嘉慈、林曉剛、黃子坤、童菁毓、竺義鴻、李羿迪、馮秀全、蔡承翰、張素山、黃婉韋、王志茵、張志人、黃素貞、林佩君、王琬甫、黃子福、黃耀映、林與玫、劉以洋、楊玉玲、侯剛嘉、劉雅琳、陳惠美、馮雅雯、阮毅能、楊江合、張雅智、徐羽湖、陳映皓、林靜明、吳靜春、孫雅惠、劉雅君、陳子揚、林育劭、吳國維、王筱月、孫怡芳、蔡志維、楊政峰、弓俐方、暴慧富、李士銘、張祐幸、周亞法、郭喬花、莊韻如、陳素珮、黃苑軍、劉淑強、羅大善、紀宏達、林佳依、謝佩仁、阮志穎、楊倍銘、陳新陽、張耀淑、杜宛蓉、林淑彬、劉愛珠、黃睿妃、藍怡安、丁雅萍、沈珮瑜、林庭瑋、賴秀珍、陳威羽、周俊成、王則富、吳俊穎、張婷婷、林宏雅、陳建宜、余馥能、劉千佩、劉文全、林乃姍、洪怡菁、楊秀靖、周駿心、黃珮瑜、劉柏廷、黃舒涵、嵇希茂、翁倍為、謝依鑫、林鈺芷、詹香瑋、洪恭然、那宛玲、鐘韻雪、賴行駿、馮進鴻、郭延雲、賴博成、楊若屏、黃志博、林婉瑜、王欣泉、翁右谷、夏家榮、鄭盈潔、蔡志遠、郭淑惠、陳慧玲、林盈和、王淑惠、楊佳蓉、陳淑敏、林丞玄、孫欣愛、張姿珍、黃宗穎、李雅萍、楊明倫、施大鈞、林昱名、張涵仁、洪琬福、程碧琬、朱松學、吳偉哲、蔡欣潔、魏亭亮、戴怡伶、蔡廷宣、黃珮發、陳宜霖、陳淑娟、王林柔、楊南桂、黃鈺星、吳瓊文、謝嘉宏、李惠君、朱文凱、曾靜怡、陳靜宜、吳聖隆、林盈妮、劉俊姍、許哲坤、董依婷、黃禮姍、倪彥宏、張美君、余仕儀、柳晏明、沈尚郁、吳育廷、魏素威、溫岳賢、陳芳福、季彥志、周心怡、黃美青、廖家玟、陳曉沛、羅玉萍、張俊銘、曹柏翰、侯偉哲、林佳孜、石裕妍、黃郁文、張怡婷、郭志維、葉勝傑、林雅桂、李家人、孫逸謙、陳文萍、彭台夢、林伶如、汪喜玉、黃智傑、林任士、謝韋琬、鄭惠君、林志鑫、王水中、許文欣、杜嘉惠、陳郁勇、林俊達、李淑任、賴奕辰、張孟軒、李念正、鄧威廷、呂榮如、楊勳淳、韓函堯、陳如妹、林世傑、林姵君、王紫霞、徐英旺、張玲芸、房韻如、周呈來、吳家瑋、侯慧茹、陳香君、韓昀珍、陳思穎、蔡明軍、陳呈瑄、王婉珊、高秀美、游孟琇、林紀夫、陳俊杰、張宗毅、林明綸、蔡麗華、蔡孟春、李家財、張舒堅、溫韋廷、蔡良玟、張則禮、鄭佳燕、吳肇潔、林函凡、鄭人軍、蔡智強、陳姵禾、洪冠傑、陳靜宜、陳文皓、劉宜真、陳建智、陳美剛、陳婷婷、張玉秀、李雅慧、鄭均生、鄭聖慧、黃文財、黃文恩、韓怡君、孫秉修、廖家昇、陳信芷、蔡志明、邱維侑、劉旺韻、李美玲、袁瑋玲、劉欣云、竺美玲、林香君、張宛強、魯淑琴、黃志昆、黃俊真、許淑姍、何卉新、阮承恩、陳文君、陳欣玫、林育誠、蘇文豪、林珮如、王智俊、吳曉雯、陳宜欣、吳佳玲、祝健俐、葉璇漢、和佳慧、高伊弘、張馨軒、李俊威、林俊盛、白慧敏、周姿妤、吳家豪、陳佳昆、吳姿秋、柳淑芳、蔡智超、翁思穎、謝哲薇、胡可秋、林明輝、劉奕廷、翁志成、謝函茹、謝雅雯、游儒維、蔡宗其、林玉婷、王馨文、盧兆忠、袁盈秀、駱均萱、陳嬌念、李成恬、鄭貴伶、林世伶、張夢強、張一倫、洪俊佑、葉筠士、蔡志琬、林宜年、吳家銘、刁義佳、劉玉紹、陳馨慧、楊善竹、柯宜夢、趙治帆、林家銘、林萱琪、羅士凱、夏雅文、楊揚定、臧佳慧、吳健豪、謝家銘、鐘峻映、黃俊富、陳夙玉、李珮瑜、林欣吟、林筱麟、姚昀芝、王紫珠、陳靜如、陳柏宣、謝穎漢、陳怡君、許家麟、劉淑慧、張淑貞、林志羽、林為恬、錢巧仲、林玉卿、黃浩君、李恆彥、陳韻季、蔡姵彬、陳柏彥、李惠萍、張秀能、林憶平、林淑萍、林玄正、吳柏毅、姚軍和、林志泉、黃莉易、許雯婷、陳柏定、彭家瑜、劉瑋天、林怡婷、蒙靜雯、葉萱婷、彭建德、黃佳慧、駱惠佳、楊孟湖、黃琬妤、胡倍雯、陳宏美、陳亮吟、黃胤彬、鍾哲宇、游怡如、蔡克禾、陳雅以、孫興慈、葉慧玲、賴宛臻、蔡光淳、柯重伯、楊宏泰、陳金男、鐘家瑩、江淑芳、鄭佩芬、李婷婷、賴瑜弘、陳文智、張玄一、柯怡文、吳文賢、林佳南、吳威淳、姜碧忠、林裕冰、張俊明、林以茜、簡欣怡、楊彥君、鄭雲恩、陳夙尹、李麗萍、陳思均、袁初方、魏博仁、黃惠雯、顏庭丞、張雲盛、黃貞儀、吳建廷、陳怡欣、羅珮臻、李宗志、劉美君、陳忠蓮、劉雅婷、孫慧萍、饒治玉、程偉翔、曲旺成、林俊男、陳有韋、陳佩蓉、葉怡育、劉水昆、張筱雯、張儒仁、林瑜信、鐘珮君、林政修、張惠貴、李志傑、溫貞蕙、江郁文、李柏利、查力和、湛志忠、林凱文、陳子揚、黎莉禾、倪俊賢、酆瑞全、陳家豪、白呈年、詹玉原、張志婷、陳惠君、倪乃美、王建珊、鄭豐靖、方秉甫、張維汝、魯庭勇、蔡裕恭、陳明如、路志杰、潘肇宇、陳俐慈、趙典正、許雅竹、林佑霖、林凱元、李珍夢、蔡雪盛、張俞茹、謝展順、趙盈如、邱于斌、席怡靜、武秀玲、劉文彬、蔡鈺善、謝婉瑜、李家義、黃宇翔、韓昆信、陳祐士、趙宇萱、林冠侑、蔡文婷、林清婷、莊惠亨、鈕竹源、陳玉恭、羅其明、劉美妤、洪冠宇、鄭文伶、劉以良、何俊宇、陳昱香、陳柏羽、曾鳳珠、杜妙清、林志彬、蔡琇航、劉淑華、李義花、張菁迪、曹曉美、李淑娟、蔡佩茵、林裕仁、柯亮郁、黃世筠、王佳雯、吳建宏、張俊憲、富耿州、陳南邦、韓靜吟、謝欣雨、陳信帆、葉依辰、李國昕、陳弘谷、別淑華、姜碧潔、蔡一季、方瓊謙、林昭全、林健銘、翁欣怡、趙惠雯、吳有心、杜柏宏、宋明杰、吳燦元、張湘廷、徐若泉、阮堅義、郭慧敏、王俊憲、王人琦、黃美君、王建芳、王偉傑、陳倩苓、吳雅慧、張志成、陳奕廷、戈士傑、黃俞亞、朱淑敏、張台琦、陳威海、傅禹辛、許勝玲、王國辛、張宗穎、顧智文、崔泰汝、涂俊男、曹江柏、楊筱婷、林彥廷、蔡佳堯、李美志、劉益枝、吳曉典、林可琳、余惟妃、黃鈺璇、張恭勇、王祐軒、張群秀、李雅玲、黃雅琪、賈玉萍、白雅雯、張家文、邱宗翰、潘世昌、林婉梅、洪玉華、賴玄真、周羿火、弓欣怡、李建盛、張玉佩、黃佳甫、賴芃恭、吳雅文、梁志賢、謝鈺婷、張惠哲、張秀城、蔡志豪、金建良、盧京靖、黃峻菱、高雅玲、楊淑利、王鎮宇、鄭初睿、翁韻如、陳郁新、劉冠淑、杜佑義、林柏姍、杜雨菁、許志宇、姜俊凱、侯俊廷、蘇向謙、劉雅婷、楊振豪、李恭友、周正季、劉明典、黃美珍、謝山淳、方秋寶、潘依婷、張育萱、游燦芳、程吟勳、林良倩、李宗鑫、王佩琪、苗怡光、李世偉、楊雅華、翁珊白、柯家慧、黃俊德、陳孟璇、唐紋合、林大蘋、庾俊宏、杜嬌穎、黃秀仁、羅怡如、林茹正、劉子韋、李珍綸、林炳鈺、藍怡婷、林庭瑋、李育冰、郭怡菁、馮金真、王夙妃、黎昌函、劉智強、李育意、王建成、鍾淑梅、葉俊玫、黃幸蕙、趙致岑、陳麗富、呂建宇、陳君玲、陳怡靜、陳世郁、夏雅惠、郭于倫、陳家榮、楊羽維、邱淑青、陳學駿、崔莉雯、張之娟、陳群芝、支雅惠、鄭政勳、黃彥凌、李雅雯、胡信宏、賴妙慈、李健山、郭永季、金翠毓、黃愛美、蔡政喜、黃威辛、陳嘉妏、陳逸佩、黃國芸、謝婉婷、李志宏、陳怡君、張秀宸、沈名辰、劉皇哲、柯俊穎、蘇俊宇、李宗喬、林威廷、蔡宗憲、黃玉芳、賴心怡、池青喬、林平豪、陳彥汝、宋佳穎、陸怡文、鄭兆凡、許琬婷、黃珮芬、郭亭君、廉江紋、陳冠茜、陳逸昆、謝岳合、林亮妏、王麗霞、張韋喬、陳行妮、謝艾禎、陳宜凱、夏明軒、房鈺娟、毛康東、王志州、許宏儒、饒愛恒、許行君、白行哲、王真揚、胡俞霖、宋雅婷、王則瑋、吳佩亦、王毅舜、戴士輝、劉奕君、吳介穎、劉郁雪、張玉昇、林姿琇、王怡民、李佳穎、童于真、王宜靜、許昱虹、黃秀娟、趙智宏、許家賢、蔡明禾、何凱元、王佳瑜、袁怡秀、黃呈順、何邦然、林貞然、黃佳玲、涂建宏、許純聿、盧亞孜、方瑜紋、林素貞、黃婉君、沈德弘、郭峻豐、王江雨、張婉伯、林冠群、李美惠、黃偉郁、王博仁、翁俊豪、張兆維、羅俊賢、李欣民、陳岱維、倪璇瑜、賴國榮、劉佳喜、林家銘、丁昆樺、顏惠珊、李威全、黃淑惠、毛隆育、劉育容、杜思婷、吳勇岳、孫季沛、楊威德、劉祥惠、張郁涵、陳彥祥、藍旭治、呂琇全、蘇妍茹、陳牧芷、李威任、蔡婷行、王小堯、陳劭源、袁旻一、鄭有竹、陳盈南、簡香原、楊芸丹、宋兆華、藍一智、趙克伯、藍喜人、李淑敏、姚曉玲、楊惠茹、楊逸瑋、劉呈軒、李建興、林政哲、張美愛、張玉瑞、張毓志、宋展霞、陳靜孜、翁德漢、葉志遠、李怡宣、盧詩雅、謝嘉文、陳依法、陳伊孝、林登舜、周勳冰、陳承恩、姚桂筠、林俊維、吳善紋、劉常名、弓盈如、吳婉婷、阮堯玉、溫文明、吳辰侑、郭淑宣、林勝傑、賴長甫、郭昆伸、王芝娟、楊家瑩、江耿劭、李俊美、江馨霖、黃英傑、賴惠萍、陳嘉惠、王昱伸、楊亦德、陳芸孜、洪大鈞、範惠雯、鄧婉菁、簡淑珍、曾子傑、余宣惠、翁鈞紋、方盈如、張俊宏、葉昆霖、許慧萍、吳宗治、簡佳宏、王于庭、劉宗儒、邱行瑞、游彥安、李慈凌、黃文彬、馮亭郁、張振榮、葉為仲、何智強、黃之昆、李丹民、吳方蕙、張玉茂、鄭惠陽、陳美玲、楊宣菁、蔡馨弘、賴意奇、林宜傑、郭家柔、陳方漢、阮靜怡、林裕翔、陳曉名、李新彬、潘家豪、羅峻洋、呂淑惠、王雅紫、張偉婷、李皓燕、吳俊廷、邵宜欣、黃玟君、張佳琪、周麗玲、陳志芬、盧宏傑、蔡欣輝、李家綺、何舒涵、劉又娥、謝緯蘋、趙佩珊、葉俊廷、程雅嵐、李宜珊、林力州、錢旭燦、廖立生、白晉霖、洪涵祥、李怡伶、柳馨慧、陳家谷、王東恬、楊雅君、黃怡珊、劉麗忠、吳欣平、陳欣琇、林玲民、林秉然、郭建成、陳海屏、吳茜宣、白志強、劉友吉、彭維為、賴健銘、林德佩、陳佳燕、駱懿蘋、林民洋、鄭淑君、陳敬聖、王俊翔、江怡君、金國偉、陳珮瑜、吳淑君、黃欣怡、郭佳慧、林裕沛、黃大行、陳家華、張慶屏、楊浩白、丁宗憲、王佳燕、林珊恭、姚綺喬、陳宛弘、許志然、吳承原、陳威良、王鈞利、張兆亨、彭岳瑜、蘇芃廷、陳心怡、劉雲育、吳淑善、戴韋成、黃俊凱、沈肇寶、陳智雲、徐志軒、高俊維、林育德、張真亨、黃宛真、曹尹德、賴貴夫、李柏宏、李怡倫、陳裕妹、彭冠廷、洪淑萍、于雅琳、陳盈原、陳美玲、賴慧敏、徐淑慧、趙敬云、鄭心怡、蘇怡靜、侯之真、林瑜俊、游佳燕、劉劭宜、張孝成、林文萱、蘇潔仰、陳康睿、林婉年、黃書瑋、倪雲陽、江夙孜、李慶鴻、洪怡伶、蕭禮鴻、黎雅君、楊昇林、柳怡廷、黃冠宏、簡育菁、崔姿火、蔡倩筠、李紫易、陳俐樺、李添姍、孫佑霖、孫乃文、賴哲豪、王予南、黃佩芸、蔡博心、劉枝喬、陳佳岑、溫志傑、李哲星、陳重希、林智仁、陳淑慧、陳玉芸、潘俊德、謝丞茂、劉翊火、楊冠志、李昇惟、劉俐丹、陳曉雯、蔣仲宸、林芳軒、屠火璇、劉燕喜、陳冠意、黃仁傑、胡鈞璇、陳銘政、蔡枝樂、張姿吟、張孟倫、黃建安、張惠佩、李承松、陳瑞強、余昱洋、陳佳博、朱雅婷、夏雅婷、吳科政、姜豪芬、張佩玲、黃士賢、程世豪、許茂歡、沈湘群、張惠芷、吳家軒、邱玲正、鄭宜蓁、劉惠君、袁承翰、林台德、葉冰祥、李肇珊、林肇禮、林秀念、阮琳宏、林柏豪、呂怡婷、童建輝、傅山善、張亭嬌、許雨延、黃聖傑、溫之惟、張宗翰、元美玲、黃耀妃、周竹德、葉美玉、李嘉書、謝堯蘋、蔣俊傑、皮幼尹、錢雅婷、陳鈺雯、邱晴白、林仁豪、楊雅雅、韓佩蓉、陳采白、毛俞帆、趙惠雯、勞穎勇、林毅玄、苗智強、趙珮音、謝卓能、陳維哲、蔡怡靜、許雅惠、李語樺、陳泓琇、陳思妤、李育坤、宮豐如、黃莉然、王俊穎、黃韻如、蔡家娟、陳泰瑞、蔡維哲、林石文、林憲坤、劉芃芸、謝思方、吳修法、黃志維、陳聖龍、陳欣穎、陳筱涵、王書豪、陳志人、路政達、張凱琳、鐘淑惠、趙伶善、黃哲偉、余士豪、許育心、阮曉雯、陳國凱、李宜成、林義茹、孫薇雄、許承穎、鄧怡君、桑逸弘、郭佳蓉、張馨達、鄧信宏、陳秋哲、王振羽、林宛弘、陳俐善、童若合、吳建慈、林佳宏、韓秋燕、吳行信、楊雅芬、陳姿映、黃淑珍、曹燦行、林威廷、羅美惠、詹晴旭、駱浩潔、陳佳怡、陳瑤芃、陳靜法、陳雅琳、錢韻生、賴彥旭、劉與亨、黃佳蓉、司民木、陳信亞、謝秀玲、顏美惠、曾百雨、陳美珠、吳志強、葉瓊廷、王裕江、張姿穎、易瑋倫、黃淑甫、楊雅婷、陳冠奇、陳怡文、李佩雨、龍紫萱、鄭鈺順、曾毓念、林怡海、洪欣潔、陳秋燕、錢筱婷、張姵君、王華堅、張淑華、陳冠霞、毛麗娟、陳人正、陳逸凡、吳志傑、袁心怡、林宏隆、林展斌、胡庭瑋、趙雅慧、陳思吟、金雅如、黃瓊雪、康宗儒、吳曼緯、李喬俊、郭貞儀、黃瑤潔、王家豪、許宛蓉、林依弘、林辰財、陳允恬、李嘉柏、林婉菁、劉雅雯、林家豪、陳明治、陳怡希、賴哲瑋、陳巧云、蔡孟甫、韓奇裕、江天丹、王佳穎、郭俊霖、許琬良、黃怡芳、賴政蘭、趙宗翰、劉佩枝、陳育琇、蕭青志、劉允博、王俊華、洪怡杰、馮書瑋、張雅瑄、黃怡妤、鄭雅岑、彭凱婷、林柏仲、許泰泉、簡倫亞、傅若城、彭旺歡、楊宏偉、黃辰仰、黃淑珍、陳世志、戴明珠、吳文正、胡凱妹、羅宗毅、陳琳州、劉千惠、鄭承翰、朱欣潔、宋彥廷、呂祐誠、彭瑋婷、謝雅婷、楊堯蘋、王英恆、張松蕙、何凱婷、翁怡伶、張淑娟、蕭郁志、張雅倫、瞿常吉、黃琬婷、鄭群雪、梁軒豪、楊雅真、方俊傑、張耀中、楊怡孜、蔡明名、葉喜嘉、劉希年、吳秋生、陳星樺、鄭薇其、鄭伊軒、蔡宛雄、謝玉鳳、王清琇、徐俊嘉、蔡政珍、陳韻婷、劉雅喜、徐堯合、黃佳琪、蕭心裕、余德映、陳建一、李劭男、喻淑卿、王婉菁、林幼馨、李雅和、烏素成、邱瑋綺、陳崇剛、戴晉如、黃麗雪、陳政靖、呂芃琴、吳政孝、黃枝一、賴語州、李靜宜、李雅惠、黃子啟、陳晴宜、陳怡婷、王明紹、楊孟涵、江辛紫、李佳貴、馬明珠、阮堯珍、劉怡樺、姚美惠、韓士賢、駱玉鳳、汪雅玲、呂琳豐、莊惠定、謝佳航、陳雅治、陳盈麟、白文傑、李姿美、王佳慧、張嘉琪、蔡政達、陳威廷、吳淑芬、陳璇名、陳淑紹、蘇文欣、駱映茹、邱郁婷、吳淑貞、李以泰、黃順穎、林韻民、郭彥伶、張山雪、涂馨儀、吳嘉志、楊思賢、許玉堅、陳千樺、張雅如、郭宜紹、方裕儀、劉政仲、林家綺、陳雅玲、蔡懿治、竺麗卿、黃雅琳、鄭柏翔、楊群水、張盈文、邱淑媛、李孟茵、吳佳幸、袁予婷、唐文凱、林欣瑞、林靜雯、王俊嘉、吳一成、黃姿念、陳婉桂、黃婉盛、藍建德、王宜真、洪瑋雨、張紀弘、王景翔、楊倫喜、王怡君、詹銘洋、李俞方、陳威雅、陳燦倫、唐智強、李學榮、袁豪花、盧文欣、趙美娟、黃雅雪、陳培珮、陳怡萱、袁柔安、林勇迪、王明慧、蘇嘉尹、蔣函桓、陳雅婷、王思潔、林育汝、張美一、楊瑤杰、林敬妤、陳宗翰、吳子苓、吳惠雯、王美惠、張瑋財、王姿妹、王韋志、陳柏芝、陳美亨、黃儒誠、王承奇、陳雅琪、黃彥廷、劉雅慧、李貞延、林泰妹、王聖卿、陳惠如、利志忠、林建哲、張婉君、陳昌廷、方舜柏、楊琬婷、李威偉、周儒岳、王翠霞、蔡怡雯、古毅州、馬妤均、黃佩珊、鄭冠良、黃曉恩、王志和、劉卉竹、曹建霖、張偉翔、蔡介斌、黃鈺倩、徐怡慧、王建志、黃建俊、林佳山、林彥均、何家偉、彭展清、宋山正、王崇妏、李國偉、蔡天沛、謝佳芳、李政翰、方筠妍、吳欣蓁、傅傑添、謝阿揚、陳旻志、鄭禎信、李雅嵐、詹雅萱、姜晏竹、鄭志維、吳怡君、楊珮雯、袁世昌、李明雨、陳嘉仲、周怡君、吳凡丹、杜珊志、陳家恬、柳淑慧、楊昇倫、陳品新、夏佳芳、陳建元、林彥翔、劉善珮、林靜琦、戴百洋、林玉萍、江政峰、邵睿毓、陳佳琪、淩志銘、周碧尹、程曉萍、楊茜興、徐珈宏、余育凡、陳有丞、張佳蓉、廖珮諭、杜柏毅、黃辰妍、林品丹、施麗萍、張書瑋、連雅雯、張俊緯、高雅琪、蔡一喬、陳俊銘、陳和淑、倪立芝、陳宜真、鄧孟修、徐庭宜、林子杰、王育德、張智純、許宏儒、黃美修、駱季昇、胡一淑、陳宏湖、王孝亨、葉承翰、郭力信、吳俐誠、周思華、林逸群、侯婉婷、黃筱珊、陳素珠、王宇德、王郁婷、汪志偉、鄭順韻、王麗貞、曹勝茵、馮羿原、蔡曼念、王怡君、陳世廷、張貴湖、吳淑娟、賴聿輝、梁淳伯、符一行、陳姿以、郭婕妤、林金君、吳珮良、王嘉行、蔡聖鑫、毛建能、滑均謙、吳江宜、林志翔、裘偉傑、梁竹忠、張上水、吳弘昆、文茜方、陳淑茹、陳育筠、陳芷翰、詹凱琪、徐廷毓、李雅雯、房邦辰、劉雅人、李予山、葉夢辛、方于珊、黃逸群、鄧晴琴、李杰洋、陳博鈞、張士傑、錢淑玲、毛羿德、吳佩云、隆千香、李怡孜、王江軒、賴聖純、楊文君、盧孟奇、楊宏豐、李菁青、陳佳愛、韓予妃、劉宜蓁、郭淑芬、蔡家慧、林家珊、葉之治、簡宇軒、傅淑華、李政陽、林雅琳、陳佳容、孫正偉、吳山欣、黎雅鈴、王建豪、吳瓊蘭、林妙儒、劉進蓉、黃信淳、林君琴、賴佳穎、林儒希、李冠宇、艾俊樺、戴麗娟、楊世昌、黃美惠、林佩芸、傅惠婷、游子傑、查清能、謝向琳、王宜臻、李菁堅、鄭淑芳、孫怡婷、張彥宇、楊珮珊、陳靜茹、張佳欣、黃文弘、曾昀廷、劉冠達、鄔牧霞、楊艾軒、吳民貴、林羽美、林琬婷、林瑋惟、林國筠、陳宛儒、許琬妮、張世昌、陳昱勇、江佳和、林君哲、戴宥安、曾亮芸、張毓宸、鐘雅惠、朱翊典、張惠如、蔡彥智、夏佳蓉、宮倩全、翁俊毅、郭雅婷、郭妤旺、張政勳、賴俊伯、劉建宇、陳佩倩、張家茜、李羿勇、葉雅萍、黃馨航、王詩雅、張芃俊、陳家瑋、林奕哲、吳志豪、任芝彥、劉慧敏、林憶誠、卓豪軒、吳明沛、李苡均、陳昆坤、吳舒涵、郭宇成、楊淑安、王佳俐、林羿谷、刁易松、鄧怡如、盧常孜、蔡美惠、馮仕婷、謝素山、吳彥廷、邱豐邦、陳玟文、明惠文、蔡俊霖、黃佩儒、洪嘉良、馮國維、楊綠帆、李志豪、林巧臻、趙順恩、江文君、張武舜、蔡文孝、吳智翔、鄭依婷、林怡文、李明翰、鐘美玲、黃琬均、黃美竹、李明雲、許芷霞、鍾明羽、王永蓉、楊彥安、許秀岑、施志峰、張佳慧、柳佩怡、杜淑華、黃偉蓁、陳慧琴、劉志杰、吳柔芝、鍾明貞、張偉倫、陳正麟、謝佳雯、張馨、陳宗翰、許阿迪、黃雅美、張建宇、吳雅婷、梁耀中、胡竣喬、胡思夢、黃常純、何珮昇、曾淑娟、金雨禮、張盛玉、黃家雯、孫樂弘、鄭秀嘉、陳政瑄、黃筱佩、傅成禎、葉家華、胥雅芳、鄧綺安、陳義彬、劉喜辛、賴秀娟、楊佳蓉、王右政、謝佩裕、劉文原、林啟樺、吳之汝、李樂雨、陳靜謙、王竣銘、邱映芳、賴彥孝、郭寶月、郭振豪、陳美珠、張維心、黃俊傑、謝淑敏、張怡禎、王智傑、黃志文、陳曉玲、曾瑤富、汪慶娥、陳松茜、黃靜蘋、曹怡婷、唐伊火、吳政哲、郭綺隆、吳宗舜、林雅茜、楊隆沛、古志松、鄭恆諭、蕭涵宏、游佩怡、莊惠恭、郭家幸、柳雅文、張維菁、汪岱綸、冉偉智、簡昱豐、廖勇枝、童少然、何婉君、王旺月、陳怡君、黃艾仲、鄭常松、潘京偉、葉馨文、吳威德、邱怡芬、陳鈺桂、張婉君、狄彥君、林建揚、劉盈如、吳智文、高雯英、余修憲、黃冠宇、曹奕辰、陳怡君、陳恒舜、駱曉萍、潘威廷、鄭宇洋、賴倫平、童韻孝、吳慧俐、曾宗翰、駱祥辛、林琬婷、李晉綸、姜美玲、陳晏禎、方瑋為、陳聖佐、楊冠伶、毛雨生、郭雅婷、陳奕以、袁順堯、王家偉、黃詩育、林怡貞、林剛劭、賴怡貞、黃政宇、楊哲維、林孟哲、吳浩甫、蘇石雪、楊雅芸、林彥平、駱怡文、黃麗卿、張淑威、許淑敏、陳凱婷、白少歡、林欣宜、陳秉虹、許歡琦、蘇治羽、林正謙、何玄榮、蔣宇玫、林苑定、張品昇、文冠中、連宇正、王郁婷、楊智凌、張柏宏、陳宛原、魏俊明、扈辛婷、陳俊雪、張智琬、劉彥欣、郭江城、周俊瑋、李美慧、黃惠松、劉岱凡、趙佩芬、賈佳蓉、石詩婷、雷勳盛、王明韋、周聿清、陳芳瑜、連怡潔、林士賢、楊宜君、趙民州、周佩儒、黃建豪、張淑芬、巫威德、陳佩貴、鄭琇偉、陳麗君、李政雲、錢香善、葉怡君、劉美玲、林昀妹、華國瑋、李念智、周俊賢、蔡明霖、吳月駿、黃瑋淑、楊亞潔、葉財洋、張淑美、李書瑋、吳韻如、張雅旺、張瑞郁、鄭偉誠、蔡雅芬、李思生、劉妤方、劉政廷、李意婷、楊承翰、王耿邦、劉惠敏、黃建俊、林亮宇、張安綠、黃妍添、黃淑萍、林世廷、林育倩、林威德、詹俊傑、陳維倫、吳建宏、溫家辰、高雅文、黃亦美、林彥翔、徐弘學、何天妹、林思祥、王喜禾、林建佑、李美亞、林建志、黃士菁、涂建宏、陳智凱、汪雅慧、林淑珍、連郁雯、黃婉如、蔡合妃、李志偉、李芷意、楊惠玉、郭希啟、梁麗娟、蕭千麟、王思蘋、林曉念、韓博儒、林雅萍、楊信定、張思穎、蔡宗和、王志平、鄭博文、高雅玲、張素法、江淑惠、李可蘭、林怡竹、駱慧玲、殷瑤花、林明翰、李怡珊、陸新善、張俊良、曹右怡、陳育妤、黃雅茹、賴瑤木、談昱廷、王勇麟、吳怡伶、徐小娥、林柏喬、何莉雯、方志忠、袁逸竹、江佳穎、林仁杰、金丞州、張嘉洋、曾冠宇、蘇慧萍、王佳慧、張美萱、張百火、胡曜宇、彭奎添、駱耀辰、曹冠勳、陳亞治、陳慧萍、蔡育霖、趙振瑋、楊于真、陳雅如、謝明珠、趙雅玲、陳盈娟、曾致杰、李士裕、鬱采昀、吳柏豪、張明康、張柔慧、吳南華、莊怡廷、陳俞妃、謝守祥、陳曉安、鄧舒意、陳曼貴、王寶雅、陳志良、錢聖光、詹雅恩、王紹原、林美娟、楊政緯、劉秉侑、孫建霖、顏力仲、林怡君、李明珮、廉雅婷、林宜岳、郭鳳倫、張玉茹、陳秀娟、黃佩芬、陳惠仲、何勝傑、張景利、吳雅真、蔡宛辛、黃彥伶、賴宛昌、楊嘉宏、簡珮君、劉協航、陳綠法、盧泰吉、黃珮瑜、黎介安、尤佳慧、張博鈞、鄭冠中、王俊琇、傅俊賢、謝科伶、倪宗純、汪紋惟、陳欣潔、張吟江、劉聿菱、劉卓山、曾仁豪、李思蓁、黃馨傑、李建桂、彭容昌、李姿吟、黃志成、李承容、駱詩雲、洪淑芬、許傑白、許思玄、林東辛、許瑋伸、宋淑惠、蔡怡文、黃耀湖、謝文云、謝嘉木、王于真、陳志菁、王凱文、林家欣、林松福、林俊怡、蔡佳哲、淩堅鴻、徐嘉隆、呂名茹、蘇彥良、袁常陽、許思珊、吳康妍、張世昌、張士傑、陳佩哲、陳毓昆、李惠珠、王育瑋、王筱皓、查淑惠、吳珮羽、洪嘉伸、溫玉梅、吳志南、張怡婷、王筱芸、陳瑋婷、蔡慧玲、林奕君、郭曼惟、趙建豪、吳珈為、余雅婷、楊淑娟、黎瑜毓、劉育芳、劉卉枝、簡志祥、宋辛雅、張韋志、吳登心、趙佳芳、楊雅正、張永欣、吳文屏、扈建宏、張為虹、林哲吉、吳雅政、陳瓊慧、周怡孜、陳詩信、張吉毓、余禹儒、高子軒、王翰和、李孟蘋、黃玉強、高怡婷、潘建志、林致遠、胡佩芸、陳偉妤、林立蓮、胡士哲、林淑光、景紋萍、盛靜宜、洪則修、黃任秀、鄧昕玉、郭美惠、丁恭廷、袁昱廷、蘇穎男、張慧萍、陳佩皓、賴安依、林麗志、彭信人、施凱鈞、張珮瑜、朱冠廷、黃國娥、郭偉翔、李郁堯、陳冠麟、包雅琪、許偉珍、劉喜利、劉怡臻、吳玫名、姜倍偉、李國恩、周夙臻、楊淑智、邰萬謙、王家豪、白沛新、王琇和、涂志平、曹美君、吳亦紋、王思映、經之筠、張鈺婷、黃勇姍、劉姿穎、許昭睿、余冠傑、祖子夫、陳志嘉、陳可紫、吳思賢、黃宛君、王皇恆、吳宜怡、謝育萱、郭喬菱、林孟貴、何佳吟、張冠宇、劉宗憲、王奇火、陳宛玲、黃佳玲、陳俐竹、翁妤薇、陳康燕、吳孟慈、蔡重宸、林素映、陳昭花、沈于婷、彭文賢、陳建筠、陳立星、郭亦夢、傅玉蕙、黃馨彥、劉俊甫、陳志偉、李柏禾、林建娥、孫建毓、陳國竹、陳威仁、吳淑慧、盧群豐、鄧永幸、黎政廷、羅雅慧、黃慧谷、林奕年、陳欣迪、楊康桓、陳勇琦、蔡麗美、林怡伶、鄔美珠、郭婉菁、溫文欣、陳俊逸、陳佩齊、林俊佑、周世嘉、李凱倫、吳雅琳、蔡家弘、白孟勳、劉羿淑、楊宗憲、蕭宏雨、吳建其、郭佳慧、鍾明福、陳維水、陳瑞恭、劉祥盈、周美惠、郭清友、張淑敏、洪佳玲、陳雅雯、黃于揚、白士賢、梁玉娟、陳建銘、林恩侑、劉佩君、許進哲、李政筠、邊巧元、劉家榮、謝泓淑、張雅萍、俞涵柔、謝漢明、林瑞名、陳政凱、黃信珊、鄭雅婷、劉冠亦、蔡宜伶、吳燕慈、黃文彬、藍予泰、李宜靜、王艾玫、張志豪、彭建志、吳家豪、鍾明怡、謝進星、廖婉美、劉仁菁、劉智超、徐明杰、林鳳諭、涂崇幸、吳禹堅、蘇欣瑜、葉韻白、林彥文、賴世昌、江憶士、丁美玲、蔡宗綸、王秀玲、王尹美、陳怡裕、楊宗翰、鄭則維、彭明軒、陳建麟、鄭俊緯、黃文賢、施鈺婷、陳瑋玲、陳俊娟、林盛吉、李智偉、陳原洋、王昀瑜、朱吟桓、賴怡如、吳家強、張冠霖、郭俊毅、張智裕、朱秋燕、林函茜、林宇民、許昱萍、鄭家弘、王秀娟、陳芳康、李佩善、陳筱婷、吳慧婷、張俊憲、錢采尹、王素貞、王怡均、張雅坤、蔡孟齊、劉淑惠、甯淑惠、李忠翰、黃于峰、蔡佩義、黃嘉琪、鐘夢恭、許俊諺、劉淳意、郁玉萍、李盛芸、張志香、蔡聖文、張淑芳、李淑真、陶容仲、陳昱禾、賴嘉玲、林于婷、劉純祥、何昕為、林孟泉、張凱法、陳泓紫、許羽聖、李鳳中、游竣夢、黃書豪、李依達、程宛真、溫玉梅、賈明江、席惠珠、阮啟茂、王友典、張志強、黃重揚、林佳慧、孫鈺婷、王國胤、胡豐洋、羅延歡、劉聿欣、曾任興、黃德彬、黃俊廷、曾介瑜、劉玉岳、鄭耿鈺、郭瑤漢、陳映琇、陳政宏、林英年、陳意婷、張郁婷、蔡佩君、陳珍達、鄭呈弘、陳信民、陳智偉、陳惠茹、吳倩凡、張志芷、王岳華、邱宜靜、蕭麗萍、許惠芬、何皓延、鄭群瑤、黃佳儒、張瓊文、吳揚琬、韓善胤、楊冠傑、李秀民、張詩涵、韓軒亮、劉鈺婷、楊柏青、王美珠、吳政菱、李威琪、陳志裕、林啟剛、李羿喜、陳家沛、唐凱文、彭詩涵、莊淑方、吳宜潔、曾哲銘、紀穎青、李宗憲、蔡佩璇、林美達、楊彥安、藍映琦、彭仲來、柳文賢、程香君、黃靜芳、陳詩貴、林昆旺、盧靜怡、林艾玲、蔡依山、李美玲、何皇惠、王儒人、吳文凱、巫必凡、胡秋仲、李孟來、賴政穎、鄭惠婷、施肇良、李家弘、秦希海、李婉安、索筱芸、呂台盈、羅夙奇、鄧新誠、潘行菱、林建良、陸欣怡、鄭建宏、李淑芬、杜浩裕、張淑岑、錢山齊、陳欣迪、蔡曼玟、陳玉倩、楊家瑜、鄧德城、李重宜、趙麗汝、蔡雅婷、張心怡、楊智偉、郭士弘、李雅馨、陳文伯、王菁士、黃琬婷、甄怡君、周又定、劉嘉月、阮少謙、王家慧、許慧君、黃惠美、陳鈺婷、許士哲、盧小心、宋盈君、蔡涵秋、袁鈺婷、張祥智、柳蘭、巴奇典、吳湘南、林玉玲、王美慧、郭漢城、周怡慧、李亮瑩、詹雅慈、劉憶佩、陳玟堅、鄭建淑、錢國華、蔡俊賢、洪家良、黃秀玲、陳淑萍、王詩雅、張嘉玲、劉桂嘉、張嘉誠、林正紫、邱鎮琇、許嘉政、吳景翔、李怡如、黃志茂、吳佩樺、吳玉玲、楊盈君、陳亭君、劉雅成、袁振強、李任岑、張玉鳳、王玉婷、毛鈞紹、林宏傑、梁佳琳、王昱宜、李育孜、陳郁宇、張千惠、宋淑玲、林家宏、邱子傑、張淑芬、陳振倫、林麗蘋、于怡君、趙耀德、蔡雅君、張怡芳、王南昆、施珈啟、梁亞月、王淑娟、林佩君、曾淑娟、劉昕柏、楊皓綸、林揚璇、張聿琦、陳家慶、章惠如、陳俊倩、黃瑤哲、徐嘉來、陳馨慧、李威青、吳嘉卿、陳玉婷、蔡宗喬、彭家維、宋容珊、盧貞孝、陳順茹、陳仲蘭、李家榮、陳憲芳、彭秉州、胡順柔、林威廷、涂珍清、楊惠仁、林威吟、林俊來、張志宏、焦玉玲、王雅慧、林孟文、李和夫、張璇任、鐘雅馨、劉淑芳、傅旭桓、洪倍霞、陳俊陽、陳惠卿、陳宜靜、林瑋貴、劉羿睿、黃千慧、黃姿樺、陳與群、黃琇明、戴儒萱、王家維、陳彥雪、陳子軒、邱雅婷、李怡利、富玉生、林秀玲、方琇卉、吳威岑、羅子揚、吳雪琦、許奇瑋、王育瑋、卞世善、林奕諭、林姿恬、張怡婷、璩佳法、王蓉傑、林美來、黃淑敏、駱怡慧、李韻如、林瑜銘、陳柏中、李雅婷、毛宇茹、李明哲、鄭明憲、黃心怡、許鈺賢、錢宛珮、李秀美、劉妙函、沈家瑜、郭慧萍、王俊妹、楊佑霖、姚智傑、袁佳燕、吳紀儀、蔡淑芳、王雅婷、李宜映、吳舒涵、賴冠伶、楊天成、陳志亞、黃緯芝、鄭琬霖、陳昇珍、崔家豪、羅維名、王芝其、楊倍蘭、羅碧中、何佩珊、王彥蘋、吳豪方、林佳琳、馬勝傑、鄧淑娟、李慧萍、林仲英、陳馨智、黃喜桂、陳映宸、鄭緯彥、蔣姵蓉、黃韋廷、陳芝水、廖辛臻、張京昇、蔡江珍、王雅雯、郭孟涵、周瑋婷、黃軒康、劉琳盛、傅佳燕、顧大俐、蔡元芳、吳佳霖、王淑芳、蕭喬竹、朱智偉、許淑芳、沈振宇、滑佳玲、張先發、伊馥韻、黃峻盈、鄭倍妍、武哲緯、葉辛杰、劉欣瑜、顏宗儒、丁曜宇、林子伶、林美君、高白仰、吳珮雯、陳介恒、袁苡傑、賴先辰、張佳儒、張尹喬、沈鎮宇、吳胤隆、陳柏妏、陳嘉文、郭哲瑋、陳美伸、吳文文、汪志宇、吳政菱、蔡孟倫、陳秉勳、徐紹年、鄭奕辰、李瑋茜、張雅君、賴吉蓉、金慧敏、唐玟君、謝郁婷、劉慧敏、林育盛、黃淑琴、韓珊純、陳振友、許惠吟、謝冠儒、溫秀娟、蔣蕙沛、翁智凱、陳冠廷、林國育、楊惠盛、陳華雅、余穎汝、諸馨儀、張志珍、金文彬、賴俊宏、葉佳玲、郭怡文、王少政、黃雅玲、陳勇如、張尚竹、陳予瑋、方佳玲、李佳真、周善函、郭祐誠、陳婉君、鐘伊恬、曾秀美、林建俐、黃柏俐、蔡佩珊、吳建智、桂上佐、蔡奎博、楊家偉、羅彥芳、鄭茜泰、蔡儀婷、蔡信宏、傅翰伶、廖興成、蔡孟軒、王凱合、謝明恬、林筱廷、周志臻、劉政霖、林君義、游百賢、郭俊英、李惠雯、葉翠琪、吳華喬、張克芸、高欣怡、楊堅福、吳思花、蘇允瑞、姚登妤、王雲真、吳水人、賴艾帆、孫惠玲、謝詩源、蘇佩璇、陳冠恩、杜宜靜、郭柏成、蔡上娥、曹孟儒、郭志偉、黃淑仁、林俊輝、林宗岑、李思穎、黃孟哲、蔡宗任、孟佩珊、吳翰瑞、周雅婷、陳月法、林欣瑋、魏采心、劉秋源、楊登綠、曾冠宇、黃姿凱、陳建元、謝政達、羅毅綺、陳順宸、童佳玲、許盛銘、袁承麟、黃建緯、李瑜佐、倪睿昇、王書琪、張郁雯、徐佩珊、林家倩、李恆容、潘萬屏、吳玲菁、張芝昆、許瑋倫、鐘又治、陳麗貞、柳俊德、劉素月、王郁侑、賴上德、李方延、游逸凡、宋仕柔、張仰云、黃昱昌、陳文芳、宋鎮宇、陳思夫、林英嘉、葉羿源、陳怡玟、林建宇、胡孟涵、郭瓊慧、高秀玲、宋堅治、劉禹香、駱百謙、鄭淑娟、易為丞、吳雅娥、何睿玫、謝姿吟、陳儀元、鄭家豪、陳詩涵、李文香、陳翰丞、黃麗菱、鄭雅云、林君紫、簡哲維、蕭瑞美、張尹琴、彭宗翰、張嘉謙、邱士凱、孫建中、魏偉婷、鐘歡中、魏詠添、吳宗祥、林宜航、錢光孜、何伊堅、胡左沛、林秋嬌、林嘉愛、陳康茹、姚苡聖、黃郁齊、張育州、王東苓、盧勇旭、陳建宇、詹慶吟、林冠廷、李愛茂、陳怡伶、盧凱倫、郭明慧、黃家萍、楊恩泰、張雅君、黃柏法、魏伊翰、蔡家良、張易苓、張朝玲、林姿瑞、穆依婷、毛俊宏、劉文聖、袁孟璇、廖婉治、蔡乃君、張兆中、蔡雅昇、陳鎮臻、鄭玉和、陳明軒、柴家華、余君豪、吳琳安、吳樂典、陳佑瑋、張杰璇、吳平名、柯怡君、鄭慧玲、蘇修紹、吳俊德、邱冠儒、許威廷、謝陽木、戴欣怡、鄭惠白、楊瑞祥、周景苓、蔡嘉玲、梁彥安、沈耿辛、梁敬寶、王璇福、葉家瑩、魏沛弘、游青孜、龐俊逸、郭旭倫、劉皇雨、陳姿吟、許怡如、黃禮妃、陳嘉雨、黃慧敏、王志嘉、陳宗翰、白秀玲、黃靜雯、陳孟揚、張聖凱、戎淑娟、張豐萱、王大鈞、宋彥安、林冠伶、沈弘旭、戴怡珊、黃佳儒、劉美盛、陳怡芳、林芳侑、吳依婷、李淑男、詹初婷、彭舜佳、黃郁珊、梁郁婷、徐誠堅、林家瑩、陳群芃、余憶佐、陳登琇、陳新祥、惠恒月、毛祥軍、陳光芸、林育文、許夢富、錢信宏、梁可揚、李玟恭、韓漢劭、方怡如、陳紋雯、戴燕馨、郭淑霖、王尹士、陳盈治、邱瓊文、黃松月、謝家豪、陳瑩苓、鄭姵松、林念綸、黃伊啟、顏貞念、陳先恭、洪富恆、蘇欣樺、林育坤、蘇星淑、侯忠定、李駿桂、張柏宇、陳季發、齊信豪、吳建宏、張欣怡、黃冠偉、賴建宏、陳儀裕、許良全、黃明倫、吳方春、黃方雲、吳文彬、李俊宏、楊信豪、傅至憲、方政達、黃昱琴、游靜雯、錢建彰、吳品良、王偉聖、許紹依、鐘家豪、王俊杰、林俊欣、劉百睿、陳嘉慧、張民亮、蔡于珊、張志銘、林志宏、林世豪、韋華海、李俊彥、劉品陽、趙思涵、陳均玟、陳婉如、楊昱紫、李若枝、黃郁蓮、謝彥妹、吳俊佑、黃建吟、張千慧、吳佩亦、藍冠傑、蔡依航、吳偉智、楊彥智、黃珊韋、吳俊誠、謝左維、傅瑜杰、羅怡靜、劉家勇、陳嘉喬、陳韻函、沈雅玫、馬紋音、陳素珍、趙毓旭、陳培新、王世昌、馮月紫、王雅云、梁婉君、周倫韻、張育昀、王宗翰、杜美惠、王雅慈、張志弘、曾信宏、陳秋萍、黃亮方、林嘉龍、林逸羽、吳哲豪、施佳蓉、林韻如、周韋廷、鄭婷芬、黃怡芳、賴必平、張夙盛、張佳念、竇方山、謝恭漢、柳彥廷、葉淑媛、劉彥純、趙允剛、侯淑如、林智傑、黃珈谷、林國康、張慧婷、吳俊仲、鄭景慧、張彥博、吳于婷、鄧均順、林雪蓁、徐淑禮、孫舒舜、桂孟倫、張俞成、王昭宇、曾家銘、唐聖欣、楊琪岑、鄧怡君、羅怡瑋、曹倍文、吳百博、吳哲豪、黃博鈞、魏俊維、童雅婷、黃世豪、郭家弘、黃啟花、鄭立琇、趙昌哲、李志軍、黃筱涵、吳宜潔、楊蕙依、鄭志嘉、李雅婷、李雅婷、馮萱春、郭又臻、李耿威、陳家偉、賴家豪、李美惠、鄭哲裕、蕭佑霖、孫宏雄、林雅玉、蔡雅萍、王海任、賴柏宏、吳世豪、楊炳琴、林永弘、金興水、楊秀英、阮哲豪、舒怡君、林惠雯、孫芝和、楊威宇、吳志貴、蔡佩英、蕭偉誠、陳綺玲、陳怡萍、黃台修、孫鈺婷、錢珮甄、李文凱、廖珮隆、林永添、陳國凌、周湘凡、許嘉書、王嘉伶、張以凌、洪建宏、洪嘉貴、孫建緯、奚淑珍、郭美珍、黃俊毅、賴俊傑、黃淑珍、蔡思賢、李聿佳、柯少孝、錢凱鈞、倪啟元、黃淑枝、郭佳霖、趙信宏、陳豐源、鄭美玉、趙皓心、曹禎佐、謝安宜、許定翰、劉雅中、蔡志枝、毛心怡、陳奕堯、陳文馨、陳惟正、陳淑貞、趙子天、陳家賢、蔡台均、林偉傑、林江全、李思易、張康天、傅璇蓉、楊嘉鴻、黃昀行、懷財枝、陳立君、李宛全、盧怡文、張亭君、楊泓蓉、陳孝鈺、陳合軍、黎宜珊、黃嘉豪、李佳瑩、康佩璇、蔡怡慧、趙韻宸、劉怡均、許惠俊、張薇亮、楊芳瑜、黃介亞、陳春幸、丁佐蕙、李依芳、王冠良、程康仰、洪偉純、蔡玫樺、童昌希、陳柏康、黃若年、許堅凌、唐雅鈴、許容凡、陳永花、張治聿、唐函愛、陳阿甫、陳冠岳、郭韋吉、吳聖貞、王偉倫、楊一欣、楊琳孜、楊佳燕、陳威陽、高曉薇、黃慧妹、陳玟幸、吳冠傑、陳麗伶、趙添屏、陳信坤、鄭蓉雲、林筱涵、程彥均、李秀琴、張淑媛、李志偉、謝曉玲、陳柏宸、李承翰、周軒財、鄭琳名、劉政勳、謝文霖、明鈞友、杜雅芬、吳延毓、林信鴻、郭昱廷、施茂隆、杜郁翔、王裕仁、王淑芬、陳家卿、殷宥英、陳凱穎、鄭中郁、馬志岑、李冠正、王靜怡、丁宣江、吳靜喜、涂俊宏、孫志平、劉家岑、李俊賢、張子豪、張玉婷、許敬傑、張育任、陳慶勇、陳怡臻、黃靜齊、鄧晴友、賴士傑、楊淳中、詹雅琳、丁涵正、陳智強、邱侑霖、李玲江、蔡予傑、蔡苡喜、陳語羽、李彥虹、林淑芬、邱芃為、蔡沛江、鄭湖華、朱政達、陳玲妮、馬孟璇、丁白馨、張吉天、蔡承芸、楊靜怡、楊志軒、余昇美、涂志傑、賴介玉、夏俊凱、黃柏季、連麗華、張冰辰、王雅雨、吳之辛、郭仁杰、蔡瑞名、葉志豪、蔡文凌、李心怡、劉敏水、張文馨、詹雅淑、汪恒彬、王建彰、陳淑玲、吳家娥、黃佳佳、黃恆燦、黃佳玲、李先翰、阮定佩、賴綺亦、林婉君、陳威元、張維妮、謝希廷、曾昱淑、陳育幸、卞丹春、郭婉君、陳品胤、吳蘭聿、呂貞儀、周敬龍、黃冠盛、王雅茹、郭永宸、吳博文、屈克源、涂志瑋、彭馨明、張秀玟、姚俊霖、黃世桓、周佳盈、陳于芬、吳美修、侯堯富、蔡雅婷、陳東玫、王仁亮、柳家亦、吳佳霖、呂雅惠、黃涵樺、吳益坤、王向胤、邱力南、張怡蓁、謝家瑩、劉辰琪、陳炳芷、楊誠昌、杜怡禎、項宛臻、楊惠瑞、陳佳春、李美甫、李京鳳、吳佩辰、張翔祥、汪彥廷、鄭文岑、張劭秋、林武昇、陳振桓、賴倫嘉、吳靜如、劉致易、郭進生、林火嘉、金芝紫、王允禮、吳佳儀、李冠儒、李雅婷、張志意、郭家銘、張毅妏、張家婷、吳文白、蕭雅琪、趙方芝、葉柏毅、蔡欣琪、謝毅竹、吳朝文、劉純梅、陳涵羽、張薇然、吳怡亦、鄧冠緯、郭雅慧、林志峰、鄭雅松、蔡宗筠、郭秀娟、楊玉婷、陳雅竹、黃潔青、陳巧甫、郭建彰、李允任、郭琳貴、陳健歡、陳國珍、李佳士、童水其、吳玉婷、陳宥法、賴思潔、劉俊昆、鄭佳霖、蔡明音、鄭憶恬、諸國榮、袁惠婷、童婉瑜、林書豪、蔡巧昀、謝婉菁、竇俊德、謝宗翰、李淑惠、程芸劭、連雪凌、黃俐苓、梁純生、楊誠樂、鄭軒文、施先宸、李盈其、許秀珍、陳佳桂、皮愛緯、陳玉梅、趙萱雄、謝佳怡、張怡臻、陳珮達、賴怡廷、陳賢雪、林政儒、方瓊慧、謝仁豪、鄭宇伸、張冠傑、張與軍、李敏銘、蔡雅亦、李良輝、傅政憲、黃怡潔、陳雅茹、陳欣湖、陳以雪、柯岱惠、王怡芷、宋智翔、陳馨海、吳依婷、程雅喜、張文傑、張俊杰、林筱光、吳紀梅、劉怡瑄、王志法、黃美珍、蔡秉博、許瑋婷、余育萱、李育麟、潘惠珊、廖婉發、鄭佳玲、陳佳玲、陳明易、鄭郁婷、夏雅惠、王文財、林昌和、謝俊翔、夏惠君、蔡曜宇、林惠珊、熊宣樺、陳志枝、陳宗翰、蔡彥安、張家筠、簡志郁、黃思翰、葛彥志、徐政哲、杜登諭、丁佳慧、陳恭美、賴建盈、趙林傑、許健銘、林正霖、蔡信生、楊淑貴、趙凱文、程怡芳、黃美洋、丁佩君、張必伶、陳恩胤、賁緯甫、陳韻旺、林秀麟、車秉珊、陳曉玲、張瑋倫、賴姿伶、鄭采惠、謝靖全、張馨儀、馮志銘、許宗翰、吳建文、連宏偉、李宇翔、黃嘉貞、戴千桓、林崇軒、黃馨儀、陳堯強、王淑梅、李冰瑜、賴秀玲、鄭惠宜、王詩任、廖詩瑋、林崇芃、王冠蘋、鄭淑均、蔡文君、張吟甫、陳品欣、陳韻奇、吳怡如、李依謙、劉均伯、邱志豪、彭淑君、許冠儒、呂晉中、劉雅雯、張玫甫、張哲維、陳威杰、黃祥佐、陳呈峰、張佑綸、吳名珠、張政勳、李佳修、陳慶雨、楊吉惟、黃偉純、許淑萍、吳邦音、陳俐洋、徐佩珊、林宣邦、陳和揚、盧昱廷、黃冠儒、方健愛、劉文恭、李薇冰、林韻雄、王建霖、謝湖榮、邱惠雯、劉意合、蔡景翔、劉欣穎、鄧姿珊、許重沛、張嘉慈、郭珈俐、王昇珊、陳俊傑、許惠順、曹詩學、王明珠、陳偉來、劉育謙、翁韋志、鄭淑貞、劉江伸、王俊吉、謝佐清、吳瑋婷、林博仁、胡晉揚、韓奎均、錢建廷、林雅琳、李杰民、陳美杰、韓宜芳、宋盛燦、吳禹辛、邱豪博、陳協文、黃彥一、陳智傑、林靜怡、張怡君、李育喜、屠孟君、蕭軒志、張季軒、林佳誠、劉韋志、簡昭函、周俊成、吳又愛、邰凱文、鄭凱甫、林佳慧、丁國維、陳子豪、陳思潔、邊蕙馨、謝豪苓、潘雅慧、許美珠、田庭年、許雅季、阮怡妍、陳慧玲、藍慧玲、蔡雅妮、黃孟寧、賴慶芷、賴姿穎、蔡夙書、趙麗玲、王郁燕、黃靜東、王予雯、劉偉翔、劉民姍、翁杰容、金勳盈、邱家榮、梁俊賢、葉舒涵、吳淑婷、劉瑋婷、廖偉臻、林逸雪、黃嘉妃、謝毓雨、林怡如、金軒如、陳采恭、王緯昕、丁俊嘉、李常義、黃琬婷、李賢亮、曹怡靜、李欣合、林怡紹、王嘉偉、汪岳昕、陳立偉、吳明輝、馮怡萍、劉建紫、林婉君、陳良雅、何靜怡、王宗喜、李俊佑、楊秉修、隆聖竹、陳淑容、張世偉、駱弘冰、林艾發、陳聖傑、溫盈秀、蔣嘉文、林芳堯、陳呈卿、鄭奕云、陳宜珊、杜玲函、劉俊緯、陳家夫、李志弘、謝冠志、洪岳卉、鄭惠雯、陳志旭、羅晴文、張綺哲、劉雅婷、崔至苓、鄭彥宇、黃鎮強、鄭伊新、鄭家偉、李登豐、賴展緯、黃韻法、陳秋萍、陳慈卿、劉欣怡、陳侑仁、柯佩珊、吳緯淑、錢必郁、鄭雅婷、林翊歡、張世辛、馮政翰、嚴彥智、謝志杰、柯國瑋、吳怡禎、鄭志豪、劉克甫、林峻豪、方家慶、李亮妤、陳丹成、鄧清綺、劉志祥、王詠明、林俊諭、陳智傑、賴俊屏、黃雅雯、沈雅帆、郭天貞、王菁旭、藍偉甫、廖珮昇、蔡怡慧、許紹信、陳柔育、陳怡毓、張雅文、林怡容、鄭柏鈞、侯宛君、陳婷馨、顏舒婷、吳必谷、伏雅鈴、張乃佐、溫佳蓉、伏志軒、邵柔偉、吳建蓁、陳政儒、柳智堯、鄭軒昆、簡旭亞、王香貞、溫治孝、吳雅偉、吳玉玲、高郁婷、陳玟啟、林孟軒、黃士諭、柯慧玲、張士琴、張林揚、李詠友、許志穎、林南士、吳佩吟、傅虹奇、楊興任、吳嘉雯、林巧惠、張政霖、程佳欣、王文傑、高素為、葛進雅、許俊維、別嘉雲、王郁侑、盧仲州、張政憲、喬怡如、蔡惠堯、簡志佩、李俊明、吳于珊、鄭智傑、郭家綺、吳素駿、蔡孟玟、朱怡君、李希芳、張育雄、王泓昌、洪仁英、劉家偉、蕭萱弘、黃思潔、林任珮、黃茹孝、黃柏成、張淑博、吳裕軒、林國維、林于婷、陳禹亨、林彥琇、吳侑臻、洪雅婷、林哲恭、陳嘉隆、陳依宣、蔡淑裕、蘇博珊、唐愛典、鄭修福、王俊愛、楊向樂、謝文湖、高馥玲、郭勝洋、李孟隆、許凱倫、黃柏翔、陳柏卿、李東花、鄧怡芳、林政琦、黃月方、蔡佩杰、黃東民、鄧家榮、林慧孝、何真航、蔣伊其、黃文傑、陳必珠、許哲琳、郝珮宣、梁文馨、林慧達、鄭曜宇、劉俊順、郭志平、盧玉娟、陳宛希、吳韋玫、許雅紫、江冠志、簡守平、黃詩冰、祝民琪、陳鈺凌、黃士盛、黃建筠、袁裕安、張誠怡、謝冠宏、李凱婷、蘇妤萱、吳智傑、洪國美、黃乃雨、黃麗信、黃馨儀、許涵弘、張雅行、陳馨昇、梁慶利、王正東、姜石憲、翁靜怡、居靜宜、李佳昆、王舜琳、潘心怡、徐偉音、李家倫、杜峻慈、林宜良、張明翰、陳佳娥、劉豪豐、陳元富、許志強、許美雲、郭偉孝、陳志慈、邱建豪、張怡均、鄧芷裕、邱亮勳、李怡潔、何軍璇、張偉倫、洪嘉剛、張哲成、蔡雅雄、趙潔鈺、許怡伶、施家榮、黃筱婷、陳欣儀、陳鈺亨、鄭金佐、溫孟儒、屈慧玲、劉羿芝、謝逸來、李江芝、朱佳琳、陳信珮、林靜妏、汪登琴、吳淑琴、洪翊依、陳雅湖、杜和年、許玉芳、陳玉仁、謝依琦、姚孟哲、翁鈺軍、李美玲、施雅芳、黃可綸、連依潔、蔡真丹、李吉智、柳之意、黃育儒、陳佳蓉、林淑海、郭美玲、陳政善、楊怡文、蘇琬婷、李鈞恭、黃姿筠、吳珊恒、張世菱、楊忠龍、林佩珊、葉育如、陳慧弘、蘇世倫、黃麗雯、林景蘭、陳巧齊、李佳琪、陳郁淑、林宛意、黃軒豪、許雅佩、林秉青、林玉湖、陳右怡、陳耀德、黃恆沛、鄭淑盈、許詠恭、羅名白、黃怡貞、黃文玟、林松琇、朱俊宏、李奕君、何佳枝、梁政哲、鄭淑惠、黃建瑋、賁左玄、曹家佑、陳雅玲、林家豪、林曉雯、陳上琦、黃芃竹、張明珠、潘瑋婷、黃麗音、陳孟君、柯隆尹、吳姵璇、陳台杰、謝博方、陳家銘、蔡宜柏、黃耀琪、羅怡君、高翊淑、陳新玉、金淑娟、蘇美惠、謝家宏、黃明雄、陳慈來、李智岳、周怡婷、張冠良、王晉泰、張珮雯、王儀潔、蔡瑋婷、黃珮合、蘇秀玲、杜介凌、周喬君、許哲韋、朱揚仰、曾人漢、孫依婷、解冠伶、林怡君、徐奕玫、葉瑋季、林建智、許羿旺、張白坤、陽亮惠、梁石凱、曲淑芳、陳淑慧、陳雅君、魏林正、平雅琳、劉順恩、李建豪、王賢任、賴振慧、陳昇財、林佩山、蒙詩雅、李佳穎、黃鈺英、劉麗卿、林仲祥、姚瑤憲、王政勳、潘剛然、陳志紫、丁佳琪、王怡臻、鄭予怡、張妍州、黃貴隆、錢婉玲、勞傑典、謝瑋君、陳孟修、胡定貞、許昱合、趙智偉、羅蓉瑩、謝雅竹、陳善良、羅佳蓉、彭吟榮、陳怡璇、林家佑、羅佳定、裴淑娟、韓宗穎、吳芷法、趙一孜、何怡潔、黃紫木、張台群、王筱純、孫家卉、蔡維旺、楊珮雯、張士瑋、許哲茵、嵇雅雯、簡夢泰、馮雅娟、蔣麗美、黃志文、楊詩治、姜法華、張淑媛、施建宇、陳信沛、錢靜雯、吳旺群、張凱恭、林明啟、毛卓玟、鄂琳人、邰雅雯、黃韻如、梁靜雯、陳昱威、王佳慧、林明彬、賴婕妤、鄭亦淳、賴俊賢、王佩如、許裕翔、林韋桂、鄭泰倫、王郁美、張恒為、蔡佩芸、連婉婷、鄭玉婷、林子軒、林文凱、劉宜蓁、倪琪綺、魏翠山、李茹馨、游豐吉、連振瑋、溥慧芳、吳義榮、範淑卿、陳思妤、陳美瑋、林雅星、謝彥廷、蔡怡如、游佩芬、王珍來、許偉倫、劉雨蕙、劉雅玲、陳柏英、葉哲青、陳雅文、蕭尹睿、陳奎燦、林郁萱、江雅嵐、陳素萍、陳怡萱、謝心怡、劉家豪、陳方任、黃林順、陳耀弘、張瑞凱、蔡育廷、艾良青、郭俊明、張雅婷、黃慧玲、吳凱倫、黃文堅、皮志維、夏育萱、文宛玲、江麗娟、王宛臻、陳美凡、杜明杰、吳佳柏、陳睿桓、謝宜潔、韓秀琴、馮先堯、劉傑任、陳均芬、李佳容、陳逸聖、陳柏青、劉淑如、詹安琪、詹仰辰、葉惠芷、曹家佑、李淑惠、宋士瑋、黎書慈、林良其、藍順杰、周佳玲、陳美如、林威廷、郭弘倩、林宜宜、路行鳳、張怡伶、陳家豪、吳敏軍、鄭欣樺、謝美仲、陳靜菁、何志明、陳佩菱、吳堅坤、楊志任、林聖傑、吳夢水、林育倫、徐芳霖、李宛蓉、黃莉婷、李俊達、林冠伶、陳怡婷、陳宜強、黃耀靖、黃舒涵、詹思茂、王建智、劉柏宏、管佩如、陳柏威、鄧書仁、陳定茵、楊東佩、莊美惠、陳佑霖、陳志文、劉子辛、陳玉娟、王俊茵、鄭苑玉、傅岱美、吳承弘、廉若宏、唐國榮、林昇南、崔翰成、殷雅筑、顏昱月、林晉儀、王亦偉、劉依婷、曾淑華、李冠花、吳志嘉、謝佳桂、張耀恬、林郁岳、吳旺恆、翁瑋婷、趙聿春、韓卓政、林柏汝、趙淑芬、王惠元、劉彥如、李茂鈺、張璇柔、連靜如、楊偉智、陳雅惠、李彥娟、朱宇邦、王昇信、葉志海、林美君、林宥侑、李彥霖、黃忠翰、陳幸泰、盧伯州、陳展易、黎春堅、陳欣儀、鄭惠信、陳竣傑、黃玫翰、溫文宏、張怡萍、黃淳妹、張建良、許佳慧、王欣穎、郭威任、張坤誠、李昌妹、黃俊英、陳怡靜、白新緯、陳怡蓮、王容財、錢柏青、王傑凡、陳玄宣、陳寧侑、邱怡雯、李姿盈、林文怡、黃偉泉、鄭伊珍、王振瑋、李婉孝、王懿侑、陳懿樂、柳枝年、林冠霖、陳毓惠、盧亦柔、吳佳天、張嘉名、潘聖凱、楊心怡、鞠苡輝、蔡佩萍、陳恒妃、王志偉、陳婉裕、洪姿伶、蘇振豪、崔守紋、鬱函純、王偉銘、許育康、鄭雅然、蔡宜新、蔣柏鈞、王琪修、林士豪、陳俞宸、蔡宗純、傅方辛、周麗雯、唐昕佩、林冠真、黃若孝、鄭兆軍、林怡福、崔艾強、何政廷、鄭瑋倫、王月元、賴珮甄、林秀盈、張乃文、劉敏潔、孫皓嬌、徐健欣、陳永瑜、陳亭妮、林正桓、姜雅茹、黃家銘、楊彥志、王凱鈞、涂豐樂、王郁韋、高佳蓉、楊智韋、王裕星、陳松寧、潘維仰、李麗原、茹誠瑄、姚敏伸、林吉蓉、陳恆嬌、林恒鳳、陳建宇、吳珮芝、黃孟谷、陳盈琪、黎阿江、楊振木、謝俊德、陳彥純、趙哲維、王人興、景智文、方茹輝、張俊宏、齊佩玲、黃春惟、黃俊年、郭淑貞、李怡雯、程奇凌、何添昀、祖書綸、劉品寧、蔡堅亦、郭羿宸、羅惠茹、詹雅玲、林靜雯、張蕙瑜、倪婷昕、朱杰月、喻湖玲、郭宜玉、魏湘茹、王家豪、顏志欣、楊郁俊、李喜玄、張智凱、陳家筠、劉佳宸、蘇旻勳、張長紹、王偉貴、楊郁韋、邱宜芳、鄭怡君、張沛新、施之樺、李美惠、吳丹泉、呂奕君、黃俊賢、駱水桓、吳欣全、劉志恒、謝玟枝、李志傑、吳彥君、陳貞姍、詹典華、陳善霞、張俊賢、黃家傑、林耀德、顏聖凱、張希恬、許朝琴、祝佳儒、蔡思妤、趙昀聿、黃智星、陳孟宸、楊景信、吳柏豪、張湘行、吳莉雯、王彥瑋、陳芳儀、鄭彥智、藍祥剛、涂先治、葉思穎、郭淑娟、林政海、林文君、王妍明、劉俊嘉、陳文豪、李明柏、黃家希、錢伶誠、林冠盈、柳仁杰、何惠婷、林琬桓、黃鈺胤、王皇法、羅韻如、蔡筱婷、顏學心、程美以、張姿婷、楊育維、陳雅惠、張金旭、陳語卉、葉偉毓、李思涵、林憶宣、楊倍冰、楊怡如、游玉裕、陳振斌、許惠孝、涂紋銘、張鈺芃、黃惠如、江介真、吳合秀、吳雅文、鄭映桂、李志福、廖志虹、莊惠娟、林芳善、何馥亮、周怡婷、李千孜、唐逸尹、林芳靖、呂芳如、鄧欣怡、徐偉彬、王雅文、陳俞弘、張安琪、王雅婷、符宗憲、蕭文華、白又寧、黃琇燦、陳雅惠、蔡宛峰、王韻軍、張淑芬、蔡依花、林婷倫、王美生、陳芃元、王亭恒、廖貞弘、吳皇為、黃雅婷、陳正韋、陳品正、王孟璇、曹詩涵、胡莉婷、蔡群邦、石英妍、蔡宜仲、宋麗娟、楊淑惠、黃雅夫、阮靖皓、呂志穎、李堯霞、謝君豪、林雅慧、周美玲、高凱琳、韓群潔、楊筱冰、林雅群、蘇逸秋、洪華薇、黃逸鈺、溫石鴻、張明士、吳麗君、陳家祥、葉冠勳、吳惠雯、林逸茂、張俞夫、陳玉忠、王冠育、蔡惠玲、蔡美玲、林建人、金惠如、高慧玲、張于康、許美慧、吳佐旺、張昀汝、彭佳瑩、米碧斌、謝育蓉、戴家榮、駱惠婷、馮燦倫、黃昇玟、姜其生、李胤蘋、陳佩芸、謝思妤、阮靜如、郭維聖、王名柏、吳心怡、蔡瑤淳、李建德、張景紹、林佑航、陳涵貴、陳哲銘、陳柏惠、許奕君、張家佳、張明翰、黃聖谷、楊雅盈、劉佑義、許雅惠、許欣穎、吳與清、蔡盈甄、張秀慧、劉凱婷、張詩婷、汪玉婷、夏枝宸、崔佳伶、林明宏、白家豪、宋貞薇、戴雅君、馬禮芃、黎心怡、蔡宗玟、吳宗青、林函紹、林力財、查典馨、張聿蘭、李意婷、王瑋航、侯建文、張佩芬、翁珮君、韓俊穎、吳姿吟、李秀年、陳明宏、王貴琳、盛慧娟、賴彥喬、劉麗娟、吳美君、周宜珊、黃靜如、吳雲名、孫雅玲、藍玉婷、林雅迪、黃容宏、唐欣怡、顏瑞函、陳玉嬌、謝幸仁、張雅玲、閔財英、游美惠、洪國桓、郭家華、王俊茹、吳欣盈、陳婉昀、許博文、陳詩盛、林宜婷、孫妍瑩、蔡宜萱、楊建豪、童靖興、曾慧萍、陳舒俊、黃雅盈、陳冠玉、黃綠真、郭麗娟、許偉婷、郭郁婷、呂佳皓、林佳穎、林巧生、吳玉凱、蔡宇翔、楊淑湖、蔡嘉慧、吳孟涵、吳靜勳、林沛紹、陳行堅、張淑婷、洪佳儒、謝承翰、袁淑惠、林育瑋、陳姿舜、李筱雯、陳冠孜、張伊婷、黃國柏、貢郁昌、趙弘娥、鄭翔強、陳佳均、孫家豪、陳千婷、賴信玉、張偉銘、伍姿亦、陳瑞玫、林嘉凡、黃春舜、楊凱慈、劉博文、林雲卿、洪淑映、李嬌江、李榮松、林語韻、陳秀姍、李琬富、黃江凡、蔡嬌瑜、吳淑惠、陳琇豐、李淑惠、林俊豪、楊少霞、何凱琳、王智翔、黃志忠、方呈燕、黃清鑫、潘佳琳、杜劭福、趙正佳、陳宛康、柯瑜偉、張馨文、劉林光、齊雅芳、張秀雅、田博芷、盧佳蓉、洪富珊、黃韋成、李淑萍、賴月聖、李幸皓、王雅韋、王家豪、葉冠志、楊珮君、趙曜宇、楊彥文、晏明宏、謝淑惠、溫紹欣、林俊逸、錢政霖、顏嘉琪、賴仁豪、王雅芬、夏忠翰、林明哲、洪冠宇、李雅鑫、蔡晏寧、陳志嘉、邱郁皓、利青吉、嚴淑婷、蔡曼虹、蔡家豪、黃國侑、楊哲維、陳盈斌、郭安琪、周偉紋、陳佳蓉、吳向裕、王玉玲、周佳英、吳怡君、彭雅雯、楊文蓮、權台泉、蔣雅如、傅華琦、黃鈺卿、林艾真、王嘉偉、張俊宏、林彥志、李家霖、李琳淑、蘇展揚、劉玉中、郭曉薇、何劭成、丁立偉、黃俊榮、許儒茹、柯萱聿、張康桓、黎嘉偉、盧韻群、鄒佳穎、林志育、何嘉宏、林思茜、吳宜名、李佳謙、曾銘智、李玉玲、謝依松、於竹宸、林長豐、龔展茹、葉懿麟、黃立偉、吳佳亦、陳俊傑、溫人豪、張志旭、張珮韻、吳于啟、鄭學和、劉宛臻、楊智源、張婉瑜、李彥文、江綺清、黃美法、項珮、黃右皓、王秀生、楊仲為、郭俊明、葉義蓁、張筠霞、楊人豪、周宜音、張嘉發、林儀鑫、陳宜明、李易琳、陳惠婷、賴威廷、杜舒蓁、呂建德、郭仁豪、賴邦順、倪茂漢、黃雅惠、陳柏琳、蔡詩婷、陳柏宏、王嘉夫、魏劭妍、何丹燦、劉思穎、陳維倫、林台堯、鍾明瑜、藍智純、詹明杰、林志明、韓林倫、劉怡安、鄭美玲、汪元湖、袁呈政、許美華、魏奕宣、洪怡婷、林伶傑、陳麗茜、賴國維、魏吉萱、張育亞、陳品茜、胡鎮樺、王培綸、楊凱琳、鄭旺玉、邱凱倫、徐育如、朱侑意、倪雅芳、簡婉瑜、謝宗汝、冷廷胤、賴禎春、楊雅惠、吳家一、彭綺智、吳芷仁、許家原、陳韻如、黃冠貴、方靜怡、王雅云、王彥定、王巧中、游賢均、鐘怡婷、李美偉、王智強、陳旻木、張凱芸、喻雅琪、許淑卿、謝淑婷、林玉婷、陳明一、陳傑香、荊羽伯、蔣政枝、鄭岱駿、黃鈺虹、蔡于真、賴逸群、謝淑玲、田士銘、許佩君、施佳慧、陳善木、吳純昌、賴妍山、汪立燦、黃必達、黃毓瑞、李馨文、謝明佳、蕭雅桓、黃鳳珠、鞠峻萍、張裕翔、滑惠婷、劉姿瑋、余彥君、蔣宇翔、宋孟涵、鐘駿士、陳志瑋、徐培泰、汪彥智、陳俊豪、洪家良、王怡婷、黃允聖、林淑玲、李肇琪、李芷禎、劉弘伸、謝昆綺、陳禎雪、謝軍易、王進剛、盧于薇、蔡政英、傅紹淑、林昱廷、張家亦、李宛英、楊惟如、張伊卿、潘隆行、施致遠、翁家偉、彭敏依、汪書琳、謝文柏、黃子姍、王雅慧、桑婉如、楊宗翰、廖怡璇、王素志、李耀彥、宗佩蓉、楊志廷、楊佳茵、郭仰堯、李寶佩、林志芝、黃予昆、陸玉婷、梁芸薇、趙雅芳、陳子軒、葉玄歡、趙詩昇、周志文、楊協豪、吳承雪、黃世傑、吳晉茂、賴聖傑、黃于綸、張火志、吳珮如、曾坤韻、林鴻霖、陳宛易、葉雯來、游興仁、林修汝、張維君、郭怡靜、黎怡婷、周靜康、林惠如、林家恩、蘇尹山、盧順偉、郭明輝、李珊星、方欣法、顏俊谷、和人易、連建文、洪仲劭、張家卿、吳紹豪、徐偉凌、俞崇翰、李佳玲、李淑惠、鄭裕仁、周淑慧、林瑤妮、賴建芝、許光添、陳盈萱、林麗雯、謝肇雪、季淳行、嚴貴昕、林韋美、陳昱喬、盧耿儀、璩倫豐、呂淑慧、謝宛裕、居姿婷、洪偉其、李雅雯、羅嘉季、梁陽祥、郭弘茹、陳瑞君、林君成、李書瑋、邱建瑋、賴美君、楊智泉、張家宏、黃姵蘭、王恭秋、王佳穎、陳金茜、藍明儒、黃怡行、劉育廷、楊舒惟、姜苡一、陳宗竹、柯冠霖、韓穎源、黃育萱、林家光、王偉慈、吳聖枝、蔡佩辰、耿婷婷、李子揚、劉國洋、鄭奎萍、李成珠、張惠婷、黃郁峰、楊冠宏、陳志銘、李石南、柳佳玲、鄭韋伶、陳敏毓、郭欣平、暴守寶、吳博鈞、庾宛君、吳孝青、吳佩威、白志祥、呂語英、林嘉祥、艾賢廷、劉南合、張耀松、米弘紹、劉恆航、杜美旭、秦俊穎、李姿政、王筱臻、江佳洋、張淑娟、倪琬琇、李柏霖、高鈞凡、馮秀美、張耿玄、韓德妏、江忠慈、陳恆瑄、林芳儀、林俊行、曾皓東、林雅琳、于淑惠、沈雅雯、蔡琦銘、駱勳屏、李致隆、蘇柏翔、陳俊福、林聖文、宋婕妤、劉芷芝、費仲勇、廖珍堅、袁佳雯、陳奎倫、毛京以、彭青修、葉信宏、劉俊彥、張庭法、高至玉、張秀富、張舜山、王慧玲、陳曼一、彭慧君、郭紀妤、王山喬、張淑天、楊家泉、吳明杰、楊貴瑋、吳文東、王于真、林典茂、楊卓廷、林淑州、徐劭誠、余蕙蘋、宋曉萍、王皇玄、劉佩蓉、陳君豪、王賢恭、彭筱涵、張明男、吳政源、張俊銘、馬鎮宇、陳宜恩、賴華治、王萬恒、何為意、蘇于倫、陳宗名、蔡少亞、鄭淑玲、吳火添、王宛蓉、林雅婷、林惠玟、蔡偉智、周重侑、張瑤妤、洪偉愛、邊彥君、傅書琬、林君豪、謝夢佩、陳世偉、周怡慧、李雯育、郭淑念、趙麗娟、黃文然、黃鈺雯、葉欣宜、林偉誠、王淑谷、沈淑惠、林怡樺、郭燦卉、宋哲豪、吳俊正、廖彥博、張雅婷、李雅惠、王皓竹、楊穎發、王淑一、傅鈺憲、陳怡如、陳姿瑩、黃欣宜、楊孟哲、謝依城、張靜豪、蔡政貴、張明彥、劉家榮、廖俊霖、王宜欣、陳俊男、馮朝安、潘怡婷、何靜怡、張郁涵、費佩樺、吳政雲、陸海卉、陳玉均、戴靜怡、羅佩君、盧文欣、陳建宇、林佳蓉、黃美惠、酈意盈、王昱明、楊瑋倫、郭呈孝、林昱福、張淑君、陳柏怡、胡璇蓉、江佳康、陳慧齊、張佳玲、葉季火、黃雅嵐、陳春蘭、陳素貞、劉奕廷、劉于真、蔡名行、倪博仁、許怡文、劉天玟、曾曉雯、周志銘、林慧妃、庾俊毅、蔡宗育、李美亨、楊志文、蔡行竹、唐尹昇、杜成玫、王彥均、陳靜隆、張玉恭、劉柏宏、王淑珍、白佳慧、陳哲榮、張雅鈴、杜靜怡、林鎮延、李瓊慧、潘珊歡、林琪諭、陳馥玟、鄭先佳、李耀德、陳雅竹、謝秀娟、黃木綠、梁家榮、賴玉玲、謝沛宸、鄭子軒、陳文凌、陳心政、方雅雯、趙威定、蔡文凌、楊淑媛、夏哲枝、潘凡嘉、曾淑郁、張睿星、李鎮宇、鐘婷婷、吳孟琦、詹伊倩、林慧萍、潘介茹、黃惠盛、連偉州、楊淑一、陳麗茹、蔡明瑄、洪嘉恩、毛承穎、王玉雲、李辰利、吳家維、黃軒豪、郭心怡、楊雅琪、陳嘉峰、翁金鳳、黃郁光、林郁瑜、賴怡芳、張俊賢、勞淑雄、楊瓊孜、曹姿蓉、吳怡文、賴心怡、羅淑惠、陳秋冰、湯智卿、賴怡誠、林智傑、李明文、姚珊良、林淑美、林小妏、黃兆雄、陳昭政、曹致遠、許冠宇、高建志、施淑芬、彭彥翔、李俊賢、宋政儒、陳光水、游欣怡、蕭千慧、潘佩琪、楊鈞修、陳佩璇、劉美俊、張雅慧、宋惠珍、宋俊彥、陳文琦、劉雅婷、黃珮瑜、黃怡君、戴卓瑄、陳琦文、夏愛婷、黃湘婷、陳志天、鄧怡臻、黃珮君、劉平蓮、錢子揚、宋亞為、張政哲、陳淑航、林建宇、王毅信、張書新、陳志丞、杜宛玲、張家紹、楊婉芝、陳堯儀、黃瓊正、趙依婷、陳佳穎、何郁馨、林俊彥、蔡文彥、黃涵鑫、吳懿岑、李裕合、鐘怡潔、陳翔花、鄭伊松、林俊佑、王予義、傅凱鈞、李嘉惠、趙曉萍、高姵淑、趙雅文、葉幼群、許智梅、黎秉勳、蕭雅念、鄭俐喬、胡佳純、李安雪、陳智偉、江仲堅、陳姿穎、林亦齊、黃昱廷、張冠秀、冷文宏、孫俊賢、李至威、趙智超、黃慧娟、吳蘭歡、趙珮珊、楊子翔、金瑜龍、趙志豪、林靜紫、黃智宸、韓泓偉、翁冠宇、許凱軍、鄧玫均、張丹來、邱常福、胥麗君、李必麟、林冠元、張雅琪、林威任、王聖傑、劉建宏、黃紋麟、賴儀明、王冠廷、盧冰法、楊曉薇、方建緯、朱志賢、鄒忠然、王威任、黃家宏、倪子傑、閔木明、王郁婷、戈旻花、陳孟仁、林宜蓁、王致玫、方瑩秀、周雅娟、侯俊豪、張劭儒、吳欣湖、吳幼宇、吳品寧、林慧萍、鄭恒伶、李璇天、車志賢、戴玉婷、張郁雯、柯子揚、柯朝如、賴群冰、林佩君、黎豪嬌、林子財、黃偉恬、陳俞斌、岑瑞信、陳向禮、林雅慧、王宗翰、陳雅如、李佐珮、藍光元、李韻如、施慧娟、蔡冠傑、高財彬、陳賢恆、楊淑芬、曾筠方、王欣潔、宮婉婷、王麗芷、郎宜彥、許平玄、林行人、李芳如、李宛竹、黃莉斌、白曉雯、黃曼成、丁登郁、呂文馨、鄭美慧、李大鈞、劉孝心、曾聖文、林佳慧、汪雅慧、高毅宸、郭紫蓁、黃吉倩、彭志軒、王郁珊、柳怡欣、吳孟筠、蔡振瑋、顏俊瑋、吳承劭、吳佳蓉、黃坤志、王郁宣、周怡明、楊俞歡、林怡紫、黃偉智、謝依夫、陳俊修、韓惠雯、張潔蓮、程姿妤、江海茵、劉和治、楊冠志、黃正全、何英仁、陳宇聖、黃愛亦、楊凱婷、宋武歡、陳宏正、黃白夫、謝志峰、張俊德、張鴻昌、楊書秀、廖婉書、阮建彰、李淑華、張育茹、吳麗忠、李玲仰、丁意水、林志利、楊佳勇、葉家豪、林剛玉、方凱映、林文城、陳吉能、姜建宇、吳學正、邱侑佳、柳振瑋、陳欣香、朱郁婷、劉佳元、朱雅馨、陳意啟、孫怡君、王志豪、陳佩凌、陳瑋婷、王書瑋、童志文、曾佳純、蘇曉裕、馮伯育、鄧淑玲、郭恒元、葉慧茹、許明哲、吳淑敏、黃茜哲、馬雅雯、尚淑敏、李政真、吳惠雄、曾筱婷、尤家豪、郭姿蓉、周佩君、郭裕盛、吳俊蓮、黃孟倫、劉宜靜、林哲榮、林奇邦、葉佩如、林慧君、鄭苡姍、李俊賢、楊語凡、陳惠偉、蘇淑萍、李哲豪、張育雅、孫國華、陳景竹、李宗音、陳隆民、趙俊男、馮亞瑤、林世季、吳柏豪、吳志祥、楊郁宏、陳聖凱、張木林、許依潔、倪惠愛、胡英宜、賴建豪、陳君生、李孟君、彭恩齊、耿盈君、金隆廷、彭元華、劉健豪、劉朝強、賴耀祥、陳台韋、張坤順、陳君宣、李怡宜、林琬延、楊坤季、袁家榮、錢姿蓉、張俊男、張馥穎、周仰邦、林凱鈞、陳威人、蔡淑娟、顏琪均、吳佳雯、柳慧玲、陳儒泰、孫依婷、陳則學、李千穎、陳嘉玫、呂佳均、許美娟、柏怡伶、吳怡君、曾佩君、張宣歡、林宏海、王湖泉、吳珮辰、駱聖菁、郭麗君、劉培佳、林筱帆、陳文婷、盧韋吟、宋信宏、吳啟蓉、阮佳蓉、陳力妮、陳欣秋、陳俊傑、李彥宏、危慧敏、陳舒婷、曹佳燕、白婉希、周佳慧、林義琴、林秉其、李依仁、張千剛、林政儒、王靜宜、鐘堅謙、林育雄、張欣儀、陳忠綸、陳軍秀、王成勳、張志賢、謝禮甫、陳永琇、章慧璇、林振月、陳正育、胡傑珠、黃鈺淑、王信宏、白展賢、林麗白、黃與雲、蕭少歡、吳天意、鄭宇純、羅俊毅、林正偉、張向桂、曾于年、賴佳伶、王維芃、王韋成、劉炳惟、黃耀芝、吳淑霖、潘偉銘、毛嘉雯、倪怡伶、顏平亮、洪淑芳、蘇晉怡、陳宥如、許宛蓉、侯秀美、陳夙芸、陳淑真、陳佳玲、翁俊憲、張子翔、陳晏男、劉恒宣、郭俊穎、李富瑞、吳慧順、李祐誠、蔡昀佐、張茂善、王宗達、周倩誠、劉子蓮、方怡菁、林立裕、謝萱雅、傅彥翔、黃石方、宋恒柔、林吟康、陳逸妹、謝明慧、吳明憲、陳喬水、侯台吉、侯力正、吳仁傑、葉一佐、陳美靖、陳燦伸、馮盈君、蔡家宏、林雅晴、李嘉琪、夏詩青、謝明惠、江彥智、陳民雲、王均依、吳儀芷、楊丹琪、陳敬育、陳怡婷、洪佩穎、陳姵君、王勳生、張明君、劉玟禎、林彥亞、鄧凱鈞、郭智翔、盧怡璇、許雅琪、黃維容、張耀中、孫學航、蕭政霖、吳秉琬、羅珮瑜、涂傑臻、林麗新、鬱姵惠、歐陽婉婷、許欣怡、張至茵、吳珮杰、張凱麟、陳婉如、邱智翔、儲芳瑜、林子香、高尹添、黃世怡、劉俊豪、林政雪、劉浩典、謝瑋玲、陳思泉、張書元、謝欣東、黃佳欣、周志萱、盧初安、陳育誠、蒙智堯、陳乃英、黃柏宇、林廷琴、胡怡傑、洪志傑、黃勳菁、林筱涵、馮金鳳、賴宇翔、宋孟勳、吳靜汝、林韻如、陳雅惠、陳念吉、陳佩璇、李健偉、蔡涵甫、彭旻雄、楊韋珍、陳慧強、賴姿禮、李逸原、劉佳穎、甄家祥、鄭欣怡、郭勝雪、林祖英、楊姿吟、錢雅雯、高琳玲、黃雅慧、陳潔茜、陳柏民、陳白淑、吳怡伶、黃文傑、林逸紫、賴淑萍、黃雅羽、陳宜仁、沈曉萍、王仰卿、劉怡白、張嘉毓、吳郁翔、李欣怡、李雅娟、焦智超、林旺念、謝歡諭、黃佳燕、林宗毅、吳美倩、王雅琪、謝宗翰、邱淑娟、葉法璇、吳博鈞、林士易、林水臻、郭昭維、謝平恩、彭琬婷、白惠如、袁白峰、陳山岳、柳常生、郭儀尹、陳玉萍、夏儒方、林友皓、蔡柏勳、林惠雯、王雅雯、王文婷、陳宜奇、潘佩珊、鄒敏郁、禹宗憲、賀富英、馮哲嘉、林鈺峰、張佳蓉、吳冠中、黃國信、溫彥璋、黃郁文、葉茜綺、彭惠君、趙以龍、張函惟、蔡孟勳、陳奕妃、潘力泉、林育霖、張瑋婷、藍秉勳、邱思婷、陳國瑄、彭允凌、賴琦心、黎儒蕙、程雅惠、高紹佩、陳慧君、林婷純、詹雅姍、江宛玲、蔡威廷、張琦、張志隆、曾嘉玲、楊皓花、王怡馨、方世娥、劉婷婷、陳威能、方建宏、楊平榮、林正茵、蕭宏達、林靜珊、陳怡如、王郁婷、席初琬、張如恬、周常白、林冠宏、陳欣怡、余美惠、曾泓枝、林佳燕、陳治達、鮑劭伸、高佳霖、潘怡如、林亦賢、張志苓、張貴睿、陳莉婷、徐宜君、吳美玫、陳珮芝、羅哲蓮、曾卉吟、黃家孜、吳玫蘭、周丹愛、陳季春、蕭彥儒、楊宗瑋、曾俞元、洪政廷、林兆修、潘姿貴、吳麗娟、王仕均、林育誠、林冠霖、彭侑文、黃瓊迪、邱聖軍、李韋康、黃嘉昆、陳正霖、林牧琪、葉燕哲、黃勝汝、郭武鈺、蔡詩婷、李正漢、鐘于婷、陳雅文、李怡靜、張沛群、陳志谷、林興翰、吳雅劭、陳雨瑜、金智凱、陳宗穎、袁玉婷、吳湖玄、溫玟江、王詩柏、李威皓、鄧靜怡、陳文芷、張木雯、賴雅嵐、王政儒、邱小茵、林佳宏、陳介水、林茂義、張隆豪、黃惠任、徐聖仰、林士合、陳文仲、林筱桓、黃佑竹、姚佩君、陳信合、謝佩齊、林馨文、洪宗德、黃大人、王淑娟、黃明宸、方亞欣、姚妙惠、李建宇、陳文民、葉少云、陳國月、蔣振凱、張亮瑋、王宜君、吳政妤、李培倫、陳建勳、吳佩士、劉雅玲、王向邦、吳凱鈞、徐宛臻、林郁菁、黃淑舜、崔益菁、黃筠蕙、林盈泉、趙雯水、藍宜蓁、葉軍芷、周之然、林秀妹、卓婷婷、許智妏、蕭婷勳、林愛元、彭志軒、黃承能、毛韋成、陳玟宇、郭凱婷、黃士英、朱柔淳、邱筱婷、陳淑惠、吳彥志、孫坤泰、辛素旺、洪皓全、許柏成、陳玟凡、吳綺茜、陳莉雯、賴嘉祥、李傑凱、張珮君、劉雅霖、林良仲、林佩瑩、陳琦韋、陳燦一、陳淑成、吳心怡、黃文彬、張士哲、張碧定、張依婷、周明翰、劉大姍、王惠靖、陳昱福、劉俊傑、王昱樺、陸淑珍、林傑善、林馥霞、陳信岳、林家豪、姜安倩、蔡佳劭、張惠政、張惠萍、柳婉佐、陳昱山、蔡思潔、林佐昕、張奎珮、李美淳、謝士哲、葉淑萍、黃伶民、蘇建安、游英盈、簡純善、周姿妤、王牧志、彭志遠、李志祥、吳政霖、陳宛城、劉雅萍、蔡嘉迪、杜靜宜、連寧娟、夏家維、劉英均、曹艾心、黃佳怡、陳星以、藍至火、趙邦花、黃偉治、劉怡芬、張予齊、胡世偉、陳偉倫、林石如、黃偉傑、賴蓉娥、李威蘋、袁為祥、柳宜婷、陳家岳、蔡逸凡、黃婉君、吳佩穎、陳佳霖、鬱詠胤、張曉萍、黃于鑫、杜文良、陳彥法、馬淑惠、溫冠勳、陳麗瑋、沈亞宜、楊士豪、楊恩翰、林白霞、夏麗娟、陳玉梅、耿良博、連晉文、林淑貞、謝佳穎、陳佩璇、吳智凱、葉芳奇、林怡芷、陳志強、張淑婷、簡峻樺、胡雅玲、王嬌來、黃耀谷、王美辰、王佩豐、張玉菁、陳柏皓、全建志、吳明祥、許詩雄、王家弘、黃惠茹、謝美玲、韓嘉偉、陳怡秀、陳欣怡、洪昶玲、黎佳怡、蔡承香、蔡元惟、齊聖文、吳良瑩、鄭詩婷、鄭雅希、陳淑君、童思芝、簡敏隆、楊志秀、黃宗翰、劉志季、駱雅惠、廖妍辰、顏素凌、馬美玲、余韋成、陳宥凡、林冠紫、黃俊桓、楊宜愛、連淑貞、劉雅雯、陳若蘭、陳志光、陳俊貴、王世欣、林慧萍、何懿芃、梁長孝、沈于真、吳明海、周建宏、姜淑惠、林力妃、蔣阿穎、李宛雨、吳亮弘、張菁瑜、許怡孜、方麗娟、黃茹玉、謝鈺婷、楊孟儒、鄭宇海、吳欣強、郭冠宏、林香君、蔡之利、劉嘉英、李鳳雄、趙良典、富威任、黃貞樺、陳致遠、許淑能、賴春原、傅昌全、陳祥貴、李建緯、張聿方、林定一、楊毓玫、李惠雯、賴宛臻、陳雅玲、林筱婷、謝石仰、謝淑芳、李怡陽、張姿蓉、麻寧禾、高佳霖、李宜秀、李馨儀、胡思州、陳世豪、陳欣潔、夏育瑋、吳慧萍、謝德佳、楊雅侑、楊致遠、謝筱毓、李彥宇、陳馨易、葉順芬、冷建凡、胡威德、黃俊珠、潘曉萍、黃意帆、劉貞瑋、陳嘉琪、張芝民、杜敬珮、孫堅添、葉寧沛、陳振城、賴士辛、林為泉、陳鈺誠、貢少皓、黎佩儒、蔡國謙、郭佳霖、陳雅雯、劉豪剛、賴威任、陳韻然、林彥妹、蕭睿江、張牧璇、蕭心歡、王姿妤、陳佩儒、王揚仁、陳瑩元、林鈺辰、陳紹木、吳佩玲、林英傑、蔡浩茹、關政勳、劉軒谷、趙耀中、詹萱恒、李佳燕、洪堯隆、李佩書、陳宜東、張皓康、祁初火、謝家豪、蕭建安、吳湖鳳、蔡淑貞、林佩容、魏怡君、林宗財、洪如昌、謝軒豪、袁欣怡、李文發、吳慧善、戴冠霖、陳思蓉、吳純月、白于真、吳明憲、馬鈺婷、黃俊宏、李耀德、李夢利、陳正欣、魏靜怡、賴琦淑、孫筱涵、鄭瑋婷、洪佩璇、蔡于云、陳書瑋、黃靖怡、鄧佳儀、陳美娟、張志豪、鄭怡伶、劉惠婷、黃雅柏、昌意婷、蔡雅迪、李宜凱、陳思謙、黃宏儒、吳則恆、丁潔聿、李建宇、林慧玲、李佳男、李孟一、夏柏豪、劉純智、朱君豪、陳政孜、柳沛賢、張志昆、李珮君、林欣瑜、蔡雅和、陳珈陽、蔡俊傑、杜建彰、翁俊達、林建洋、陳奎菱、邱郁婷、張尚泰、王信宏、徐香維、蕭雅廷、劉健仁、張怡君、陳萬為、吳美君、楊文峰、黃柔賢、林淑萍、蔡若映、陳光興、陳登湖、蔡宜季、趙樂霞、陳康桂、劉仁杰、楊成恬、溫冠勳、林育盛、黃郁正、賴淑惠、王鈞忠、張玉妃、金菁亦、盧慧君、林士利、黃美音、胡湘岑、賴賢漢、王子倩、呂佳瑋、陳淑娟、蔡文郁、陳定善、林立人、賴玉穎、高與恭、方卓成、黃瑋木、黃崇諭、黃琬汝、張哲州、王玉迪、陳忠恩、楊雅玲、陳隆俊、張欣諭、林卉春、楊承穎、陸瑤季、葉函妹、李致蘋、陳怡禎、廖婉良、馬妍瑋、李俊禾、張傑春、王淑敏、賴惠雯、陳昭均、劉佳志、謝義木、林綠凌、蔡沛柔、黃孟璇、林冠辰、高凱婷、陳文傑、傅怡君、陳柏勳、張昌南、白智翔、張維哲、吳其蓉、王仁芝、周惠玲、方士緯、陳慧輝、黃嘉玲、陳志遠、黃薇明、馬俊銘、林欣智、吳宗財、吳怡松、陳信筠、宋雅雯、韓秀慧、陳思良、張書芳、趙欣怡、陳思毓、王堯鳳、王怡欣、劉江怡、鄭晉松、牛珮歡、李玟玄、劉俊德、鄭靜柏、謝白為、黃偉合、王家銘、林文欣、馮淑玲、陳瑩筠、梁士傑、張舒涵、陳家年、邱怡君、邱依潔、汪品弘、林瑋奇、周彥廷、刁思亞、林子怡、陳憲韋、梁振豪、傅夙寶、邱佳慧、杜郁婷、蔡志宇、蘇芸夢、余惠文、賴政儒、黃靜新、王天定、江淑玲、張修春、姜俊嘉、王鈺玄、蘇佳玲、蔡冠憲、連美丹、李嘉智、林俞彬、李思彥、李亦清、謝舒夢、劉素貞、徐莉恭、林美弘、郭宇燦、吳哲維、李綺貞、李慧禾、唐愛揚、張信宏、張智雅、林俐貞、許嘉惟、陳盈甄、陳奕雅、陳雅雯、李美湖、陳君惟、李俊宏、張心怡、楊志維、簡坤伸、邵淑芬、陳政邦、彭盈苓、林蕙玫、房旻慧、蘇曉薇、張彥文、楊卉勳、陳法坤、黃必茜、杜丹辛、林佳瑄、陳玟凌、鄧淑華、陳宜彬、蘇玫意、陳博琬、袁德蘋、陳家賢、黃愛德、黃宗儒、馮淑芬、溫辰甫、蘇宜潔、吳賢英、王子祥、翁映珠、毛財友、趙美惠、葉怡君、黃瑋玲、陳詩涵、吳俊彥、莊品啟、林立偉、扈佳郁、陳春博、莊惠傑、陳琇琦、王怡陽、黃怡萍、王俊依、白守伶、鄭秀珍、陳右聿、莊雅佳、張惠茹、林婉玲、周倍瑤、李國哲、陳儀書、王皇南、李明以、陳麗岑、杜聖芷、陳佳孝、涂詩雅、洪星良、林士哲、徐盈君、範軒仰、駱宣儒、黃思意、曹佳雯、黃韻如、李凡華、陳詩軍、洪俊憲、李柏元、陳佐韻、張韻、賴柏臻、陳政宏、黃智筠、張樺、蘇仁豪、許雅嵐、黎佳穎、陳威廷、陳岱諭、洪學綠、潘吟延、仇紀淑、邱先凌、歐陽韋伶、王盛嬌、謝欣蓮、溫永民、李俊宏、葉仁杰、陳勇學、賴仁傑、林信宏、曹昭宜、吳慧來、謝威廷、黃雅芳、廖婉妤、張家宜、白曉宇、彭士文、林嘉聿、吳嘉皓、林美泉、張與惠、林秀亨、婁家豪、張茹仰、劉凱倫、王嘉茂、陳郁峰、謝晴靖、王郁心、韓燕丹、孫松貞、林佳書、葉慧萍、羅韋成、蔡家良、林雅惠、蔡智仁、鐘嘉鴻、郭榮毓、王家邦、李秀靖、陳怡樺、黃詩筠、林慧恒、高家銘、麻佳雯、陳立生、李怡凡、吳常蘭、劉淑娟、曹巧齊、吳玉婷、李映冰、鄭筱婷、林百峰、洪偉月、周人豪、陳信宏、黃宜甫、王玄昌、陳火凱、林子軒、李儒沛、林芷昕、蕭詩亦、楊欣宜、盧雲俊、許韋志、許秀慧、游翊卉、賴依婷、王佩樺、林佩臻、葉冠宇、陳行輝、林祥宸、黃翰冰、張志秋、陳麗美、陳丞昌、毛淑娟、翁家慧、洪淑原、鄭夢枝、林志綸、楊嘉祥、高吉海、吳可良、許惠君、唐朝輝、林欣怡、陳上誠、熊志豪、陳原珮、陳雅湖、黃佳怡、黃協念、趙喬禾、袁宗翰、呂士凱、林和淑、萬名妏、陳國堅、李孟傑、陳韻義、吳容紫、鐘慧敏、黃安琪、張秀芸、陳佑佳、黃心輝、林佳均、張丹真、黃雯如、岑羿華、林佩盈、鄭曉薇、鐘信豪、梁心怡、曹宛虹、王偉銘、曾建豪、劉庭發、徐嘉信、李明諭、林芳成、陳雅文、姚嘉英、吳莉星、王姿吟、許家祥、楊俊憲、駱雅琪、王哲豪、蕭志傑、李佩依、張志宏、戴揚維、祖哲瑋、楊淑行、方盈君、張世傑、楊慧士、吳典添、施大映、毛毓睿、周曼清、鄭雅其、梁詩涵、梁玉美、李淑芳、游玉鳳、林盈如、謝明行、劉怡婷、林佳真、劉冠宇、許怡如、洪怡君、劉政峰、蕭雅梅、童宛玲、賴怡東、白家齊、鄧孟修、陳孟吟、趙志明、蔣元洋、許哲豪、陳雅婷、蔡欣音、溫家豪、周佳偉、蘇美君、陳志虹、鄭初憲、周佳紫、游智翔、冷琬憲、陳逸枝、張登陽、陳嘉玲、衛惟馨、陳立年、蔣世偉、廖雅惠、林曉韋、楊雅婷、羅淳季、陳寧鴻、王晏凱、林靜怡、劉佳豪、吳思昆、袁鳳邦、金玉玲、丁俊嘉、黃志松、楊智定、楊書豪、王玫和、黎雅玲、阮永民、蔡廷善、曾佩如、傅禮玫、楊惠裕、王聖瑄、蔡佳名、李雅萍、陳向宸、賴彥均、陳文汝、彭佩君、呂雯婷、簡心怡、陳芝智、張映尹、謝薇玟、袁俊傑、郭建志、鄭正偉、錢素仁、李宜盈、郭淑恩、王耀文、劉肇定、翁奇珮、吳珮瑜、林士州、鄧小芬、吳宛玲、李月士、沈鳳瑄、張辛誠、林明輝、蘇玉鳳、吳美珊、姜心怡、林妍桂、林昆文、曹茜萱、張羽梅、王瓊孝、孫嬌慧、趙筱婷、高詩婷、吳文俊、王秀娟、李欣宜、林秋燕、葉哲瑋、陳偉陽、林俊毅、吳志賢、楊欣怡、廖婉竹、楊皓菱、丁靖音、李婉如、楊哲維、謝佩政、夏琬珊、賴明宏、李玉剛、沈清珠、劉家辛、鄭培奇、李承玟、王明郁、張耀劭、姚希凌、蔡雅雯、黃曜宇、陳麗昆、李欣姍、楊雅幸、王佳蓉、蔡琳香、周湖音、丁欣怡、徐怡伶、林宗達、王卓睿、李育易、藍文凱、陳美君、朱佳儀、林士盛、周浩為、郭怡如、張智傑、李淑娟、侯威德、朱華瑄、林雅婷、馬莉婷、黃筱萍、張威宇、張玉華、杜佳玲、王茂恒、許昇恩、林柏豪、潘麗珠、鄧勇志、丁靜怡、張詠虹、林怡珊、李建銘、蘇慧娟、張雅婷、胡怡婷、沙仁珮、林奇航、柯淑真、黃孟倫、李泰發、戴韻如、吳明奇、張惠伶、蔡玉梅、李廷倩、邱孟修、李雅君、方彥博、林宗能、王佩玲、李方旭、黃婉伸、陳杰翰、王建美、許勳豐、曹珮甄、劉淑貞、吳玉邦、周文華、陳婷任、蔣欣瑜、陳玉萍、吳俊安、王怡雯、王怡婷、黃淑慧、張孟倫、王玉韋、張馨珮、余白旭、黃勝傑、賴英傑、扈嘉鴻、林世竹、劉昕菁、張育源、王馨文、丁常馨、張冰苓、李婉菁、吳威廷、紀名琇、劉法彬、李志宏、吳佳和、羅景為、劉承恩、吳俊榮、蔡建勳、陳家欣、張俊宏、吳仁杰、王淑芬、林昕汝、姜筱雯、陳佳盈、王岳春、林祥邦、楊培倫、藍光勳、孫素念、竇智偉、張佐迪、張文馨、陳紋修、賴豪玄、虞紹山、柯左博、洪佩儒、張維全、黃政憲、趙怡君、陳志偉、劉友昌、黃建倩、林淑娥、林淑谷、羅育霖、黃先卿、黃妍欣、林立華、楊詩婷、蔡明昆、王智鑫、劉姿君、許雅念、林靜名、孫怡靜、林雅婷、溥雅琳、鄭聖苓、楊志宇、白明宏、蔡偉婷、許家舜、吳耿堯、成靖雯、闕睿桂、潘建文、陳玉義、涂建中、顏盛任、顏淑玲、邱靖元、陳亦年、鄧致遠、吳孟安、錢佳伶、林柏玉、沈怡靜、林淑傑、林彥青、洪冠中、王法啟、陳芃昌、朱立偉、錢姿妤、李泰名、黃紀羽、陳正航、魏惠婷、陳倍林、毛善希、楊珮豪、蔡阿昕、汪宇翔、任初群、祝仁男、林信琪、陳佑菱、林欣哲、宋旺修、羅嘉偉、李美昆、伍坤能、陳韻安、游筱婷、黃惠惠、劉健瑩、謝文欣、黃尹湖、姚昭函、劉軍樺、劉淑真、邱與娥、林婉婷、王淑惠、李宗毅、李珮香、林尹劭、陳毅淳、吳行、李雅凡、陳欣怡、李政儒、張家一、邱延憲、謝雅惠、盧昌善、金喬昇、夏昱宏、林怡君、吳靖芃、曾寧海、王玉婷、石佑霖、黎婉婷、張千均、汪旻水、陳志成、黃琇平、黃雅婷、白佩璇、李群星、張馨峰、黃昇芝、陳彥青、陳佳枝、黃彥君、柳怡君、呂佳玲、林珈發、張惠雯、張博其、謝木汝、楊宜臻、仇寧以、張雲強、王彥安、林筱其、黃俊賢、王怡如、鄭淑慧、袁奕君、張心凌、姜嘉玲、顏建豪、陳佳君、黃詩昆、陳美吉、王浩諭、李琇臻、周力昀、鄭軍為、彭宏偉、吳宗毅、林育宜、王淑玲、傅德鴻、吳俊榮、張惠孜、馮俊賢、蕭綠恩、郭淑威、姜玄珠、陳雅婷、陳俊瑜、韓美惠、林玉斌、李承翰、黃慧輝、陳志文、李志豪、劉苡名、周銘璇、曹緯白、陳信瑜、陳宏福、黃于婷、廖蕙瑋、趙哲宇、林哲月、劉杰純、林俊毅、陳瑜昆、石佳瑩、吳靜誠、王明輝、王美惠、李耀文、寇雅雯、李逸凡、陳雅菱、劉育桂、陳傑菱、彭凱琇、顏芷岳、宋家昆、彭祖容、童怡禾、林佳慧、曾峻韻、謝庭瑋、劉靜怡、蔡仁杰、左郁婷、黃淑娟、顏德綺、李玄盛、蔡惠萍、房天依、陳惠雯、張文傑、石宗璇、金佩珊、陳信冰、胡永法、楊耀德、周哲維、陳瑞禾、酆國偉、邱敏汝、林芷湖、韓軍州、劉舒星、陳勳湖、鄭玫坤、趙茹儒、王玉婷、楊昕彥、童麗娟、劉有月、謝興姍、王紋書、陳景翔、張政勳、林信宏、陳怡雯、李珮甄、劉鈺達、林秀天、鄭怡婷、蔡佳樺、蘇志豪、呂必政、馮慧玲、高禹海、蘇義漢、林典玟、林素伯、余彥霖、周秀玲、林佩智、李品心、孫宏達、阮雅惠、李士凱、胡志維、秦怡君、郭雅茹、鐘淑芬、黃玉東、鍾明雯、趙旻美、劉玉鳳、楊姵君、陳香君、陸祥音、洪嘉韋、曾玄胤、許靜宜、邱靜怡、王惠如、蔡佳靜、杜佳雯、陳嘉吟、黎禮平、鄭玉婷、鐘佳怡、阮琪妮、張志偉、李旺花、周人娥、張文彬、蔡家齊、劉家華、趙友松、冷雅惠、鄭宇忠、張淑娟、陳秋妤、楊晴伯、林依婷、林依萍、陳姿松、江淑珍、林士東、韓誠依、黃于友、龐瑩康、潘倍仰、林佳奇、平振豪、簡科鈺、葉俊賢、馬聖文、林易佩、張美辰、倪延中、溫英航、陳彥志、陳初仲、黃麗卿、王辰貴、吳玫儒、陳姿辰、邱裕翔、陳怡福、林彥君、焦雅竹、潘炳青、吳呈宣、黃郁辰、朱宜君、尤慧全、陳揚合、蔡庭邦、張毅新、陳筱恭、林之亦、馮學倫、黃麗倩、黃婷婷、李宗迪、吳雅云、林馨文、杜朝璇、賴怡安、陳景翔、劉菁季、王欣儀、王郁翔、劉玉松、趙苡丹、張銘仲、王志賢、張馨蓉、姜志文、黃志文、賴安青、黃建菁、王隆剛、謝智偉、戴至岳、蔡貞儀、張安海、韓靖南、黃維倫、陳佳穎、黃盈榮、吳玟萱、張山幸、黃豐伸、張建宏、黃俞其、童喜寧、林瑋書、賴建發、陳憶順、林莉婷、陳裕仁、羅薇亦、徐嘉東、葉怡慧、陳英啟、李尹嘉、王曼侑、張靖雯、湯榮妏、李珮瑜、馬雅惠、劉文恭、李怡如、索禹妏、劉子旺、顏有瑞、吳彥侑、常宏偉、李承恩、蘇奇嘉、張雅琳、顏文凱、陳國旭、吳俊禾、陳佩玲、蔡宜仁、林玟劭、黃信美、黃淑華、蔡政福、鄭朝男、錢孟涵、陳慧君、梅亮泰、杜惠儀、李聿仲、陳淑玲、葉翔霞、黃慧生、吳嘉鴻、賴怡如、張維堅、林和仲、吳思盈、李育湖、涂筱妃、林雅婷、蘇又芬、王盈慧、陳琬天、張禎甫、陳幼香、蔡雅原、王雅玲、王柏翔、曹羿茜、蔡林修、楊成年、阮雅婷、吳岳鴻、劉禮宣、林啟淳、陳宗翰、陳真潔、謝明旭、林俊嘉、胡向雲、楊惠淑、夏德然、吳玫菱、陳宏星、陳瓊峰、李靜如、張喬香、邱思穎、戴德興、謝淑惠、郭雅琴、耿軒信、陳裕宣、李建宏、施世昌、曾育儒、陳尹宣、連宜珊、李榮星、趙薇孜、陳淑娟、吳淑原、李幼年、陳彥雲、朱良利、許喜以、謝豪夢、劉美珠、陳欣怡、黃鈺珠、張玲春、張如辛、謝佩郁、陳映剛、朱宗翰、黃俊信、廖珮音、黃孟美、張淑華、潘佩春、劉惠玲、郭培郁、宋庭皓、黃雅雯、謝子妤、陳昭宸、張志明、林淑勇、林昆凡、程志偉、林家豪、何佳燕、傅牧雅、楊淑華、曹淑惠、郭岳翰、曹靜怡、林曉軍、王忠燦、張志文、黃萱宸、黃志友、黃孝枝、劉雅琳、吳啟豐、陳秀淳、李家瑜、溫佳怡、林君豪、宋玉婷、陳淑婷、杜宜珊、嚴俊男、黃俊廷、王俊傑、彭燦蘋、陳雅婷、韓佳蓉、陳郁婷、王家偉、沈家佑、蘇依婷、張瑞谷、張雅珊、許玉玲、劉泓皓、陳浩冰、楊孟音、蔡晏紹、王念辛、羅芳瑜、黃信虹、金妍隆、顏牧民、俞義虹、李威順、黃雅冰、李峻新、王佳霖、楊為彥、潘方堯、蘇明翰、王行琦、蔡欣盛、曾翊雅、王俊紫、權忠駿、胡昕均、王安白、吳佳坤、李怡瑜、謝明卿、伏鳳珠、陳治正、黎淑慧、黃金雲、陳嘉一、王淑俐、郁美君、陳志銘、吳雅雯、王政睿、龔秀玲、李美玲、黃心真、江易季、方祐綺、劉明軒、蔡慧萍、李庭翰、王郁謙、趙倫和、黃佳亨、鄭建宏、陳柏翰、李佩芳、李家維、黃怡君、劉家瑋、楊淑君、楊興行、陳智麟、張彥君、胡容綸、馬建瑋、洪雅惠、蘇興發、陳慧純、林孟海、吳昆孝、陳卉剛、黃思妃、賴哲瑋、江采亦、趙伯恩、王家銘、李志蘋、袁彥儒、劉岱合、張文夫、林姵君、李思隆、王哲嘉、趙培勳、王彥芝、李宜民、鄭哲豪、李依汝、潘新安、胡維林、吳美雪、劉子君、池育霖、洪淑玫、謝俞良、戴孟君、錢倩純、白妙堅、許艾英、鄭致遠、林柏鈞、李淑美、連良琬、鬱誠愛、許育珠、吳蘭珊、陳木妍、賴郁文、羅淑美、秦宜蓉、羅秋苓、林欣芳、陳宜君、王勝傑、蔡孟智、趙國偉、周吟尹、高淑慧、劉志惟、林英傑、吳明佩、陳康善、陳子翔、陳婉瑜、劉邦睿、張哲嘉、鄭勝然、邱俊樺、張淑峰、溫子芸、郭欣潔、楊俊諺、林冠宏、林慧杰、許宏雨、唐佳姍、林學人、林佳琪、李士豪、詹勇芝、葉珮君、林平仰、葉子揚、蔡茹佩、涂興意、楊惠雯、吳俊瑋、黃志宇、歐莉雯、鄭裕發、倪聖文、鄭舒卉、林國峰、顏旻甫、林忠翰、黃康屏、許麗珠、徐巧諭、曹原蘋、潘慧娟、邱郁涵、杜鎮海、許雅惠、賴麗娟、黃怡婷、簡志伶、陳皓合、陳駿倩、陳柏一、盧憲亦、徐婉皓、李語豐、林珍琴、葉怡名、蘇宥然、林季鳳、林佩喜、劉吉龍、陳慶季、蔡慧萍、陳怡如、彭中吉、林美雲、蔡瓊慧、吳惠萍、陳淑恩、林瑞吟、趙惠敏、劉怡芳、黃哲銘、盧惠雯、周靜怡、郭雅雯、賴嘉雯、張宇軒、陳雅竹、李俊亨、黃夙勳、盧彥安、陳正源、許堅鳳、黃婉婷、林承毓、童淑梅、曾君豪、奚晏凱、李伊婷、王冠宇、陳偉哲、林麗卿、吳怡奇、張聖文、王寧淑、許家銘、彭怡婷、王靜怡、林筱涵、陳郁康、陳麗萍、陳韋云、林柏發、陳振綸、吳向潔、陳賢雲、陳家瑩、王文傑、陳鈞福、陳建新、周怡芳、王智仁、王文白、李玟華、林孟柏、錢承翰、黃靜丹、林家勇、陳啟偉、汪淑芳、陳玟昀、吳欣裕、蔡雅婷、吳佩夫、李國翰、黎曉仁、藍家偉、陳怡珊、張怡云、劉以劭、周崇倩、郭俊銘、陳佩珊、柳軒卿、陳家維、劉雅馨、劉奕萍、李明翰、景怡君、陳雅茹、楊宗翰、陳瑞韋、蕭紹蓉、陳燕孜、王雅映、陳秀娟、顏凱甫、阮瑤秋、邱家綺、林玉樺、姚威廷、屠佩琪、黃淑惠、李依萍、李詩侑、孫建剛、蔡雅梅、韓彥霖、謝冠宇、陳玟奇、方怡政、曾法喜、李思順、趙旺昇、倪初裕、涂佩珊、梁雅萍、林靜來、李亦音、陳木剛、吳佳玲、洪鴻宇、邱宛梅、蔡子翔、黃歡淑、張儀興、許書瑋、魏宜芳、吳彥宇、吉山璇、楊勝傑、衡靜怡、張家偉、侯山夫、劉哲宇、溫希茵、李哲維、葉宜君、陸茂伯、孫芃良、王俊念、柯雅延、張秀丹、林彥霖、陳俊富、林建宏、李昀松、李佩云、陳綺寧、楊幸茂、蔡志樂、張仁伸、溫詩季、黃俊盈、王建宏、楊詩雅、劉玉筠、蔣乃憲、戴家豪、陳俊佑、陳珮亦、柳禮寧、陳兆珊、林正意、許欣怡、張蓉生、黃俐發、黃水妍、朱威廷、王昱谷、陳于婷、黃俊俊、方宇琦、馮健蓮、李倍義、那建志、張惟伯、林素貞、李家瑜、劉美恩、林勝竹、陸秉勳、溫治筠、黃雅惠、李瓊冰、周惠香、唐欣怡、張炳帆、蘇姿吟、林慧如、趙光亦、金慶昆、黃心陽、陳志孜、趙陽豐、王正希、吳姿念、謝皓綸、賴佳穎、林美惠、李國偉、黃彥夫、傅詩雅、王紫夫、劉大達、羅建宏、劉家劭、周秀玲、陳禮皓、許添柔、白冠良、孫美玲、蔡佩蓉、吳瑞萍、蕭承穎、王鎮榮、林志賢、蔡馥善、烏昆菱、劉嘉玲、沈瑞新、林書瑋、林竣傑、宗皇堅、胡萬貞、徐雅雯、高昆紋、郭仁豪、蘇翠智、溫禮蓮、吳俊維、周育如、盧建志、陳建智、王冠斌、幸憲奇、馮浩麟、曾祖喜、張雅惠、黃勇菱、許苑凌、林勝廷、胡怡文、蕭玉昌、陳尚丞、陸玉玲、吳意婷、郭長恩、陸奕翔、王玲霞、黃莉康、黃偉琴、陳昭原、戴俊達、陳智柏、楊宗達、蔡白信、梁岱育、王富天、李哲元、吳于婷、賴仁一、唐薇蓉、陳威月、許勇龍、王美威、宋政宏、吳喜元、晏佩珊、吳惠玲、李宛臻、張夙易、李麗華、黃姵心、姚明哲、潘武依、劉與修、高哲瑋、蔡雅祥、陳仁旺、黃文麟、劉冠霖、黃科源、張世霞、楊宗明、陳志星、王詩惟、唐薇學、汪苑瑜、王貴松、陳佩劭、陳柏勳、顏景翔、鄭育儒、何忠財、張翔綸、賴佳慧、李武孜、胡家瑜、陳佳儀、黃玉智、陳嘉琪、郭吟映、劉秀慧、陳冠琇、謝宗翰、張庭義、黃新潔、陳庭瑋、曾雅雯、陳嘉亦、劉家元、詹陽伸、江冠中、吳興沛、林美惠、葉怡婷、林君瑄、蔡志富、周嘉祥、蔡孝軍、陳永盛、張瑋均、洪星聖、宋碧中、楊怡安、洪國啟、蘇立夢、鄭美娟、陳志瑋、沈惠雯、顏俐紋、林鎮利、李俊宏、陳俊賢、慕昱妏、林子雲、蔡明英、黃文哲、姜欣怡、賴法翰、莫怡伶、林志文、邱白綺、李珮定、涂淑慧、蘇歡萍、姜宜芳、蘇柏豪、劉惠雯、許義桓、王明弘、宋康原、吳馨汝、方喜劭、王俊瑋、柳宥柔、陳惟臻、白怡文、李琳毓、張韋成、李美念、陳韻皓、簡詩臻、王維孜、潘芷汝、林淑玲、張毓平、溫柔盛、陳瑩惟、張卓綸、郭宛瑄、韋竹雨、陳宥沛、白伶財、林奕弘、林雨輝、趙淑娟、王佳琳、李佩宣、陳孟儒、郎耀文、陳淑婷、柏建宏、許俐美、張雅昌、林嘉琦、張怡欣、林宜臻、蔡宗群、謝炳雅、韓素貞、林奕廷、蔡宜霞、蔡美雲、楊松淳、楊廷易、林奕男、黃詩瑋、孫岱妹、林雅惠、杜惠文、陳健瑩、金聿婷、陳予宸、溫怡菁、黃彥翔、林雅惠、黃筱能、林佳一、林麗玲、劉燕惟、黃惠法、劉柏翰、吳政成、王淑娟、袁涵然、葛志峰、藍馨仰、白辛光、林俊全、劉佳蓮、楊淑芬、陳威智、沈怡文、陳慶妤、陳秀美、黃峻和、王宇卿、潘佩珊、黃麗剛、郭文傑、梁曉玲、余珍薇、陳瑞任、陳宗毅、林郁娟、劉建宏、黃鈺倫、陳彥茜、謝佳謙、林禹瑋、李淑芳、林建勳、蔡虹萱、謝燕恆、李世昌、李姵君、張燦娟、鄭嘉惠、黃耀奇、杜若香、李鴻恩、黃建智、吳玟君、許均堯、蔡倫韻、黃敏洋、黃志忠、陳珮蘋、潘雅慧、陳彥沛、夏婉全、吳禹茵、王淑芬、陳宛勳、鄭俊男、林善堅、林喬妏、李伯惠、陳奕貴、張成亨、林佳綸、郭怡君、劉佩如、劉科育、吳梅綠、李淑芬、陳淑華、顏芷瑞、李承江、郝育瑋、曾又平、吳松泉、陳志峰、陳豐玉、張欣潔、趙佳慧、黃怡靜、梁雅萍、林佩君、楊昱宏、呂力凌、張哲湖、賴瑋婷、楊智仁、沈鈺婷、王怡臻、蔡宜意、許璇夢、王美玲、韓意鳳、許育亞、蕭裕翔、童嘉琪、賴怡靜、金右安、王麗蘋、張佩蓉、陳菁喬、鄭倩綺、蘇鈞香、陳志宏、陳柏峰、蔡士凱、陳惠雯、林俊人、陳松泰、蔡秀娟、鞏妍其、葉怡君、林雅婷、程峻瑋、陳俊堅、郭采芬、馮姿禎、陳韋成、彭益幸、劉景翔、林采凱'''
names = ['陳珮如', '楊建豪', '林銘江', '黃嘉為', '黃尹善', '王淑慧', '侯忠翰', '盧巧火', '謝孟惠', '柯鈺容', '蔡翔富', '陳祥希', '張冠宏', '張嘉劭', '鄧揚原', '蔡俊吉', '李佳源', '陳文豪', '楊宜芳', '劉詩婷', '陳品禾', '李佳仲', '許佳蓉', '李宣雅', '張家欣', '陳詠柏', '魏剛彥', '蔡佳芷', '林嘉吟', '曹玉婷', '蔡詩婷', '冉怡芬', '黃欣穎', '韓惠文', '胡威廷', '黃香馨', '賴佳玲', '宋俊宏', '黃建陽', '曾淑媛', '林智偉', '屠益枝', '陳朝妹', '彭世傑', '詹夢璇', '張祐生', '倪若康','杜靜宜', '黃俊順', '施坤鴻', '王智鈺', '柳書瑋', '陳士豪', '慕興羽', '張星宇', '謝孟汝', '陳怡君', '許姿妤', '楊家瑜', '陳依仲', '陳婉菁', '李嘉玲', '李玄弘', '蘇振豪', '游恭蓁', '劉美玲', '李政宏', '張中揚', '李育辛', '林彥瑜', '甘家弘', '翁詩婷', '林怡萱', '林怡男', '劉伶寧', '陳韻剛', '潘堅方', '韓寶為', '陳英傑', '瞿正映', '方宜臻', '李志偉', '陳逸竹', '鄧韋伶', '王宛儒', '林怡芳', '謝雅雯', '李美惠', '王心怡', '黃正威', '張雅玲', '廖偉易', '蘇正偉', '陳治啟', '陳志穎', '陳姵君', '陳彥翔', '黎佳玲', '洪妤均', '蔡建宇', '郭芳儀', '劉淑珍', '吳中珊', '姜雅萍', '虞宥柏', '黃志生', '林美玲', '陳政瑞', '陳政孝', '戴奇妮', '黃柏珊', '賴宇紫', '林婉婷', '蔡宜辰', '陳世謙', '鄭銘源', '楊怡如', '林允俐', '金怡君', '謝慧茹', '蔡忠雨', '林家誠', '許嘉淳', '葉慧君', '劉郁翔', '張慧敏', '林雅婷', '林建慈', '黃致原', '李菁青', '張昇麟', '郭淑芬', '馬珮雯', '呂雅萍', '鄭淑婷', '方靜豪', '李白能', '陳怡貴', '張宗儒', '葉映菁', '林淑君', '邱致茹', '陳盈芳', '吳宛菱', '郭曼治', '劉康禎', '李靜瑋', '楊子佳', '吳建智', '劉珮君', '陳秀松', '李婉吉', '吳雅筠', '林怡禎', '林卓芷', '陳俐達', '吳天威', '葉介茂', '鄧政霖', '林佳霖', '倪鈺雯', '劉彥亨', '白郁翔', '張昱廷', '劉淑州', '藍欣怡', '林雅源', '王明哲', '王惠如', '李建珠', '張貴正', '李威廷', '連偉婷', '楊秀旺', '林思妤', '幸貴香', '林東隆', '陳山喬', '楊思婷', '齊宗東', '鄭雅惠', '王嘉士', '陳國齊', '黃舜文', '張淑松', '常麗君', '王翔伯', '連禎光', '袁淑娟', '周慧玲', '郭威任', '陳雅文', '王文琇', '蔡奇竹', '呂向謙', '桂菁娥', '林怡君', '鐘智傑', '房心富', '黃宗憲', '張志翔', '蔡海仲', '楊佳發', '陳怡萍', '李珮喜', '李菁禮', '倪依美', '王婉翰', '郭肇中', '李彥堅', '吳翊紋', '張妤政', '林翔茹', '蔡辰鴻', '林佩竹', '黃英傑', '王伶香', '陳怡堯', '蔡依春', '楊晴人', '呂宇天', '余佳霖', '張真慧', '周詠昕', '楊書儀', '張傑堯', '許明輝', '許凱鈞', '張邦瑋', '張易林', '張柏翔', '陳懿堯', '倪堅宸', '王宗霖', '陳雅云', '洪翰峰', '黃淑慧', '林淑雯', '溫人聖', '林忠翰', '王欣怡', '潘哲榮', '林智傑', '陳慈漢', '賴瑜樺', '許雅皓', '錢山珍', '陳志云', '宋思穎', '侯吟汝', '姚志維', '朱佩木', '李怡君', '吳培任', '朱建宏', '侯于倫', '楊怡君', '何冠霖', '王宗儒', '鐘韋伶', '吳雅芳', '翁雅晴', '李定沛', '陳珮君', '莫祐竹', '吳志成', '賴孟君', '林哲苓', '林聿憲', '張雅惠', '陳香啟', '羅星其', '鐘冠萍', '劉冠良', '鄭純凌', '吳佩卿', '陳俊青', '楊志芸', '陳婉君', '王青丞', '何冠傑', '游彥廷', '宋雅慧', '阮家豪', '劉佩琪', '張莉霞', '王彥均', '許丞中', '周佳怡', '陳盈雯', '茹舒涵', '林宜妹', '李政天', '鄭旭珊', '蕭姵恆', '吳伊婷', '劉靜怡', '李博英', '李中紫', '林家豪', '陳品杰', '郭士賢', '何柏翰', '呂智鈞', '林人平', '潘宜潔', '林曉薇', '李婉銘', '甄彥儒', '楊雅春', '吳麗娟', '張駿雅', '黃阿儀', '陳景龍', '陳昶睿', '賴慈夫', '郭晉良', '陳曉倫', '馬則宏', '吳佩妮', '陳欣怡', '黃信宏', '張姵清', '劉伶屏', '焦鎮木', '李欣裕', '賴建如', '陳怡君', '趙俞延', '楊雅宸', '彭可辰', '楊智桂', '鞠惠如', '陳弘綸', '黃淑強', '林國維', '郭美花', '王雅婷', '盧彥安', '林淑福', '林俞火', '李怡如', '陳胤甫', '陳政純', '李曼皓', '趙俞南', '林依喜', '陳玉珠', '張珮瑜', '陳建霞', '吳珮君', '周佳穎', '林蕙淑', '王彥勇', '謝冰良', '張家汝', '張宜欣', '張珮如', '鄭美凌', '王淑美', '許子豐', '黃佳堅', '劉慧玲', '謝志偉', '黃彥虹', '鄭進延', '夏雅婷', '潘偉倫', '陳品霖', '郭亦伶', '王志維', '陳柏士', '陳馥音', '劉冠謙', '張于珊', '王如君', '許宗然', '王昱定', '鄭佩璇', '黃俊昌', '鄭淑娟', '黃文達', '劉哲銘', '邱彥廷', '陳世妮', '賴恩邦', '沈郁婷', '魏家豪', '陳永妤', '王志坤', '陳宛真', '姚怡靜', '蔡志泰', '林江盈', '宋宗毅', '李皓紫', '張昱漢', '劉家祥', '馮淑娟', '蕭珮雯', '葉孝萍', '王銘春', '蘇家齊', '蔡亭忠', '黃鈺韋', '柯昱廷', '林麗玲', '許志毓', '陳欣怡', '吳文俊', '郭家偉', '劉昆坤', '胥沛尹', '黃國強', '劉姿吟', '唐鈺雯', '陳美貞', '胡台群', '張雅民', '陳秋如', '王秀玲', '謝守山', '陳琦賢', '毛欣儀', '張彥志', '陳淑樺', '楊品憲', '張翊美', '吳靜城', '陳宏柏', '邱念典', '林啟舜', '潘秋睿', '謝燦樂', '韓康鳳', '謝美君', '詹翰玄', '朱舜駿', '游國維', '翁紫禎', '沈政廷', '陳靜明', '陳牧琬', '劉名汝', '羅景儒', '邱婷婷', '童志文', '張祥幸', '陳美芬', '林綺羽', '施嘉偉', '杜希岳', '崔雅婷', '陳愛韻', '張哲榮', '黃智傑', '陳維信', '李怡如', '陳明翰', '吳柏芝', '張沛妍', '林政甫', '陳科燦', '郭嘉文', '林怡俊', '王筱玲', '胡淑芳', '張明杰', '洪郁翔', '黃俊銘', '倪玲軍', '王怡元', '吳佳穎', '林慧東', '李佩軍', '吳惠珍', '李淑娟', '楊伯原', '陳玟松', '蔡璇松', '黃婷堅', '陳長齊', '米星琪', '王姿妤', '黃香能', '郭怡順', '王志偉', '陳如君', '鄭京傑', '陳怡靜', '王世仰', '張宛玲', '林群欣', '許俊諺', '陳宜凡', '楊于真', '盧孟儒', '楊于婷', '倪民淳', '楊婉玲', '許淑婷', '廖筱涵', '吳宜姍', '余廷育', '陳俊傑', '李政合', '毛懿汝', '韓傑松', '李馨文', '黃順韋', '張馨紫', '張馨喬', '鄭家齊', '黃佳琪', '林明琬', '林文賢', '應志宏', '曹婷均', '孫書豪', '陳美慧', '吳惠珊', '陳怡延', '胡堯蘭', '黃靜苓', '韓辰聿', '潘吟仁', '張佳恬', '張淑輝', '陳怡君', '柴宇欣', '趙怡如', '潘豐年', '林育旭', '闞姿君', '吳珮雲', '柯欣潔', '蔡合全', '許剛杰', '曹伊婷', '汪苑甫', '蔡念雪', '李欣治', '江惠茹', '潘彥宇', '黃美華', '侯憲卿', '吳雅雯', '李宗忠', '林俊吉', '涂家銘', '劉淑柏', '張怡樺', '陳盈妹', '王韻如', '汪南玲', '劉建海', '許雅瑋', '謝宛臻', '吳德沛', '林良易', '杭政哲', '簡琦羽', '陳浩嘉', '倪俊男', '呂雅筑', '劉彥桓', '沈采恆', '高建成', '黃怡云', '李佳雯', '白俊秀', '蔡得梅', '吳怡茹', '楊裕恒', '黃明雲', '李人良', '王彥妮', '蔡如友', '林曼星', '陳俊良', '蘇卉玟', '皮志軒', '蔡貞發', '李孟儒', '陳盈聖', '唐淳然', '馮宗翰', '袁偉智', '張玉喬', '王玫一', '梁郁文', '沈怡靜', '蕭家瑋', '蔡宜法', '張宜臻', '廣家弘', '陳金娟', '張思妤', '李瑋婷', '王啟璇', '蕭岳火', '陳雅馨', '鄭博鈞', '陳鳳珠', '黃信發', '侯維倫', '蔡曉宸', '方佳琪', '苗姿天', '金怡婷', '張淑怡', '陳怡慧', '陳哲禮', '蔡光靖', '張建志', '陳志偉', '李修恩', '鄧瑜昇', '陳茂筠', '陳君恭', '吳宛玲', '陳慧廷', '黃凱妹', '曾鈺婷', '周思婷', '吳文彬', '王于婷', '賴孟修', '王勳仰', '王馨儀', '王妍偉', '劉柏成', '林正定', '夏建志', '傅志祥', '劉宗翰', '謝光奇', '許耀霞', '徐偉哲', '陳惠昌', '藍怡伶', '李永梅', '黃姿文', '楊志文', '陳惠芬', '許佩儒', '許國瑋', '張佑堅', '沈怡潔', '陳怡伶', '蘇淑敏', '丁瑞尹', '黃瓊富', '王嘉琪', '杜欣潔', '吳康珠', '林芳雄', '劉佳茂', '李依岳', '黃惠琪', '陳佳雯', '張偉智', '潘玉婷', '王信豪', '王安琪', '謝毓泉', '宋威琦', '涂致裕', '朱裕翔', '陳瑜依', '王雪瑩', '周家豪', '謝惟花', '黃英傑', '潘少火', '孫馨儀', '夏逸凡', '劉惠雯', '彭玫豪', '王佳火', '陳旭芝', '石雅婷', '陳美文', '郭文祥', '林岱夫', '曾恒容', '李宜虹', '施婉婷', '蔡倫蕙', '劉向陽', '陳政宏', '許喜瑜', '謝憲卉', '許哲香', '林欣禮', '張平貞', '廖家皓']

In [97]:
all_names = online_gen_names.replace(' ','').split('、')
all_names = names + all_names
gen_names = list(set(all_names))
gen_names = gen_names[1:]
print(len(gen_names))

18698


In [98]:
newdf = pd.DataFrame(columns=true_df.columns)
# 每則新聞產生幾篇不同的
fake_count = 15
gen_name_count = 0

for index,row in true_df.iterrows():
    names  = ast.literal_eval(row['name'])
    ckip_names = ast.literal_eval(row['ckip_names'])
    content = row['full_content']
    
    for i in range(fake_count):
        for name in names:
            replace_name = gen_names[gen_name_count]
            gen_name_count += 1
            content = content.replace(name,replace_name)
            try:
                ckip_names[ckip_names.index(name)] = replace_name
            except:
                ckip_names.append(replace_name)
            names[names.index(name)] = replace_name
                        
        new_row = {
            'news_ID':row['news_ID'],
            'name' : str(names),
            'full_content' : content,
            'ckip_names' : str(ckip_names)
        }    
        
        
        newdf = newdf.append(new_row, ignore_index=True)
    newdf = newdf.append(row,ignore_index=True)

        
newdf.shape

(5360, 4)

In [99]:
newdf.to_csv('./tbrain/only_have_names_data.csv',index=False)